## This document contains Proof Of Concepts to extract the relevant content using GraphDB

In [2]:
import sys
import os
sys.path.append(".")

In [24]:
from langchain_experimental.graph_transformers import LLMGraphTransformer
from langchain_openai import AzureChatOpenAI
from dotenv import load_dotenv
load_dotenv()

True

In [4]:
llm = AzureChatOpenAI(
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    deployment_name=os.environ["AZURE_DEPLOYMENT_NAME"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
    default_headers={
            "fds-message-id": "14485820-1e81-4f0b-a708-d386d4672a81",
            "fds-conversation-id": "9616f92c-070c-4f3b-8e51-5321b472b24c"
        }
)
llm_transformer = LLMGraphTransformer(llm=llm)

In [22]:
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
username=os.environ["NEO4J_USERNAME"]
password=os.environ["NEO4J_PASSWORD"]
url=os.environ["NEO4J_URI"]
graph = Neo4jGraph(username=username, password=password, url=url)

In [6]:
from langchain_core.documents import Document

text = """
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for several years;
in fact, Mrs. Dursley pretended she didn't have a sister, because her
sister and her good-for-nothing husband were as unDursleyish as it was
possible to be. The Dursleys shuddered to think what the neighbors would
say if the Potters arrived in the street. The Dursleys knew that the
Potters had a small son, too, but they had never even seen him. This boy
was another good reason for keeping the Potters away; they didn't want
Dudley mixing with a child like that.
"""
documents = [Document(page_content=text)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Mr. Dursley', type='Person'), Node(id='Mrs. Dursley', type='Person'), Node(id='Number Four, Privet Drive', type='Place'), Node(id='Grunnings', type='Organization'), Node(id='Dudley Dursley', type='Person'), Node(id='Mrs. Potter', type='Person'), Node(id='Mr. Potter', type='Person'), Node(id='The Potters', type='Group'), Node(id='The Dursleys', type='Group')]
Relationships:[Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Mrs. Dursley', type='Person'), type='SPOUSE'), Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Number Four, Privet Drive', type='Place'), type='RESIDENCE'), Relationship(source=Node(id='Mrs. Dursley', type='Person'), target=Node(id='Number Four, Privet Drive', type='Place'), type='RESIDENCE'), Relationship(source=Node(id='Mr. Dursley', type='Person'), target=Node(id='Grunnings', type='Organization'), type='DIRECTOR'), Relationship(source=Node(id='The Dursleys', type='Group'), target=Node(id='Dudley Du

In [40]:
#help(AzureChatOpenAI)

In [14]:
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [20]:
from langchain_openai import AzureOpenAIEmbeddings

embeddings = AzureOpenAIEmbeddings(
    model="text-embedding-ada-002",
    openai_api_version=os.environ["OPENAI_API_VERSION"],
    openai_api_type=os.environ["OPENAI_API_TYPE"],
    azure_endpoint=os.environ["AZURE_OPENAI_ENDPOINT"],
    openai_api_key=os.environ["AZURE_OPENAI_API_KEY"],
)

In [25]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('592d73fc.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [26]:
query = "Who is Dudley?"

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)


text: 
Mr. and Mrs. Dursley, of number four, Privet Drive, were proud to say
that they were perfectly normal, thank you very much. They were the last
people you'd expect to be involved in anything strange or mysterious,
because they just didn't hold with such nonsense.
Mr. Dursley was the director of a firm called Grunnings, which made
drills. He was a big, beefy man with hardly any neck, although he did
have a very large mustache. Mrs. Dursley was thin and blonde and had
nearly twice the usual amount of neck, which came in very useful as she
spent so much of her time craning over garden fences, spying on the
neighbors. The Dursleys had a small son called Dudley and in their
opinion there was no finer boy anywhere.
The Dursleys had everything they wanted, but they also had a secret, and
their greatest fear was that somebody would discover it. They didn't
think they could bear it if anyone found out about the Potters. Mrs.
Potter was Mrs. Dursley's sister, but they hadn't met for sever

In [53]:
[text[i*chunk: (i+1)*chunk + 1] for i in range(10)]

['\nThe information in this preliminary pricing supplement is not complete and may be changed. This preliminary pricing supplement and the accompanying prospectus and prospectus supplement do not constitute an offer to sell these securities, and we are not soliciting an offer to buy these securities in any state where the offer or sale is not permitted.\nSubject to Completion\nPreliminary Pricing Supplement dated June 3, 2024\nPreliminary Pricing Supplement\n(To the Prospectus dated May 23, 2022 and the Prospectus Supplement\ndated June 27, 2022)\nFiled Pursuant to Rule 424(b)(2)\nRegistration No. 333-265158\n\nUS$\nFIXED RATE CALLABLE NOTES DUE JUNE 24, 2027\nPrincipal Amount:\nUS$\nIssuer:\nBarclays Bank PLC \nIssue Price:\n100.00%\nSeries:\nGlobal Medium-Term Notes, Series A\nPayment at Maturity: \nIf you hold the Notes to maturity, you will receive 100% of your principal, subject to the creditworthiness of Barclays Bank PLC and the exercise of any U.K. Bail-in Power by the relevant

In [46]:
text = """
The information in this preliminary pricing supplement is not complete and may be changed. This preliminary pricing supplement and the accompanying prospectus and prospectus supplement do not constitute an offer to sell these securities, and we are not soliciting an offer to buy these securities in any state where the offer or sale is not permitted.
Subject to Completion
Preliminary Pricing Supplement dated June 3, 2024
Preliminary Pricing Supplement
(To the Prospectus dated May 23, 2022 and the Prospectus Supplement
dated June 27, 2022)
Filed Pursuant to Rule 424(b)(2)
Registration No. 333-265158

US$
FIXED RATE CALLABLE NOTES DUE JUNE 24, 2027
Principal Amount:
US$
Issuer:
Barclays Bank PLC 
Issue Price:
100.00%
Series:
Global Medium-Term Notes, Series A
Payment at Maturity: 
If you hold the Notes to maturity, you will receive 100% of your principal, subject to the creditworthiness of Barclays Bank PLC and the exercise of any U.K. Bail-in Power by the relevant U.K. resolution authority. 
Any payment on the Notes is not guaranteed by any third party and is subject to both the creditworthiness of the Issuer and to the exercise of any U.K. Bail-in Power by the relevant U.K. resolution authority. If Barclays Bank PLC were to default on its payment obligations or become subject to the exercise of any U.K. Bail-in Power (or any other resolution measure) by the relevant U.K. resolution authority, you might not receive any amounts owed to you under the Notes. See “Consent to U.K. Bail-in Power” and “Selected Risk Factors” in this pricing supplement and “Risk Factors” in the accompanying prospectus supplement for more information.
Original Trade Date:
June 20, 2024
Maturity Date:
June 24, 2027 subject to Redemption at the Option of the Issuer (as set forth below).
Original Issue Date:
June 24, 2024
Denominations:
Minimum denominations of US$1,000 and integral multiples of US$1,000 thereafter.
Interest Rate Type:
Fixed Rate
Interest Rate:
5.25% per annum
Consent to U.K. Bail-in Power
Notwithstanding and to the exclusion of any other term of the Notes or any other agreements, arrangements or understandings between Barclays Bank PLC and any holder or beneficial owner of the Notes, by acquiring the Notes, each holder and beneficial owner of the Notes acknowledges, accepts, agrees to be bound by, and consents to the exercise of, any U.K. Bail-in Power by the relevant U.K. resolution authority. See “Consent to U.K. Bail-in Power” on page PS-3 of this pricing supplement. 
Interest Payment Dates:
Payable semi-annually in arrears on the 24th day of each June and December, commencing on December 24, 2024 and ending on the Maturity Date (or the Early Redemption Date, if applicable). For the avoidance of doubt, the final Interest Payment Date is scheduled to occur on the Maturity Date, subject to an earlier Redemption at the Option of the Issuer.
Redemption at the Option of the Issuer:
We may redeem your Notes, in whole or in part, at the Redemption Price set forth below on the 24th day of each March, June, September and December, commencing on June 24, 2026 to but excluding the Maturity Date, provided we give at least five business days’ prior written notice to the trustee. If we exercise our redemption option, the Interest Payment Date on which we so exercise will be referred to as the “Early Redemption Date”.
Redemption Price:
If we exercise our redemption option, you will receive on the Early Redemption Date 100% of the principal amount together with any accrued and unpaid interest to but excluding the Early Redemption Date.
Interest Payment Amount:
For each Interest Period, the interest payment amount per $1,000 principal amount Note will be calculated as follows:
$1,000 × Interest Rate × (days in Interest Period/360)
where the number of days in the Interest Period will be determined based on a 30/360 Day Count Convention.
[Terms of Note continue on the following page]
Price to Public (1), (2)
Agent’s Commission (2)
Proceeds to Barclays Bank PLC
Per Note
100.00%
0.60%
99.40%
Total
$
$
$
(1) Because dealers who purchase the Notes for sale to certain fee-based advisory accounts may forgo some or all selling concessions, fees or commissions, the public offering price for investors purchasing the Notes in such fee-based advisory accounts may be between $994.00 and $1,000 per Note. Investors that hold their Notes in fee-based advisory or trust accounts may be charged fees by the investment advisor or manager of such account based on the amount of assets held in those accounts, including the Notes.
(2) Barclays Capital Inc. will receive commissions from the Issuer up to $6.00 per $1,000 principal amount, and may retain all or a portion of these commissions or use all or a portion of these commissions to pay variable selling concessions or fees to other dealers. Barclays Capital Inc. may forgo all or a portion of these commissions and sell Notes to certain institutional accounts at an offering price between $994.00 and $1,000 per Note.
Investing in the Notes involves a number of risks. See “Risk Factors” beginning on page S-9 of the prospectus supplement and “Selected Risk Factors” beginning on page PS-4 of this pricing supplement.
The Notes will not be listed on any U.S. securities exchange or quotation system. Neither the Securities and Exchange Commission nor any state securities commission has approved or disapproved of these securities or determined that this pricing supplement is truthful or complete. Any representation to the contrary is a criminal offense.
The Notes constitute our direct, unconditional, unsecured and unsubordinated obligations and are not deposit liabilities of either Barclays PLC or Barclays Bank PLC and are not covered by the U.K. Financial Services Compensation Scheme or insured or guaranteed by the U.S. Federal Deposit Insurance Corporation or any other governmental agency of the United States, the United Kingdom or any other jurisdiction.
Interest Period:
The initial Interest Period will begin on, and include, the Original Issue Date and end on, but exclude, the first Interest Payment Date. Each subsequent Interest Period will begin on, and include, the Interest Payment Date for the immediately preceding Interest Period and end on, but exclude, the next following Interest Payment Date. The final Interest Period will end on, but exclude, the Maturity Date (or the Early Redemption Date, if applicable).
Business Day Convention/Day Count Fraction:
Following, unadjusted; 30/360
Business Day:
A Monday, Tuesday, Wednesday, Thursday or Friday that is neither a day on which banking institutions in New York City generally are authorized or obligated by law, regulation, or executive order to be closed.
Settlement:
DTC; Book-entry; Transferable.
Listing:
The Notes will not be listed on any U.S. securities exchange or quotation system.
Agent:
Barclays Capital Inc.
CUSIP/ISIN:
06745U5S5      /     US06745U5S55

You should read this pricing supplement together with the prospectus dated May 23, 2022, as supplemented by the documents listed below relating to our Global Medium-Term Notes, Series A, of which these Notes are a part. This pricing supplement, together with the documents listed below, contains the terms of the Notes and supersedes all prior or contemporaneous oral statements as well as any other written materials including preliminary or indicative pricing terms, correspondence, trade ideas, structures for implementation, sample structures, brochures or other educational materials of ours. You should carefully consider, among other things, the matters set forth under “Risk Factors” in the prospectus supplement, as the Notes involve risks not associated with conventional debt securities. We urge you to consult your investment, legal, tax, accounting and other advisors before you invest in the Notes.
You may access these documents on the SEC website at www.sec.gov as follows (or if such address has changed, by reviewing our filings for the relevant date on the SEC website): 
●   
Prospectus dated May 23, 2022:
http://www.sec.gov/Archives/edgar/data/312070/000119312522157585/d337542df3asr.htm
●   
Prospectus Supplement dated June 27, 2022:
http://www.sec.gov/Archives/edgar/data/0000312070/000095010322011301/dp169388_424b2-prosupp.htm
Our SEC file number is 1-10257. As used in this term sheet, the “Company,” “we,” “us,” or “our” refers to Barclays Bank PLC.
You may revoke your offer to purchase the Notes at any time prior to the time at which we accept such offer by notifying the applicable agent. We reserve the right to change the terms of, or reject any offer to purchase the Notes prior to their issuance. In the event of any changes to the terms of the Notes, we will notify you and you will be asked to accept such changes in connection with your purchase. You may also choose to reject such changes in which case we may reject your offer to purchase.
PS-2
CONSENT TO U.K. BAIL-IN POWER
Notwithstanding and to the exclusion of any other term of the Notes or any other agreements, arrangements or understandings between us and any holder or beneficial owner of the Notes, by acquiring the Notes, each holder and beneficial owner of the Notes acknowledges, accepts, agrees to be bound by, and consents to the exercise of, any U.K. Bail-in Power by the relevant U.K. resolution authority.
Under the U.K. Banking Act 2009, as amended, the relevant U.K. resolution authority may exercise a U.K. Bail-in Power in circumstances in which the relevant U.K. resolution authority is satisfied that the resolution conditions are met. These conditions include that a U.K. bank or investment firm is failing or is likely to fail to satisfy the Financial Services and Markets Act 2000 (the “FSMA”) threshold conditions for authorization to carry on certain regulated activities (within the meaning of section 55B FSMA) or, in the case of a U.K. banking group company that is a European Economic Area (“EEA”) or third country institution or investment firm, that the relevant EEA or third country relevant authority is satisfied that the resolution conditions are met in respect of that entity.
The U.K. Bail-in Power includes any write-down, conversion, transfer, modification and/or suspension power, which allows for (i) the reduction or cancellation of all, or a portion, of the principal amount of, interest on, or any other amounts payable on, the Notes; (ii) the conversion of all, or a portion, of the principal amount of, interest on, or any other amounts payable on, the Notes into shares or other securities or other obligations of Barclays Bank PLC or another person (and the issue to, or conferral on, the holder or beneficial owner of the Notes such shares, securities or obligations); (iii) the cancellation of the Notes and/or (iv) the amendment or alteration of the maturity of the Notes, or amendment of the amount of interest or any other amounts due on the Notes, or the dates on which interest or any other amounts become payable, including by suspending payment for a temporary period; which U.K. Bail-in Power may be exercised by means of a variation of the terms of the Notes solely to give effect to the exercise by the relevant U.K. resolution authority of such U.K. Bail-in Power. Each holder and beneficial owner of the Notes further acknowledges and agrees that the rights of the holders or beneficial owners of the Notes are subject to, and will be varied, if necessary, solely to give effect to, the exercise of any U.K. Bail-in Power by the relevant U.K. resolution authority. For the avoidance of doubt, this consent and acknowledgment is not a waiver of any rights holders or beneficial owners of the Notes may have at law if and to the extent that any U.K. Bail-in Power is exercised by the relevant U.K. resolution authority in breach of laws applicable in England.
For more information, please see “Selected Risk Factors—Risks Relating to the Issuer—You May Lose Some or All of Your Investment If Any U.K. Bail-in Power Is Exercised by the Relevant U.K. Resolution Authority” in this pricing supplement as well as “U.K. Bail-in Power,” “Risk Factors—Risks Relating to the Securities Generally—Regulatory action in the event a bank or investment firm in the Group is failing or likely to fail, including the exercise by the relevant U.K. resolution authority of a variety of statutory resolution powers, could materially adversely affect the value of any securities” and “Risk Factors—Risks Relating to the Securities Generally—Under the terms of the securities, you have agreed to be bound by the exercise of any U.K. Bail-in Power by the relevant U.K. resolution authority” in the accompanying prospectus supplement.
PS-3
SELECTED RISK FACTORS
An investment in the Notes involves significant risks. You should read the risks summarized below in connection with, and the risks summarized below are qualified by reference to, the risks described in more detail in the “Risk Factors” section beginning on page S-7 of the prospectus supplement. We urge you to consult your investment, legal, tax, accounting and other advisers and to invest in the Notes only after you and your advisors have carefully considered the appropriateness of an investment in the Notes in light of your particular circumstances.
Risks Relating to the Notes Generally
●   
Appropriateness of the Notes for Investment—You should reach a decision whether to invest in the Notes after carefully considering, with your advisors, the appropriateness of the Notes in light of your investment objectives and the specific information set out in this pricing supplement, the prospectus supplement and the prospectus. Neither the Issuer nor Barclays Capital Inc. makes any recommendation as to the appropriateness of the Notes for investment.
●   
The Notes Will Be Subject to Redemption at Our Option—We may redeem the Notes prior to the Maturity Date on the dates as indicated on the cover page hereof. If you intend to purchase the Notes, you must be willing to have your Notes redeemed early. We are generally more likely to redeem the Notes during periods when we expect that interest will accrue on the Notes at a rate that is greater than that which we would pay on our traditional interest-bearing deposits or debt securities having a maturity equal to the remaining term of the Notes. In contrast, we are generally less likely to redeem the Notes during periods when we expect interest to accrue on the Notes at a rate that is less than that which we would pay on those instruments. If we redeem the Notes prior to the Maturity Date, accrued interest will be paid on the Notes until such early redemption, but you will not receive any future interest payments from the Notes redeemed, and you may be unable to reinvest your proceeds from the redemption in an investment with a return that is as high as the return on the Notes would have been if they had not been redeemed. In addition, our right to redeem the Notes may also adversely impact your ability to sell your Notes and the price at which they may be sold.
Risks Relating to the Issuer
●   
Issuer Credit Risk—The Notes are our unsecured and unsubordinated debt obligations, and are not, either directly or indirectly, an obligation of any third party. Any payment to be made on the Notes, including any repayment of principal, depends on our ability to satisfy our obligations as they come due. As a result, the actual and perceived creditworthiness of Barclays Bank PLC may affect the market value of the Notes and, in the event we were to default on our obligations, you may not receive the payment due at maturity or any other amounts owed to you under the terms of the Notes.
●   
You May Lose Some or All of Your Investment If Any U.K. Bail-in Power Is Exercised by the Relevant U.K. Resolution Authority—Notwithstanding and to the exclusion of any other term of the Notes or any other agreements, arrangements or understandings between Barclays Bank PLC and any holder or beneficial owner of the Notes, by acquiring the Notes, each holder and beneficial owner of the Notes acknowledges, accepts, agrees to be bound by, and consents to the exercise of, any U.K. Bail-in Power by the relevant U.K. resolution authority as set forth under “Consent to U.K. Bail-in Power” in this pricing supplement. Accordingly, any U.K. Bail-in Power may be exercised in such a manner as to result in you and other holders and beneficial owners of the Notes losing all or a part of the value of your investment in the Notes or receiving a different security from the Notes, which may be worth significantly less than the Notes and which may have significantly fewer protections than those typically afforded to debt securities. Moreover, the relevant U.K. resolution authority may exercise the U.K. Bail-in Power without providing any advance notice to, or requiring the consent of, the holders or beneficial owners of the Notes. The exercise of any U.K. Bail-in Power by the relevant U.K. resolution authority with respect to the Notes will not be a default or an Event of Default (as each term is defined in the senior debt securities indenture) and the trustee will not be liable for any action that the trustee takes, or abstains from taking, in either case, in accordance with the exercise of the U.K. Bail-in Power by the relevant U.K. resolution authority with respect to the Notes. See “Consent to U.K. Bail-in Power” in this pricing supplement as well as “U.K. Bail-in Power,” “Risk Factors—Risks Relating to the Securities Generally—Regulatory action in the event a bank or investment firm in the Group is failing or likely to fail, including the exercise by the relevant U.K. resolution authority of a variety of statutory resolution powers, could materially adversely affect the value of any securities” and “Risk Factors—Risks Relating to the Securities Generally—Under the terms of the securities, you have agreed to be bound by the exercise of any U.K. Bail-in Power by the relevant U.K. resolution authority” in the accompanying prospectus supplement.
PS-4
Risks Relating to Conflict of Interest
●   
We and Our Affiliates May Engage in Various Activities or Make Determinations That Could Materially Affect Your Notes in Various Ways and Create Conflicts of Interest—We and our affiliates play a variety of roles in connection with the issuance of the Notes, as described below. In performing these roles, our and our affiliates’ economic interests are potentially adverse to your interests as an investor in the Notes.
In connection with our normal business activities and in connection with hedging our obligations under the Notes, we and our affiliates make markets in and trade various financial instruments or products for our accounts and for the account of our clients and otherwise provide investment banking and other financial services with respect to these financial instruments and products. These financial instruments and products may include securities, derivative instruments or assets that may relate to interest rates. In any such market making, trading and hedging activity, and other services, we or our affiliates may take positions or take actions that are inconsistent with, or adverse to, the investment objectives of holders of the Notes. We and our affiliates have no obligation to take the needs of any buyer, seller or holder of the Notes into account in conducting these activities. Such market making, trading and hedging activity, investment banking and other financial services may negatively impact the value of the Notes. 
In addition, the role played by Barclays Capital Inc., as the agent for the Notes, could present significant conflicts of interest with the role of Barclays Bank PLC, as issuer of the Notes. For example, Barclays Capital Inc. or its representatives may derive compensation or financial benefit from the distribution of the Notes. Furthermore, we and our affiliates establish the offering price of the Notes for initial sale to the public, and the offering price is not based upon any independent verification or valuation.
In addition to the activities described above, we will also act as the Calculation Agent for the Notes. As Calculation Agent, we will make any determinations necessary to calculate any payments on the Notes. In making these determinations, we may be required to make certain discretionary judgments. In making these discretionary judgments, our economic interests are potentially adverse to your interests as an investor in the Notes, and any of these determinations may adversely affect any payments on the Notes.
Risks Relating to the Value of the Notes and the Secondary Market
●   
Certain Built-In Costs Are Likely to Adversely Affect the Value of the Notes Prior to Maturity—While the payment at maturity described in this pricing supplement is based on the full principal amount of your Notes, the original issue price of the Notes includes any Agent’s commission and the cost of hedging our obligations under the Notes through one or more of our affiliates. As a result, the price, if any, at which Barclays Capital Inc. and other affiliates of Barclays Bank PLC may be willing to purchase Notes from you in secondary market transactions will likely be lower than the price you paid for your Notes, and any sale prior to the Maturity Date could result in a substantial loss to you. 
●   
Lack of Liquidity—The Notes will not be listed on any securities exchange. Barclays Capital Inc. and other affiliates of Barclays Bank PLC intend to make a secondary market for the Notes but are not required to do so, and may discontinue any such secondary market making at any time, without notice. Barclays Capital Inc. may at any time hold unsold inventory, which may inhibit the development of a secondary market for the Notes. Even if there is a secondary market, it may not provide enough liquidity to allow you to trade or sell the Notes easily. Because other dealers are not likely to make a secondary market for the Notes, the price at which you may be able to trade your Notes is likely to depend on the price, if any, at which Barclays Capital Inc. and other affiliates of Barclays Bank PLC are willing to buy the Notes. The Notes are not designed to be short-term trading instruments. Accordingly, you should be able and willing to hold your Notes to maturity.
●   
Many Economic and Market Factors Will Impact the Value of the Notes—The value of the Notes will be affected by a number of economic and market factors that may either offset or magnify each other, including:
o   
the time to maturity of the Notes;
o   
interest and yield rates in the market generally;
o   
inflation and expectations concerning inflation;
o   
a variety of economic, financial, political, regulatory or judicial events; 
o   
supply and demand for the Notes; and
o   
our creditworthiness, including actual or anticipated downgrades in our credit ratings.
PS-5
TAX CONSIDERATIONS
You should review carefully the sections in the accompanying prospectus supplement entitled “Material U.S. Federal Income Tax Consequences—Tax Consequences to U.S. Holders—Notes Treated as Indebtedness for U.S. Federal Income Tax Purposes” and, if you are a non-U.S. holder, “—Tax Consequences to Non-U.S. Holders.” The discussion below applies to you only if you are an initial purchaser of the Notes; if you are a secondary purchaser of the Notes, the tax consequences to you may be different. In the opinion of our special tax counsel, Davis Polk & Wardwell LLP, the Notes should be treated as debt instruments for U.S. federal income tax purposes and, based on representations provided by us, the Notes should be treated as issued without original issue discount. The remainder of this discussion assumes that this treatment is correct. 
Interest paid on the Notes will generally be taxable to you as ordinary income at the time it accrues or is received, in accordance with your method of tax accounting. Upon a sale or exchange (including upon early redemption or redemption at maturity), you will generally recognize taxable gain or loss equal to the difference between the amount realized on the sale or exchange (not including any amount attributable to accrued but unpaid interest) and your tax basis in the Notes, which will generally equal the amount you paid to acquire the Notes. This gain or loss will generally be long-term capital gain or loss if you have held the Notes for more than one year. The deductibility of capital losses is subject to limitation.
The discussions above and in the accompanying prospectus supplement do not address the consequences to taxpayers subject to special tax accounting rules under Section 451(b).
Non-U.S. holders. We do not believe that non-U.S. holders should be required to provide a Form W-8 in order to avoid 30% U.S. withholding tax with respect to the interest payments, although the Internal Revenue Service could challenge this position. However, non-U.S. holders should in any event expect to be required to provide appropriate Forms W-8 or other documentation in order to establish an exemption from backup withholding, as described under the heading “—Information Reporting and Backup Withholding” in the accompanying prospectus supplement. If any withholding is required, we will not be required to pay any additional amounts with respect to amounts withheld.
You should consult your tax advisor regarding the U.S. federal tax consequences of an investment in the Notes, as well as tax consequences arising under the laws of any state, local or non-U.S. taxing jurisdiction.
PS-6
SUPPLEMENTAL PLAN OF DISTRIBUTION
We will agree to sell to Barclays Capital Inc. (the “Agent”), and the Agent will agree to purchase from us, the principal amount of the Notes, and at the price, specified on the cover of the related pricing supplement, the document that will be filed pursuant to Rule 424(b) containing the final pricing terms of the Notes. The Agent will commit to take and pay for all of the Notes, if any are taken.
We expect that delivery of the Notes will be made against payment for the Notes on the Original Issue Date, which is more than one business day following the Original Trade Date. Notwithstanding anything to the contrary in the accompanying prospectus supplement, under Rule 15c6-1 of the Securities Exchange Act of 1934, as amended, trades in the secondary market generally are required to settle in one business day, unless the parties to any such trade expressly agree otherwise. Accordingly, purchasers who wish to trade the Notes on any date prior to one business day before delivery will be required to specify alternative settlement arrangements to prevent a failed settlement and should consult their own advisor.
PS-7
"""
n = len(text)
chunk = n//10
documents = [Document(page_content=text[i*chunk: (i+1)*chunk]) for i in range(10)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Preliminary Pricing Supplement', type='Document'), Node(id='Prospectus', type='Document'), Node(id='Prospectus Supplement', type='Document'), Node(id='June 3, 2024', type='Date'), Node(id='May 23, 2022', type='Date'), Node(id='June 27, 2022', type='Date'), Node(id='Rule 424(B)(2)', type='Regulation'), Node(id='333-265158', type='Registration number'), Node(id='Us$', type='Currency'), Node(id='Fixed Rate Callable Notes', type='Financial instrument'), Node(id='June 24, 2027', type='Date'), Node(id='Barclays Bank Plc', type='Organization'), Node(id='100.00%', type='Percentage'), Node(id='Global Medium-Term Notes, Series A', type='Financial instrument series'), Node(id='June 20, 2024', type='Date'), Node(id='June 24, 2024', type='Date'), Node(id='Us$1,000', type='Currency'), Node(id='Fixed Rate', type='Interest rate type'), Node(id='5.25% Per Annum', type='Percentage'), Node(id='U.K. Bail-In Power', type='Regulation'), Node(id='June 24', type='Date'), Node(id='December 24',

In [47]:
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

In [31]:
query = """Please extract the following information from the given document:
 
ISIN: ISIN is a standard security code that is used worldwide to identify specific securities such as bonds, stocks (common and preferred), futures, warrant, rights, trusts, commercial paper, and options.
 
Example Value: US606822DE19
Entity: Issuer is a legal entity that develops, registers and sells securities with the purpose of financing its operations.
 
Example Value: Mitsubishi UFJ Financial Group, Inc.
Deal Type: Deal Type is the security type grouping of the instrument being offered.
 
Example Value: Medium-Term Notes
Seniority: Seniority of a security shows the order of repayment in the event the issuer goes bankrupt or defaults.
 
Example Value: Senior
COCO: Flag for Contingent convertibles, also known as CoCos, are a type of hybrid debt security that can be converted into equity shares if a specified trigger event occurs.
 
Example Value: null
Pledge Status: Indicator whether the debt is secured by a specific asset of the issuer to give protection to the bondholders in case the issuer defaults.
 
Example Value: Unsecured
Issue Price (%): Issue Price is the price at which investors buy the bonds when they are first issued, which will typically be approximately equal to the nominal amount.
 
Example Value: 100.00000
Face Value: Face Value/Par Value is the actual currency amount that each security is worth. Any calculation of redemption, conversion or liquidation of the security is based on Face Value.
 
Example Value: 1000.00000
Min Denomination/Subscription: Min Denomination is the minimum amount an investor must subscribe to in order to purchase the securities.
 
Example Value: 200000.00
Issue Date: Date when the security is issued.
 
Example Value: 2024-04-17
Principal Amount: The aggregate nominal amount issued for the security at the time of issuance.
 
Example Value: 900000000.00000
Issue Currency: Currency of the Principal Amount that the notes are issued.
 
Example Value: U.S. Dollar
Scheduled Maturity Date: Date when the security is redeemed.
 
Example Value: 2030-04-17
First Payment Date: Date when the first coupon/divided payment will happen.
 
Example Value: 2024-10-17
Type: Type of Coupon (e.g., Fixed Rate).
 
Example Value: Fixed Rate
Rate: Fixed Annual Coupon Rate/Annual Accretion Rate Value.
 
Example Value: 5.25800
Base Index: Reference instrument the Coupon Type Variable/Conditional/Floating Rate will be using.
 
Example Value: null
Spread: Fixed value that is added/subtracted to the Base Index when calculating the coupon rate in a coupon period.
 
Example Value: null
Day Count: Day Count Conventions are used to count the appropriate number of days between two dates in order to calculate accrued interest, yields and odd coupon amounts.
 
Example Value: 30/360
First Accrual Date: First Accrual Date is the start date of the accrual interest period.
 
Example Value: 2024-04-17
Payment Frequency: Payment Frequency and Payment Frequency Units show how frequent the coupon payments are made.
 
Example Value: 2
Payment Frequency Units: Units of payment frequency (e.g., Times per Year).
 
Example Value: Times per Year
Effective Payment Date: Effective Payment Date is the first interest payment date in a coupon leg.
 
Example Value: 2024-10-17
End Payment Date: End Payment Date is the last interest payment date in a coupon leg.
 
Example Value: 2029-04-17
Call Type: Type of Early Redemption.
 
Example Value: Optional
Call Frequency Type: Early Redemption Frequency type shows how frequently the bonds may be redeemed early.
 
Example Value: Discrete on Schedule
Call Effective Date: Early redemption start date (for continuous on schedule or every coupon or discrete on schedule with periodic dates) or the effective early redemption date (for discrete on schedule – on the effective date).
 
Example Value: 2024-04-17
Call End Date: The end date for the early redemption period.
 
Example Value: 2030-04-17
Call Price: The price at which the principal will be redeemed.
 
Example Value: 100.00000
CONVERTIBILITY: Conversion details of the Bond.
 
Example Value: null
Document to extract from:
 
If any value is not available, return null for that field. Return the output in markdown table format with columns Field, Value with proper column width.
If there are multiple transactions return each in a different markdown table
"""

vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

results = vector_index.similarity_search(query, k=1)
print(results[0].page_content)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"



text: 
The information in this preliminary pricing supplement is not complete and may be changed. This preliminary pricing supplement and the accompanying prospectus and prospectus supplement do not constitute an offer to sell these securities, and we are not soliciting an offer to buy these securities in any state where the offer or sale is not permitted.
Subject to Completion
Preliminary Pricing Supplement dated June 3, 2024
Preliminary Pricing Supplement
(To the Prospectus dated May 23, 2022 and the Prospectus Supplement
dated June 27, 2022)
Filed Pursuant to Rule 424(b)(2)
Registration No. 333-265158

US$
FIXED RATE CALLABLE NOTES DUE JUNE 24, 2027
Principal Amount:
US$
Issuer:
Barclays Bank PLC 
Issue Price:
100.00%
Series:
Global Medium-Term Notes, Series A
Payment at Maturity: 
If you hold the Notes to maturity, you will receive 100% of your principal, subject to the creditworthiness of Barclays Bank PLC and the exercise of any U.K. Bail-in Power by the relevant U.K. resolution

In [54]:
#from langchain.chains import GraphCypherQAChain
#chain = GraphCypherQAChain.from_llm(graph=graph, llm=llm, verbose=True)
#response = chain.invoke({"query": query})
#response

In [49]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

result = qa_chain({"query": query})
print(result["result"])

Here's the extracted information in the requested markdown table format:

### Transaction 1

| Field                    | Value                            |
|--------------------------|----------------------------------|
| ISIN                     | US06745U5S55                     |
| Entity                   | Barclays Bank PLC                |
| Deal Type                | Global Medium-Term Notes, Series A|
| Seniority                | Unsecured                        |
| COCO                     | null                             |
| Pledge Status            | Unsecured                        |
| Issue Price (%)          | 100.00                           |
| Face Value               | 1000.00                          |
| Min Denomination/Subscription | 1000.00                   |
| Issue Date               | 2024-06-24                       |
| Principal Amount         | null                             |
| Issue Currency           | U.S. Dollar                      |
| Scheduled 

In [36]:
help(graph.add_graph_documents)

Help on method add_graph_documents in module langchain_community.graphs.neo4j_graph:

add_graph_documents(graph_documents: List[langchain_community.graphs.graph_document.GraphDocument], include_source: bool = False, baseEntityLabel: bool = False) -> None method of langchain_community.graphs.neo4j_graph.Neo4jGraph instance
    This method constructs nodes and relationships in the graph based on the
    provided GraphDocument objects.
    
    Parameters:
    - graph_documents (List[GraphDocument]): A list of GraphDocument objects
    that contain the nodes and relationships to be added to the graph. Each
    GraphDocument should encapsulate the structure of part of the graph,
    including nodes, relationships, and the source document information.
    - include_source (bool, optional): If True, stores the source document
    and links it to nodes in the graph using the MENTIONS relationship.
    This is useful for tracing back the origin of data. Merges source
    documents based on the

In [37]:
text = """424B5 1 d323958d424b5.htm MEDIUM TERM NOTES, SERIES L 
Table of Contents
Filed Pursuant to Rule 424(b)(5) 
Registration No. 333-180488
Medium-Term Notes, Series L 
We may offer from time to time our Bank of America Corporation Medium-Term Notes, Series L. The specific 
terms of any notes that we offer will be determined before each sale and will be described in a separate product 
supplement, index supplement and/or pricing supplement (each, a “supplement”). Terms may include: 
• Priority: senior or subordinated 
• Interest rate: notes may bear interest at fixed or floating rates, or may not bear any interest 
• Base floating rates of interest: 

federal funds rate 

LIBOR 

EURIBOR 

prime rate 

treasury rate 

any other rate we specify 
• Maturity: three months or more 
• Indexed notes: principal, premium (if any), interest payments, or other amounts payable (if any) linked, either 
directly or indirectly, to the price or performance of one or more market measures, including securities, currencies 
or composite currencies, commodities, interest rates, stock or commodity indices, exchange traded funds, 
currency indices, consumer price indices, inflation indices, or any combination of the above 
• Payments: U.S. dollars or any other currency that we specify in the applicable supplement 
We may sell notes to the selling agents as principal for resale at varying or fixed offering prices or through the 
selling agents as agents using their best efforts on our behalf. We also may sell the notes directly to investors. 
We may use this prospectus supplement and the accompanying prospectus in the initial sale of any notes. In 
addition, Merrill Lynch, Pierce, Fenner & Smith Incorporated, or any of our other affiliates, may use this prospectus 
supplement and the accompanying prospectus in a market-making transaction in any notes after their initial sale. 
Unless we or one of our selling agents informs you otherwise in the confirmation of sale, this prospectus supplement 
and the accompanying prospectus are being used in a market-making transaction. 
Unless otherwise specified in the applicable supplement, we do not intend to list the notes on any securities 
exchange. 
Investing in the notes involves risks. See “Risk Factors” beginning on page S-5. 
Our notes are unsecured and are not savings accounts, deposits, or other obligations of a bank. Our notes are not 
guaranteed by Bank of America, N.A. or any other bank, are not insured by the Federal Deposit Insurance 
Corporation or any other governmental agency, and involve investment risks. 
None of the Securities and Exchange Commission, any state securities commission, or any other regulatory body 
has approved or disapproved of these notes or passed upon the adequacy or accuracy of this prospectus supplement 
or the accompanying prospectus. Any representation to the contrary is a criminal offense. 
BofA Merrill Lynch 
Prospectus Supplement to Prospectus dated March 30, 2012 
March 30, 2012 
Table of Contents
TABLE OF CONTENTS 
Page
Prospectus Supplement
About this Prospectus Supplement S-3
Risk Factors S-5
Description of the Notes S-6
General S-6
Types of Notes S-7
Payment of Principal, Interest, and Other Amounts Due S-9
Ranking S-12
Redemption S-12
Repayment S-12
Reopenings S-13
Extendible/Renewable Notes S-13
Other Provisions S-13
Repurchase S-13
Form, Exchange, Registration, and Transfer of Notes S-13
U.S. Federal Income Tax Considerations S-14
Supplemental Plan of Distribution (Conflicts of Interest) S-14
Selling Restrictions S-16
Legal Matters S-24
Page
About this Prospectus 3
Prospectus Summary 4
Risk Factors 8
Currency Risks 8
Other Risks 10
Bank of America Corporation 11
Use of Proceeds 11
Description of Debt Securities 12
General 12
The Indentures 12
Form and Denomination of Debt Securities 13
Different Series of Debt Securities 14
Fixed-Rate Notes 15
Floating-Rate Notes 15
Indexed Notes 23
Floating-Rate/Fixed-Rate/Indexed Notes 24
Original Issue Discount Notes 24
Payment of Principal, Interest, and Other Amounts Due 24
No Sinking Fund 27
Redemption 27
Repayment 27
Repurchase 27
Conversion 28
Exchange, Registration, and Transfer 28
Subordination 28
Sale or Issuance of Capital Stock of Banks 29
Limitation on Mergers and Sales of Assets 30
Waiver of Covenants 30
Modification of the Indentures 30
Meetings and Action by Securityholders 31
Events of Default and Rights of Acceleration 31
Collection of Indebtedness 31
Payment of Additional Amounts 32
Redemption for Tax Reasons 35
Defeasance and Covenant Defeasance 35
Notices 36
Concerning the Trustees 37
Governing Law 37
Description of Warrants 37
General 37
Description of Debt Warrants 37
Description of Universal Warrants 38
Modification 39
Enforceability of Rights of Warrantholders; No Trust Indenture Act Protection 39
Unsecured Obligations 40
Page
Description of Purchase Contracts 40
General 40
Purchase Contract Property 40
Information in Supplement 41
Prepaid Purchase Contracts; Applicability of Indenture 42
Non
-Prepaid Purchase Contracts; No Trust Indenture Act Protection 42
Pledge by Holders to Secure Performance 43
Settlement of Purchase Contracts That Are Part of Units 43
Failure of Holder to Perform Obligations 43
Unsecured Obligations 43
Description of Units 44
General 44
Unit Agreements: Prepaid, Non
-Prepaid, and Other 45
Modification 45
Enforceability of Rights of Unitholders; No Trust Indenture Act Protection 45
Unsecured Obligations 46
Description of Preferred Stock 46
General 46
The Preferred Stock 48
Description of Depositary Shares 49
General 49
Terms of the Depositary Shares 50
Withdrawal of Preferred Stock 50
Dividends and Other Distributions 50
Redemption of Depositary Shares 51
Voting the Deposited Preferred Stock 51
Amendment and Termination of the Deposit Agreement 51
Charges of Depository 52
Miscellaneous 52
Resignation and Removal of Depository 52
Description of Common Stock 52
General 52
Voting and Other Rights 53
Dividends 53
Registration and Settlement 54
Book
-Entry Only Issuance 54
Certificates in Registered Form 54
Street Name Owners 55
Legal Holders 55
Special Considerations for Indirect Owners 55
Depositories for Global Securities 56
Special Considerations for Global Securities 60
Registration, Transfer, and Payment of Certificated Securities 61
U.S. Federal Income Tax Considerations 62
Taxation of Debt Securities 63
Taxation of Common Stock, Preferred Stock, and Depositary Shares 78
Taxation of Warrants 84
Taxation of Purchase Contracts 84
Taxation of Units 84
Reportable Transactions 84
Foreign Account Tax Compliance Act 85
EU Directive on the Taxation of Savings Income 86
Plan of Distribution (Conflicts of Interest) 87
Distribution Through Underwriters 87
Distribution Through Dealers 88
Distribution Through Agents 88
Direct Sales 88
General Information 88
Market
-Making Transactions by Affiliates 89
Conflicts of Interest 89
ERISA Considerations 91
Where You Can Find More Information 93
Forward
-Looking Statements 94
Legal Matters 94
Experts 95 S-2 
Table of Contents
ABOUT THIS PROSPECTUS SUPPLEMENT 
We have registered the notes on a registration statement on Form S-3 with the Securities and Exchange 
Commission under Registration No. 333-180488. 
From time to time, we intend to use this prospectus supplement, the accompanying prospectus, and a related 
product supplement, index supplement and/or pricing supplement to offer the notes. We may refer to any pricing 
supplement as a “term sheet.” You should read each of these documents before investing in the notes. 
This prospectus supplement describes additional terms of the notes and supplements the description of our 
debt securities contained in the accompanying prospectus. If the information in this prospectus supplement is 
inconsistent with the prospectus, this prospectus supplement will supersede the information in the prospectus. 
This prospectus supplement and the accompanying prospectus do not constitute an offer to sell or the 
solicitation of an offer to buy the notes in any jurisdiction in which that offer or solicitation is unlawful. The 
distribution of this prospectus supplement and the accompanying prospectus and the offering of the notes in some 
jurisdictions may be restricted by law. If you have received this prospectus supplement and the accompanying 
prospectus, you should find out about and observe these restrictions. Persons outside the United States who come 
into possession of this prospectus supplement and the accompanying prospectus must inform themselves about and 
observe any restrictions relating to the distribution of this prospectus supplement and the accompanying prospectus 
and the offering of the notes outside of the United States. See “Supplemental Plan of Distribution (Conflicts of 
Interest).” 
This prospectus supplement and the accompanying prospectus have been prepared on the basis that any offer 
of notes in any Member State of the European Economic Area (each, a “Relevant Member State”) which has 
implemented the Prospectus Directive (2003/71/EC) (and amendments thereto, including the 2010 PD Amending 
Directive, to the extent implemented in the Relevant Member State, the “Prospectus Directive”) will be made under 
an exemption under the Prospectus Directive, as implemented in that Relevant Member State, from the requirement 
to publish a prospectus for offers of notes. Accordingly, any person making or intending to make an offer in that 
Relevant Member State of any notes which are contemplated in this prospectus supplement and the accompanying 
prospectus may only do so in circumstances in which no obligation arises for us or any of the selling agents to 
publish a prospectus pursuant to Article 3 of the Prospectus Directive or supplement a prospectus pursuant to Article 
16 of the Prospectus Directive, in each case, in relation to such offer. Neither we nor the selling agents have 
authorized, and neither we nor they authorize, the making of any offer of notes in circumstances in which an 
obligation arises for us or any selling agent to publish or supplement a prospectus for the purposes of the Prospectus 
Directive in relation to such offer. Neither this prospectus supplement nor the accompanying prospectus constitutes 
an approved prospectus for the purposes of the Prospective Directive. 
For each offering of notes, we will issue a product supplement, index supplement, and/or a pricing supplement 
which will contain additional terms of the offering and a specific description of the notes being offered. A 
supplement also may add, update, or change information in this prospectus supplement or the accompanying 
prospectus, including provisions describing the calculation of the amounts due under the notes and the method of 
making payments under the terms of a note. We will state in the applicable supplement the interest rate or interest 
rate basis or formula, issue price, any relevant market measures, the maturity date, interest payment dates, 
redemption, or repayment provisions, if any, and other relevant terms and conditions for 
S-3 
Table of Contents
each note at the time of issuance. A supplement also may include a discussion of any risk factors or other special 
additional considerations that apply to a particular type of note. Each applicable supplement can be quite detailed 
and always should be read carefully. 
Any term that is used, but not defined, in this prospectus supplement has the meaning set forth in the 
accompanying prospectus. 
S-4 
Table of Contents
RISK FACTORS 
Your investment in the notes involves significant risks. Your decision to purchase the notes should be made 
only after carefully considering the risks of an investment in the notes, including those discussed below, in the 
accompanying prospectus beginning on page 8, and in the relevant supplement(s) for the specific notes, with your 
advisors in light of your particular circumstances. The notes are not an appropriate investment for you if you are not 
knowledgeable about significant elements of the notes or financial matters in general. For information regarding 
risks and uncertainties that may materially affect our business and results, please refer to the information under the 
captions “Item 1A. Risk Factors” and “Item 7. Management’s Discussion and Analysis of Financial Condition and 
Results of Operations” in our annual report on Form 10-K for the year ended December 31, 2011, which is 
incorporated by reference in the accompanying prospectus. You should also review the risk factors that will be set 
forth in other documents that we will file after the date of this prospectus supplement. 
The market value of the notes may be less than the principal amount of the notes. 
The market for, and market value of, the notes may be affected by a number of factors. These factors include: 
• the method of calculating the principal, premium, if any, interest or other amounts payable, if any, on the 
notes; 
• the time remaining to maturity of the notes; 
• the aggregate amount outstanding of the relevant notes; 
• any redemption or repayment features of the notes; 
• the level, direction, and volatility of market interest rates generally; 
• general economic conditions of the capital markets in the United States; 
• geopolitical conditions and other financial, political, regulatory, and judicial events that affect the stock 
markets generally; and 
• any market-making activities with respect to the notes. 
Often, the only way to liquidate your investment in the notes prior to maturity will be to sell the notes. At that 
time, there may be a very illiquid market for the notes or no market at all. For indexed notes that have specific 
investment objectives or strategies, the applicable market may be more limited, and the price may be more volatile, 
than for other notes. The market value of indexed notes may be adversely affected by the complexity of the formula 
and volatility of the applicable reference market measure, including any dividend rates or yields of other securities 
or financial instruments that relate to the indexed notes. Moreover, the market value of indexed notes could be 
adversely affected by changes in the amount of outstanding equity or other securities linked to those notes. 
Holders of indexed notes are subject to important risks that are not associated with more conventional debt 
securities. 
If you invest in indexed notes, you will be subject to significant risks not associated with conventional fixedrate or floating-rate debt securities. These risks include the possibility that the 
S-5 
Table of Contents
applicable market measures may be subject to fluctuations, and the possibility that you will receive a lower, or no, 
amount of principal, premium, or interest, and at different times than expected. In recent years, many securities, 
currencies, commodities, interest rates, indices, and other market measures have experienced volatility, and this 
volatility may be expected in the future. However, past experience is not necessarily indicative of what may occur in 
the future. We have no control over a number of factors, including economic, financial, and political events, that are 
important in determining the existence, magnitude, and longevity of market volatility and other risks and their 
impact on the value of, or payments made on, the indexed notes. In considering whether to purchase indexed notes, 
you should be aware that the calculation of amounts payable on indexed notes may involve reference to a market 
measure determined by one of our affiliates or prices or values that are published solely by third parties or entities 
which are not regulated by the laws of the United States. Additional risks that you should consider in connection 
with an investment in indexed notes are set forth in the applicable supplement(s). 
Our employees who purchase the notes must comply with policies that limit their ability to trade the notes, 
and that may affect the value of their notes. 
If you are our employee or an employee of one of our affiliates, including one of the selling agents, you may 
acquire notes for investment purposes only, and you must comply with all of our internal policies and procedures. 
Because these policies and procedures limit the dates and times that you may effect a transaction in the notes, you 
may not be able to purchase any of the notes from us, and your ability to trade or sell any of the notes in any 
secondary market may be limited. 
Usury laws may limit the amount of interest that can be charged and paid on the notes. 
New York law will govern the notes offered by this prospectus supplement. New York usury laws limit the 
amount of interest that can be charged and paid on loans, including the notes. Under current New York law, the 
maximum permissible rate of interest is 25% per year on a simple interest basis. This limit may not apply to notes in 
which $2,500,000 or more has been invested. While we believe that a U.S. federal or state court sitting outside New 
York may give effect to New York law, many other states also have laws that regulate the amount of interest that 
may be charged to and paid by a borrower. We do not intend to claim the benefits of any laws concerning usurious 
rates of interest. 
DESCRIPTION OF THE NOTES 
This section describes the general terms and conditions of the notes, which may be senior or subordinated 
medium-term notes. This section supplements, and should be read together with, the general description of our debt 
securities included in “Description of Debt Securities” in the accompanying prospectus. If there is any inconsistency 
between the information in this prospectus supplement and the accompanying prospectus, you should rely on the 
information in this prospectus supplement. 
We will describe the particular terms of the notes we sell in a separate supplement. The terms and conditions 
stated in this section will apply to each note unless the note or the applicable supplement indicates otherwise. 
General 
The following summary of the terms of the notes and the indentures is not complete and is qualified in its 
entirety by reference to the actual notes and the specific provisions of the Senior Indenture and the Subordinated 
Indenture, as applicable. 
S-6 
Table of Contents
We will issue the notes as part of a series of debt securities under the Senior Indenture or the Subordinated 
Indenture, as applicable, which are exhibits to our registration statement and are contracts between us and The Bank 
of New York Mellon Trust Company, N.A., as successor trustee. In this prospectus supplement, we refer to The 
Bank of New York Mellon Trust Company, N.A., as the “trustee,” and we refer to the Senior Indenture and the 
Subordinated Indenture individually as the “Indenture” and together as the “Indentures.” 
The Indentures are subject to, and governed by, the Trust Indenture Act of 1939. We, the selling agents, and 
the depository, in the ordinary course of our respective businesses, have conducted and may conduct business with 
the trustee or its affiliates. See “Description of Debt Securities — The Indentures” in the accompanying prospectus 
for more information about the Indentures and the functions of the trustee. 
The notes are our direct unsecured obligations and are not obligations of our subsidiaries. The notes are being 
offered on a continuous basis. There is no limit under our registration statement on the total initial public offering 
price or aggregate principal amount of the Senior and Subordinated Medium-Term Notes, Series L, that may be 
offered using this prospectus supplement. We may issue other debt securities under the Indentures from time to time 
in one or more series up to the aggregate principal amount of the then-existing grant of authority by our board of 
directors. 
Unless otherwise provided in the applicable supplement, the minimum denomination of the notes will be 
$1,000 and any larger amount that is a whole multiple of $1,000 (or the equivalent in other currencies). We may also 
issue the notes in units of $10. 
Types of Notes 
Fixed-Rate Notes. We may issue notes that bear interest at a fixed rate described in the applicable pricing 
supplement, which we refer to as “fixed-rate notes.” We also may issue fixed-rate notes that combine principal and 
interest payments in installment payments over the life of the note, which we refer to as “amortizing notes.” For 
more information on fixed-rate notes and amortizing notes, see “Description of Debt Securities — Fixed-Rate 
Notes” in the accompanying prospectus. 
Floating-Rate Notes. We may issue notes that bear interest at a floating rate of interest determined by 
reference to one or more base interest rates, or by reference to one or more interest rate formulae, described in the 
applicable supplement, which we refer to as “floating-rate notes.” In some cases, the interest rate of a floating-rate 
note also may be adjusted by adding or subtracting a spread or by multiplying the interest rate by a spread multiplier. 
A floating-rate note also may be subject to a maximum interest rate limit, or ceiling, and/or a minimum interest rate 
limit, or floor, on the interest that may accrue during any interest period. For more information on floating-rate 
notes, including a description of the manner in which interest payments will be calculated, see “Description of Debt 
Securities — Floating-Rate Notes” in the accompanying prospectus. 
Indexed Notes. We may issue notes that provide that the rate of return, including the principal, premium (if 
any), interest, or other amounts payable (if any), is determined by reference, either directly or indirectly, to the price 
or performance of one or more securities, commodities, currencies or composite currencies, interest rates, stock or 
commodity indices, exchange traded funds, currency indices, consumer price indices, inflation indices or other 
market measures, or any combination of the above, in each case as specified in the applicable supplement. We refer 
to these notes as “indexed notes.” 
S-7 
Table of Contents
If you purchase an indexed note, you may receive an amount at maturity that is greater than or less than the 
face amount of your note, depending upon the formula used to determine the amount payable and the relative value 
at maturity of the market measure to which your indexed note is linked. We expect that the value of the applicable 
market measure will fluctuate over time. 
An indexed note may provide either for cash settlement or for physical settlement by delivery of the relevant 
asset. An indexed note also may provide that the form of settlement may be determined at our option or the holder’s 
option. Some indexed notes may be convertible, exercisable, or exchangeable prior to maturity, at our option or the 
holder’s option, for the relevant asset or the cash value of the relevant asset. 
We will specify in the applicable supplement the method for determining the principal, premium (if any), 
interest, or other amounts payable (if any) in respect of particular indexed notes, as well as certain historical or other 
information with respect to the specified index or other market measure, specific risk factors relating to that 
particular type of indexed note, and tax considerations associated with an investment in the indexed notes. 
A supplement for any particular indexed notes also will identify the calculation agent that will calculate the 
amounts payable with respect to the indexed note. The calculation agent may be one of our affiliates, including Bank 
of America, N.A., Merrill Lynch, Pierce, Fenner & Smith Incorporated (“MLPF&S”), Merrill Lynch Commodities, 
Inc., or Merrill Lynch Capital Services, Inc. We may appoint different calculation agents from time to time after the 
original issue date of an indexed note without your consent and without notifying you of the change. Absent 
manifest error, all determinations of the calculation agent will be final and binding on you, the selling agents, and us. 
Upon request of the holder of an indexed note, and to the extent set forth in the applicable supplement, the 
calculation agent will provide, if applicable, information relating to the current principal, premium (if any), rate of 
interest, interest payable, or other amounts payable (if any) in connection with that indexed note. 
For more information about indexed notes, see “Description of Debt Securities — Indexed Notes” in the 
accompanying prospectus. 
Original Issue Discount Notes. We may issue notes at a price lower than their principal amount or lower than 
their minimum guaranteed repayment amount at maturity, which we refer to as “original issue discount notes.” 
Original issue discount notes may be fixed-rate, floating-rate, or indexed notes and may bear no interest (“zero 
coupon notes”) or may bear interest at a rate that is below market rates at the time of issuance. For more information 
on original issue discount notes, see “Description of Debt Securities — Original Issue Discount Notes” in the 
accompanying prospectus. 
Specific Terms of the Notes. The applicable supplement(s) for each offering of notes will contain additional 
terms of the offering and a specific description of those notes, including: 
• the specific designation of the notes; 
• the issue price; 
• the principal amount; 
• the issue date; 
• the maturity date, and any terms providing for the extension or postponement of the maturity date; 
S-8 
Table of Contents
• the denominations or minimum denominations, if other than $1,000; 
• the currency or currencies, if not U.S. dollars, in which payments will be made on the notes; 
• whether the note is a fixed-rate note, a floating-rate note, or an indexed note; 
• whether the note is senior or subordinated; 
• the method of determining and paying interest, including any applicable interest rate basis or bases, any 
initial interest rate, any interest reset dates, any payment dates, any index maturity, and any maximum or 
minimum rate of interest; 
• any spread or spread multiplier applicable to a floating-rate note or an indexed note; 
• the method for the calculation and payment of principal, premium (if any), interest, and other amounts 
payable (if any); 
• for exchangeable notes, the securities, or other property for which the notes may be exchanged, the rate of 
exchange, whether the notes are exchangeable at your option or our option, and other terms of the 
exchangeable notes; 
• if applicable, the circumstances under which the note may be redeemed at our option or repaid at your option 
prior to the maturity date set forth on the face of the note, including any repayment date, redemption 
commencement date, redemption price, and redemption period; 
• if applicable, the circumstances under which the maturity date set forth on the face of the note may be 
extended at our option or renewed at your option, including the extension or renewal periods and the final 
maturity date; 
• whether the notes will be listed on any stock exchange; and 
• if applicable, any other material terms of the note which are different from those described in this prospectus 
supplement and the accompanying prospectus. 
Each note will mature on a business day (as defined in the accompanying prospectus) three or more months 
from the issue date. Unless we specify otherwise in the supplement, the record dates for any interest payments for 
book-entry notes denominated in U.S. dollars will be one business day (in Charlotte, North Carolina and New York 
City) prior to the applicable payment date, and for any book-entry notes denominated in a currency other than U.S. 
dollars will be the fifteenth calendar day preceding the applicable payment date. 
Unless we specify otherwise in the applicable supplement, the notes will not be entitled to the benefit of any 
sinking fund. 
Payment of Principal, Interest, and Other Amounts Due 
Paying Agents. Unless otherwise provided in the applicable pricing supplement, the trustee will act as our 
paying agent, security registrar, and transfer agent with respect to the notes through the trustee’s corporate trust 
office. That office is currently located at 101 Barclay Street, New York, New York 10286. If specified in the 
applicable pricing supplement, with respect to some of our notes, including notes denominated in euro, The Bank of 
New York Mellon will act 
S-9 
Table of Contents
as the London paying agent (the “London paying agent”) through its London branch, which is located at the 48th 
Floor, One Canada Square, London, E14 5AL. At any time, we may rescind the designation of a paying agent, 
appoint a successor paying agent, or approve a change in the office through which any successor paying agent acts 
in accordance with the applicable Indenture. In addition, we may decide to act as our own paying agent with respect 
to some or all of the notes, and the paying agent may resign. 
Calculation Agents. The trustee or the London paying agent also will act as the calculation agent for floatingrate notes, unless otherwise specified in the applicable pricing supplement. We will identify the calculation agent for 
any indexed notes in the applicable pricing supplement. The calculation agent will be responsible for calculating the 
interest rate, reference rates, principal, premium (if any), interest, or other amounts payable (if any) applicable to the 
floating-rate notes or indexed notes, as the case may be, and for certain other related matters. The calculation agent, 
at the request of the holder of any floating-rate note, will provide the interest rate then in effect and, if already 
determined, the interest rate that is to take effect on the next interest reset date, as described below, for the floatingrate note. At the request of the holder of any floating-rate note that is an indexed note, and to the extent set forth in 
the applicable supplement, the calculation agent will provide the reference rate or formula then in effect. We may 
replace any calculation agent or elect to act as the calculation agent for some or all of the notes, and the calculation 
agent may resign. 
Manner of Payment. Unless otherwise stated in the applicable pricing supplement, we will pay principal, 
premium (if any), interest, and other amounts payable (if any) on the notes in book-entry form in accordance with 
arrangements then in place between the applicable paying agent and the applicable depository. Unless otherwise 
stated in the applicable pricing supplement, we will pay any interest on notes in certificated form on each interest 
payment date other than the maturity date by check mailed to holders of the notes on the applicable record date at 
the address appearing on our records. Unless otherwise stated in the applicable pricing supplement, we will pay any 
principal, premium (if any), interest, and other amounts payable (if any) at the maturity date of a note in certificated 
form by wire transfer of immediately available funds upon surrender of the note at the corporate trust office of the 
trustee or the London paying agent, as applicable. 
Currency Conversions and Payments on Notes Denominated in Currencies Other than U.S. Dollars. For 
any notes denominated in a currency other than U.S. dollars, the initial investors will be required to pay for the notes 
in that foreign currency. The applicable selling agent may arrange for the conversion of U.S. dollars into the 
applicable foreign currency to facilitate payment for the notes by U.S. purchasers electing to make the initial 
payment in U.S. dollars. Any such conversion will be made by that selling agent on the terms and subject to the 
conditions, limitations, and charges as it may establish from time to time in accordance with its regular foreign 
exchange procedures, and subject to United States laws and regulations. All costs of any such conversion for the 
initial purchase of the notes will be borne by the initial investors using those conversion arrangements. 
We generally will pay principal, premium (if any), interest, and other amounts payable (if any) on notes 
denominated in a currency other than U.S. dollars in the applicable foreign currency. Holders of beneficial interests 
in notes through a participant in The Depository Trust Company, or “DTC,” will receive payments in U.S. dollars, 
unless they elect to receive payments on those notes in the applicable foreign currency. If a holder through DTC 
does not make an election to receive payments in the applicable foreign currency, the trustee will convert payments 
to that holder into U.S. dollars, and all costs of those conversions will be borne by that holder by deduction from the 
applicable payments. 
S-10 
Table of Contents
For holders not electing payment in the applicable foreign currency, the U.S. dollar amount of any payment 
will be the amount of the applicable foreign currency otherwise payable, converted into U.S. dollars at the 
applicable exchange rate prevailing as of 11:00 a.m. (New York City time) on the second business day prior to the 
relevant payment dates, less any costs incurred by the trustee for that conversion. The costs of those conversions will 
be shared pro rata among the holders of beneficial interests in the applicable global notes receiving U.S. dollar 
payments in the proportion of their respective holdings. The trustee will make those conversions in accordance with 
the terms of the applicable note and with any applicable arrangements between us and the trustee. 
If an exchange rate quotation is unavailable from the entity or source ordinarily used by the trustee in the 
normal course of business, the trustee will obtain a quotation from a leading foreign exchange bank in New York 
City, which may be an affiliate of the trustee or another entity selected by the trustee for that purpose after 
consultation with us. If no quotation from a leading foreign exchange bank is available, payment will be made in the 
applicable foreign currency to the account or accounts specified by DTC to the trustee, unless the applicable foreign 
currency is unavailable due to the imposition of exchange controls or other circumstances beyond our control. If 
payment on a note is required to be made in a currency other than U.S. dollars and that currency is unavailable due 
to the imposition of exchange controls or other circumstances beyond our control, or is no longer used by the 
government of the relevant country or for the settlement of transactions by public institutions of or within the 
international banking community (and is not replaced by another currency), then all payments on that note will be 
made in U.S. dollars on the basis of the most recently available market exchange rate for the applicable foreign 
currency. Any payment on a note so made in U.S. dollars will not constitute an event of default under the applicable 
notes. 
The holder of a beneficial interest in global notes held through a DTC participant may elect to receive 
payments on those notes in a foreign currency by notifying the DTC participant through which it holds its beneficial 
interests on or prior to the fifteenth business day prior to the record date for the applicable notes of (1) that holder’s 
election to receive all or a portion of the payment in the applicable foreign currency and (2) wire transfer 
instructions to an account for the applicable foreign currency outside the United States. DTC must be notified of that 
election and wire transfer instructions (a) on or prior to the fifth business day after the record date for any payment 
of interest and (b) on or prior to the tenth business day prior to the date for any payment of principal. DTC will 
notify the trustee of the election and wire transfer instructions (1) on or prior to 5:00 p.m. New York City time on 
the fifth business day after the record date for any payment of interest and (2) on or prior to 5:00 p.m. New York 
City time on the tenth business day prior to the date for any payment of principal. If complete instructions are 
forwarded to and received by DTC through a DTC participant and forwarded by DTC to the trustee and received on 
or prior to the dates described above, the holder will receive payment in the applicable foreign currency outside 
DTC; otherwise, only U.S. dollar payments will be made by the trustee to DTC. 
For purposes of the above discussion about currency conversions and payments on notes denominated in a 
foreign currency, the term “business day” means any weekday that is not a legal holiday in New York, New York or 
Charlotte, North Carolina and is not a day on which banking institutions in those cities are authorized or required by 
law or regulation to be closed. 
For information regarding risks associated with foreign currencies and exchange rates, see “Risk Factors —
Currency Risks” in the accompanying prospectus. 
Payment of Additional Amounts. If we so specify in the applicable pricing supplement, additional amounts 
will be payable to a beneficial holder of notes that is a non-U.S. person. Our obligation to pay additional amounts to 
non-U.S. persons is subject to the limitations described 
S-11 
Table of Contents
under “Description of Debt Securities — Payment of Additional Amounts” in the accompanying prospectus. If we 
so specify in the applicable pricing supplement, we may redeem the notes in whole, but not in part, at any time 
before maturity if we have or will become obligated to pay additional amounts as a result of a change in, or 
amendment to, U.S. tax laws or regulations, as described under “Description of Debt Securities — Redemption for 
Tax Reasons” in the accompanying prospectus. 
For more information about payment procedures, including payments in a currency other than U.S. dollars, see 
“Description of Debt Securities — Payment of Principal, Interest, and Other Amounts Due” in the accompanying 
prospectus. 
Ranking 
Because we are a holding company, our right to participate in any distribution of assets of any subsidiary upon 
such subsidiary’s liquidation or reorganization or otherwise is subject to the prior claims of creditors of that 
subsidiary, except to the extent we may ourselves be recognized as a creditor of that subsidiary. Accordingly, our 
obligations under senior notes or subordinated notes will be structurally subordinated to all existing and future 
liabilities of our subsidiaries, and claimants should look only to our assets for payments. 
Senior Notes. The senior notes will be unsecured and will rank equally with all our other unsecured and 
unsubordinated obligations from time to time outstanding, except obligations, including deposit liabilities, that are 
subject to any priorities or preferences by law. 
The Senior Indenture and the senior notes do not contain any limitation on the amount of obligations that we 
may incur in the future. 
Subordinated Notes. Our indebtedness evidenced by the subordinated notes, including the principal, premium 
(if any), interest, and other amounts payable (if any) will be subordinate and junior in right of payment to all of our 
senior indebtedness from time to time outstanding. Payment of principal of our subordinated indebtedness, including 
any subordinated notes, may not be accelerated if there is a default in the payment of amounts due under, or a 
default in any of our other covenants applicable to, our subordinated indebtedness. 
The Subordinated Indenture and the subordinated notes do not contain any limitation on the amount of 
obligations ranking senior to the subordinated notes, or the amount of obligations ranking equally with the 
subordinated notes, that we may incur in the future. 
For more information about our subordinated notes, see “Description of Debt Securities — Subordination” in 
the accompanying prospectus. 
Redemption 
The applicable supplement will indicate whether we have the option to redeem notes prior to their maturity 
date. If we may redeem the notes prior to maturity, the applicable supplement will indicate the redemption price and 
method for redemption. See also “Description of Debt Securities — Redemption” in the accompanying prospectus. 
Repayment 
The applicable supplement will indicate whether the notes can be repaid at the holder’s option prior to their 
maturity date. If the notes may be repaid prior to maturity, the applicable supplement will indicate the amount at 
which we will repay the notes and the procedure for repayment. 
S-12 
Table of Contents
Reopenings 
We have the ability to “reopen,” or increase after the issuance date, the principal amount of a particular 
tranche or series of our notes without notice to the holders of existing notes by selling additional notes having the 
same terms. However, any new notes of this kind may have a different offering price and may begin to bear interest 
at a different date. 
Extendible/Renewable Notes 
We may issue notes for which the maturity date may be extended at our option or renewed at the option of the 
holder for one or more specified periods, up to but not beyond the final maturity date stated in the note. The specific 
terms of and any additional considerations relating to extendible or renewable notes will be set forth in the 
applicable supplement. 
Other Provisions 
Any provisions with respect to the determination of an interest rate basis, the specification of interest rate 
basis, the calculation of the applicable interest rate, the amounts payable at maturity, interest payment dates, or any 
other related matters for a particular tranche of notes, may be modified as described in the applicable supplement. 
Repurchase 
We, or our affiliates, may purchase at any time our notes by tender, in the open market at prevailing prices or 
in private transactions at negotiated prices. If we purchase notes in this manner, we have the discretion to hold, 
resell, or cancel any repurchased notes. 
Form, Exchange, Registration, and Transfer of Notes 
We will issue each note in book-entry only form. This means that we will not issue actual notes or certificates 
to each beneficial owner. Instead, the notes will be in the form of a global note or a master global note, in fully 
registered form, registered and held in the name of the applicable depository or a nominee of that depository. For 
notes denominated in a currency other than U.S. dollars, the notes may be issued in the form of two global notes, 
each in fully registered form, one of which will be deposited with DTC, or its custodian, and one of which will be 
deposited with a common depository for Euroclear Bank SA/NV (“Euroclear”) and/or Clearstream Banking, société 
anonyme, Luxembourg (“Clearstream”). Unless we specify otherwise in the applicable pricing supplement, the 
depository for the notes will be DTC. DTC, Euroclear, and Clearstream, as depositories for global securities, and 
some of their policies and procedures are described under “Registration and Settlement — Depositories for Global 
Securities” in the accompanying prospectus. For more information about book-entry only notes and the procedures 
for registration, settlement, exchange, and transfer of book-entry only notes, see “Description of Debt Securities —
Form and Denomination of Debt Securities” and “Registration and Settlement” in the accompanying prospectus. 
If we ever issue notes in certificated form, unless we specify otherwise in the applicable supplement, those 
notes will be in registered form, and the exchange, registration, or transfer of those notes will be governed by the 
applicable Indenture and the procedures described under “Description of Debt Securities — Exchange, Registration, 
and Transfer” and “Registration and Settlement — Registration, Transfer, and Payment of Certificated Securities” in 
the accompanying prospectus. 
S-13 
Table of Contents
U.S. FEDERAL INCOME TAX CONSIDERATIONS 
For the material U.S. federal income tax considerations of the acquisition, ownership and disposition of 
certain notes, see “U.S. Federal Income Tax Considerations” on page 62 of the accompanying prospectus and the 
subsection “Taxation of Debt Securities” of that section. Special U.S. federal income tax rules are applicable to 
certain types of notes we may issue under this prospectus supplement. The material U.S. federal income tax 
considerations with respect to any notes we issue, and which are not addressed in the accompanying prospectus, will 
be discussed in the applicable supplement. 
You should consult with your own tax advisor before investing in the notes. 
SUPPLEMENTAL PLAN OF DISTRIBUTION (CONFLICTS OF INTEREST) 
We are offering the notes for sale on a continuing basis through the selling agents. The selling agents may act 
either on a principal basis or on an agency basis. We may offer the notes at varying prices relating to prevailing 
market prices at the time of resale, as determined by the selling agents, or, if so specified in the applicable pricing 
supplement, for resale at a fixed public offering price. The applicable pricing supplement will set forth the initial 
price for the notes, or whether they will be sold at varying prices. 
If we sell notes on an agency basis, we will pay a commission to the selling agent to be negotiated at the time 
of sale. The commission will be determined at the time of sale and will be specified in the applicable pricing 
supplement. Each selling agent will use its reasonable best efforts when we request it to solicit purchases of the 
notes as our agent. 
Unless otherwise agreed and specified in the applicable pricing supplement, if notes are sold to a selling agent 
acting as principal, for its own account, or for resale to one or more investors or other purchasers, including other 
broker-dealers, then any notes so sold will be purchased by that selling agent at a price equal to 100% of the 
principal amount of the notes less a commission that will be a percentage of the principal amount determined as 
described above. Notes sold in this manner may be resold by the selling agent to investors and other purchasers from 
time to time in one or more transactions, including negotiated transactions, at a fixed public offering price or at 
varying prices determined at the time of sale, or the notes may be resold to other dealers for resale to investors. The 
selling agents may allow any portion of the discount received in connection with the purchase from us to the dealers, 
but the discount allowed to any dealer will not be in excess of the discount to be received by the selling agent from 
us. After the initial public offering of notes, the selling agent may change the public offering price or the discount 
allowed to dealers. 
We also may sell notes directly to investors, without the involvement of any selling agent. In this case, we 
would not be obligated to pay any commission or discount in connection with the sale, and we would receive 100% 
of the principal amount of the notes so sold, unless otherwise specified in the applicable pricing supplement. 
We will name any selling agents or other persons through which we sell any notes, as well as any 
commissions or discounts payable to those selling agents or other persons, in the applicable pricing supplement. As 
of the date of this prospectus supplement, our selling agent is MLPF&S. MLPF&S has entered into a distribution 
agreement with us that describes the offering of notes by them as our agent and as our principal. We also may accept 
offers to purchase notes through additional selling agents on substantially the same terms and conditions, including 
commissions, as would apply to purchases through MLPF&S under the distribution agreement. If a selling 
S-14 
Table of Contents
agent purchases notes as principal, that selling agent usually will be required to enter into a separate purchase 
agreement for the notes, and may be referred to in that purchase agreement and the applicable pricing supplement, 
along with any other selling agents, as “underwriters.”
We have the right to withdraw, cancel, or modify the offer made by this prospectus supplement without 
notice. We will have the sole right to accept offers to purchase notes, and we, in our absolute discretion, may reject 
any proposed purchase of notes in whole or in part. Each selling agent will have the right, in its reasonable 
discretion, to reject in whole or in part any proposed purchase of notes through that selling agent. 
Any selling agent participating in the distribution of the notes may be considered to be an underwriter, as that 
term is defined in the Securities Act. We have agreed to indemnify each selling agent and certain other persons 
against certain liabilities, including liabilities under the Securities Act, or to contribute to payments that the selling 
agents may be required to make. We also have agreed to reimburse the selling agents for certain expenses. 
The notes will not have an established trading market when issued, and we do not intend to list the notes on 
any securities exchange, unless otherwise specified in the applicable pricing supplement. Any selling agent may 
purchase and sell notes in the secondary market from time to time. However, no selling agent is obligated to do so, 
and any selling agent may discontinue making a market in the notes at any time without notice. There is no 
assurance that there will be a secondary market for any of the notes. 
To facilitate offerings of the notes by a selling agent that purchases notes as principal, and in accordance with 
industry practice, selling agents may engage in transactions that stabilize, maintain, or otherwise affect the market 
price of the notes. Those transactions may include overallotment, entering stabilizing bids, effecting syndicatecovering transactions, and imposing penalty bids to reclaim selling concessions allowed to a member of the 
syndicate or to a dealer, as follows: 
• An overallotment in connection with an offering creates a short position in the offered securities for the 
selling agent’s own account. 
• A selling agent may place a stabilizing bid to purchase a note for the purpose of pegging, fixing, or 
maintaining the price of that note. 
• Selling agents may engage in syndicate-covering transactions to cover overallotments or to stabilize the price 
of the notes by bidding for, and purchasing, the notes or any other securities in the open market in order to 
reduce a short position created in connection with the offering. 
• The selling agent that serves as syndicate manager may impose a penalty bid on a syndicate member to 
reclaim a selling concession in connection with an offering when offered securities originally sold by the 
syndicate member are purchased in syndicate-covering transactions, in stabilization transactions, or 
otherwise. 
Any of these activities may stabilize or maintain the market price of the securities above independent market 
levels. The selling agents are not required to engage in these activities, and may end any of these activities at any 
time. 
MLPF&S, a selling agent and one of our affiliates, is a broker-dealer and member of the Financial Industry 
Regulatory Authority, Inc., or “FINRA.” Each initial offering and any remarketing of notes involving any of our 
broker-dealer affiliates, including MLPF&S, will be 
S-15 
Table of Contents
conducted in compliance with the requirements of FINRA Rule 5121 regarding a FINRA member firm’s offer and 
sale of securities of an affiliate. None of our broker-dealer affiliates that is a FINRA member will execute a 
transaction in the notes in a discretionary account without specific prior written approval of the customer, see “Plan 
of Distribution (Conflicts of Interest) – Conflicts of Interest” in the accompanying prospectus. 
Following the initial distribution of any notes, our affiliates, including MLPF&S, may buy and sell the notes 
in market-making transactions as part of their business as a broker-dealer. Resales of this kind may occur in the open 
market or may be privately negotiated at prevailing market prices at the time of sale. Notes may be sold in 
connection with a remarketing after their purchase by one or more firms. Any of our affiliates may act as principal 
or agent in these transactions. 
This prospectus supplement may be used by one or more of our affiliates in connection with offers and sales 
related to market-making transactions in the notes, including block positioning and block trades, to the extent 
permitted by applicable law. Any of our affiliates may act as principal or agent in these transactions. 
Notes sold in market-making transactions include notes issued after the date of this prospectus supplement as 
well as previously-issued securities. Information about the trade and settlement dates, as well as the purchase price, 
for a market-making transaction will be provided to the purchaser in a separate confirmation of sale. Unless we or 
one of our selling agents informs you in the confirmation of sale that notes are being purchased in an original 
offering and sale, you may assume that you are purchasing the notes in a market-making transaction. 
MLPF&S and other selling agents that we may name in the future, or their affiliates, have engaged, and may 
in the future engage, in investment banking, commercial banking, and financial advisory transactions with us and 
our affiliates. These transactions are in the ordinary course of business for the selling agents and us and our 
respective affiliates. In these transactions, the selling agents or their affiliates receive customary fees and expenses. 
Although we expect that delivery of the notes generally will be made against payment on or about the third 
business day following the date of any contract for sale, we may specify a shorter or a longer settlement cycle in the 
applicable pricing supplement. Under Rule 15c6-1 of the Securities Exchange Act of 1934, trades in the secondary 
market generally are required to settle in three business days, unless the parties to a trade expressly agree otherwise. 
Accordingly, if we have specified a longer settlement cycle in the applicable pricing supplement for an offering of 
securities, purchasers who wish to trade those securities on the date of the contract for sale, or on one or more of the 
next succeeding business days as we will specify in the applicable pricing supplement, will be required, by virtue of 
the fact that those securities will settle in more than T+3, to specify an alternative settlement cycle at the time of the 
trade to prevent a failed settlement and should consult their own advisors in connection with that election. 
Selling Restrictions 
General. Each of the selling agents, severally and not jointly, has represented and agreed that it has not and 
will not offer, sell, or deliver any note, directly or indirectly, or distribute this prospectus supplement or the 
accompanying prospectus, or any other offering material relating to any of the notes, in any jurisdiction except under 
circumstances that will result in compliance with applicable laws and regulations and that will not impose any 
obligations on us except as set forth in the distribution agreement. 
S-16 
Table of Contents
Argentina. We have not made, and will not make, any application to obtain an authorization from the 
Comisión Nacional de Valores (the “CNV”) for the public offering of the notes in Argentina. The CNV has not 
approved the terms and conditions of the notes, their issuance or offering, this prospectus supplement or the 
accompanying prospectus, or any other document relating to the offering of the notes. The selling agents have not 
offered or sold, and will not offer or sell, any of the notes in Argentina, except in transactions that will not constitute 
a public offering of securities within the meaning of Section 16 of the Argentine Public Offering Law N° 17,811. 
Argentine insurance companies may not purchase the notes. 
Australia. Each selling agent has represented and agreed that in connection with the distribution of the notes, 
it: 
(a) must not make any offer or invitation in Australia or which is received in Australia in relation to the issue, 
sale or purchase of any notes unless the offeree or invitee is required to pay at least A$500,000 for the 
notes (disregarding amounts, if any, lent by us or any other person offering the notes or its associates 
(within the meaning of those expressions in Part 6D.2 of the Corporations Act 2001 (Cth) of the 
Commonwealth of Australia (the “Corporations Act”))), or it otherwise is an offer or invitation in respect 
of which, by virtue of s708 of the Corporations Act, no disclosure is required to be made under Part 6D.2 
of the Corporations Act; and 
(b) has not circulated or issued and will not circulate or issue this prospectus supplement or the accompanying 
prospectus or any disclosure document relating to the notes in Australia or received in Australia which 
requires lodging under Division 5 of Part 6D.2 of the Corporations Act. 
We are not authorized under the Banking Act 1959 of the Commonwealth of Australia (the Banking Act) to 
carry on banking business and are not subject to prudential supervision by the Australian Prudential Regulation 
Authority. The notes are not Deposit Liabilities under the Banking Act. We are the holding corporation of Bank of 
America, N.A. 
Austria. The notes may only be offered in the Republic of Austria in accordance with the Austrian Capital 
Market Act and any other laws and regulations applicable in the Republic of Austria governing the issue, offer and 
sale of securities in the Republic of Austria. The notes are not registered or otherwise authorised for public offer 
under the Austrian Capital Market Act or any other applicable laws and regulations in Austria. The recipients of this 
prospectus supplement, the accompanying prospectus and any other selling materials in respect to the notes are 
qualified investors within the meaning of the Austrian Capital Market Act, i.e., persons who purchase and sell 
securities as part of their profession or business, and are targeted exclusively on the basis of a private placement. 
Accordingly, the notes may not be, and are not being, issued, offered or advertised publicly or offered similarly 
under either the Austrian Capital Market Act or any other relevant securities legislation in Austria. We are a U.S. 
bank holding company and a financial holding company. We are not a bank under the Austrian Banking Act 
(Bankwesengesetz) and are not EU passported to perform banking business in Austria. 
Brazil. The information contained in this prospectus supplement or in the accompanying prospectus does not 
constitute a public offering or distribution of securities in Brazil and no registration or filing with respect to any 
securities or financial products described in these documents has been made with the Comissão de Valores 
Mobiliários (the “CVM”). No public offer of securities or financial products described in this prospectus supplement 
or in the accompanying prospectus should be made in Brazil without the applicable registration at the CVM. 
S-17 
Table of Contents
Chile. The notes have not been registered with the Superintendency of Securities and Insurance of Chile, and 
the notes may not be publicly offered in Chile, within the meaning of Chilean Law. 
The People’s Republic of China. This prospectus supplement and the accompanying prospectus have not 
been filed with or approved by the People’s Republic of China (for such purposes, not including Hong Kong and 
Macau Special Administrative Regions or Taiwan) authorities, and is not an offer of securities (whether public 
offering or private placement) within the meaning of the Securities Law or other pertinent laws and regulations of 
the People’s Republic of China. This prospectus supplement and the accompanying prospectus shall not be offered 
to the general public if used within the People’s Republic of China, and the notes so offered cannot be sold to 
anyone that is not a qualified purchaser of the People’s Republic of China. Each selling agent has represented, 
warranted and agreed that the notes are not being offered or sold and may not be offered or sold, directly or 
indirectly, in the People’s Republic of China, except under circumstances that will result in compliance with 
applicable laws and regulations. 
European Economic Area. In relation to each Member State of the European Economic Area which has 
implemented the Prospectus Directive (each, a “Relevant Member State”), each selling agent has represented and 
agreed that with effect from and including the date on which the Prospectus Directive is implemented in that 
Relevant Member State (the “Relevant Implementation Date”) it has not made and will not make an offer of notes 
which are the subject of the offering contemplated by this prospectus supplement and the accompanying prospectus 
to the public in that Relevant Member State except that it may, with effect from and including the Relevant 
Implementation Date, make an offer of such notes to the public in that Relevant Member State: 
(a) at any time to any legal entity which is a qualified investor as defined in the Prospectus Directive; 
(b) at any time to fewer than 100 or, if the Relevant Member State has implemented the relevant provision of 
the 2010 PD Amending Directive, 150, natural or legal persons (other than qualified investors as defined in 
the Prospectus Directive), subject to obtaining the prior consent of the relevant selling agent(s) nominated 
by us for any such offer; or 
(c) at any time in any other circumstances falling within Article 3(2) of the Prospectus Directive,
provided that no such offer of notes referred to in (a) to (c) above shall require us or any selling agent to publish a 
prospectus pursuant to Article 3 of the Prospectus Directive, or supplement a prospectus pursuant to Article 16 of 
the Prospectus Directive. 
For the purposes of this provision, the expression an “offer of notes to the public” in relation to any notes in 
any Relevant Member State means the communication in any form and by any means of sufficient information on 
the terms of the offer and the notes to be offered so as to enable an investor to decide to purchase or subscribe the 
notes, as the same may be varied in that Member State by any measure implementing the Prospectus Directive in 
that Member State, the expression “Prospectus Directive” means Directive 2003/71/EC (and amendments thereto, 
including the 2010 PD Amending Directive, to the extent implemented in the Relevant Member State), and includes 
any relevant implementing measure in the Relevant Member State and the expression “2010 PD Amending 
Directive” means Directive 2010/73/EU. 
S-18 
Table of Contents
France. This prospectus supplement and accompanying prospectus have not been approved by the Autorité 
des marchés financiers (“AMF”). Each of the selling agents has represented and agreed that: 
(a) it has only made and will only make an offer of the notes to the public (offre au public) in France or an 
admission of the notes to trading on a regulated market in France in the period beginning (i) when a 
prospectus in relation to those notes has been approved by the AMF, on the date of such publication, or (ii) 
when a prospectus in relation to those notes has been approved by the competent authority of another 
Member State of the European Economic Area which has implemented the EU Prospectus Directive 
2003/71/EC, on the date of notification of such approval to the AMF and, in either case, when the 
formalities required by French laws and regulations have been carried out, and ending at the latest on the 
date which is 12 months after the date of the approval of the prospectus, all in accordance with articles 
L.412-1 and L.621-8 to L.621-8-3 of the French Code monétaire et financier and the Règlement général of 
the AMF; or 
(b) it has only made and will only make an offer of the notes to the public in France or an admission of the 
notes to trading on a regulated market in France in circumstances which do not require the publication by 
the offeror of a prospectus pursuant to the French Code monétaire et financier and the Règlement général 
of the Autorité des marchés financiers; and 
(c) otherwise, it has not offered or sold and will not offer or sell, directly or indirectly, the notes to the public 
in France, and it has not distributed or caused to be distributed and will not distribute or cause to be 
distributed this prospectus supplement, the accompanying prospectus or any other offering material 
relating to the notes to the public in France, and such offers, sales and distributions have been and will be 
made in France only to (i) providers of the investment service of portfolio management for the account of 
third parties, and/or (ii) qualified investors (investisseurs qualifiés) other than individuals, acting for their 
own account, all as defined in, and in accordance with, articles L.411-2, D.411-1 to D.411-3, D.744-1, 
D.754-1 and D.764-1 of the French Code monétaire et financier. The direct or indirect resale of the notes to 
the public in France may be made only as provided by, and in accordance with, articles L.411-1, L.411-2, 
L.412-1 and L.621-8 to L.621-8-3 of the French Code monétaire et financier. 
In addition, each of the selling agents has represented and agreed that it has not distributed or caused to be 
distributed and will not distribute or cause to be distributed in the Republic of France, this prospectus supplement or 
the accompanying prospectus, or any other offering material relating to the notes other than to investors to whom 
offers and sales of the notes in the Republic of France may be made as described above. 
Hong Kong. Each selling agent has represented and agreed that: 
(a) it has not offered or sold and will not offer or sell in the Hong Kong Special Administrative Region of the 
People’s Republic of China (“Hong Kong”), by means of any document, any notes other than (i) to 
“professional investors” as defined in the Securities and Futures Ordinance (Cap. 571) of Hong Kong (the 
“SFO”) and any rules made under the SFO, or (ii) in other circumstances which do not result in the 
document being a “prospectus” as defined in the Companies Ordinance (Cap. 32) of Hong Kong (the 
“CO”) or which do not constitute an offer to the public within the meaning of the CO; and 
(b) it has not issued or had in its possession for the purposes of issue, and will not issue or have in its 
possession for the purposes of issue, whether in Hong Kong or elsewhere, any 
S-19 
Table of Contents
advertisement, invitation, or document relating to the notes, which is directed at, or the contents of which 
are likely to be accessed or read by, the public in Hong Kong (except if permitted to do so under the 
securities laws of Hong Kong) other than with respect to the notes that are or are intended to be disposed of 
(i) only to persons outside Hong Kong or (ii) only to “professional investors” as defined in the SFO and 
any rules made under the SFO. 
Indonesia. The notes offered have not been and will not be registered under the Indonesian Capital Market 
Law (Law No. 8/1995) and therefore are not authorized by the Capital Market and Financial Institution Supervisory 
Agency in Indonesia as a public offering of securities. Investors who intend to buy the notes should consult with 
their financial advisors, brokers or other financial experts before making any decision to buy the notes. 
Israel. This offer is intended solely for investors listed in the First Supplement of the Israeli Securities Law of 
1968, as amended. A prospectus has not been prepared or filed, and will not be prepared or filed, in Israel relating to 
the notes offered by this prospectus supplement and accompanying prospectus. The notes cannot be resold in Israel 
other than to investors listed in the First Supplement of the Israeli Securities Law of 1968, as amended. 
Italy. The offering of the notes has not been registered with CONSOB—Commissione Nazionale per le 
Società e la Borsa (the Italian Companies and Exchange Commission) pursuant to Italian securities legislation and, 
accordingly, no such notes may be offered, sold or delivered, nor may copies of this prospectus supplement or the 
accompanying prospectus or of any other document relating to the notes be distributed in the Republic of Italy 
except: 
(i) to qualified investors (investitori qualificati), as defined in Article 34-ter, first paragraph, letter b, of 
CONSOB Regulation No. 11971 of 14 May 1999, as amended (“CONSOB Regulation No. 11971”), 
pursuant to Article 100 of Legislative Decree No. 58 of 24 February 1998, as amended (the “Financial 
Services Act”); or 
(ii) in other circumstances which are exempted from the rules on offerings of securities to the public pursuant 
to Article 100 of the Financial Services Act and Article 34-ter, first paragraph, of CONSOB Regulation 
No. 11971. 
Any offer, sale or delivery of the notes or distribution of copies of this prospectus supplement or the 
accompanying prospectus or any other document relating to the notes in the Republic of Italy under (i) or (ii) above 
must be: 
(a) made by an investment firm, bank or financial intermediary permitted to conduct such activities in the 
Republic of Italy in accordance with the Financial Services Act, Legislative Decree No. 385 of 1 
September 1993, as amended (the “Consolidated Banking Act”), and Regulation No. 16190 of 29 October 
2007 (as amended from time to time); 
(b) in compliance with Article 129 of Consolidated Banking Act, as amended, and the implementing 
guidelines of the Bank of Italy, as amended from time to time, pursuant to which the Bank of Italy may 
request information on the issue or the offer of securities in the Republic of Italy; and 
(c) in compliance with any other applicable laws and regulations or requirement imposed by CONSOB or 
other Italian authority. 
S-20 
Table of Contents
Please note that in accordance with Article 100-bis of the Financial Services Act, concerning the circulation of 
financial products, where no exemption from the rules on offerings of securities to the public applies under (i) and 
(ii) above, the subsequent distribution of the notes on the secondary market in Italy must be made in compliance 
with the public offer and the prospectus requirement rules provided under the Financial Services Act and CONSOB 
Regulation No. 11971. Furthermore, Article 100-bis of the Financial Services Act affects the transferability of the 
notes in the Republic of Italy to the extent that any placing of the notes is made solely with qualified investors and 
the notes are then systematically resold to non-qualified investors on the secondary market at any time in the 12 
months following such placing. Where this occurs, if a prospectus has not been published, purchasers of the notes 
who are acting outside of the course of their business or profession may be entitled to declare such purchase null and 
void and to claim damages from any authorised intermediary at whose premises the notes were purchased, unless an 
exemption provided for by the Financial Services Act applies. 
Japan. A securities registration statement under Article 4, Paragraph 1 of the Financial Instruments and 
Exchange Law of Japan (“FIEL”) is not required to be filed in connection with this placement of notes because the 
placement falls under Article 2, Paragraph 3, Item 2-a of the FIEL and therefore such securities registration 
statement has not been filed and will not be filed under the FIEL. 
Any acquiror of the notes who was solicited to buy the notes in Japan is prohibited from transferring any of 
the notes to another person in Japan in any way other than to qualified institutional investors, as defined in Article 2, 
Paragraph 3, Item 1 of the FIEL. 
Korea. The notes have not been and will not be registered under the Financial Investments Services and 
Capital Markets Act of Korea and the decrees and regulations thereunder (the “FSCMA”), and the notes have been 
and will be offered in Korea as a private placement under the FSCMA. None of the notes may be offered, sold and 
delivered directly or indirectly, or offered or sold to any person for re-offering or resale, directly or indirectly, in 
Korea or to any resident of Korea except pursuant to the applicable laws and regulations of Korea, including the 
FSCMA and the Foreign Exchange Transaction Law of Korea and the decrees and regulations thereunder (the 
“FETL”). For a period of one year from the issue date of the notes, any acquirer of the notes who was solicited to 
buy the notes in Korea is prohibited from transferring any of the notes to another person in any way other than as a 
whole to one transferee. Furthermore, the purchaser of the notes shall comply with all applicable regulatory 
requirements (including but not limited to requirements under the FETL) in connection with the purchase of the 
notes. 
Each of the selling agents has represented and agreed that it has not offered, sold or delivered the notes, 
directly or indirectly, or offered or sold the notes to any person for re-offering or resale, directly or indirectly, in 
Korea or to any resident of Korea and will not offer, sell or deliver the notes, directly or indirectly, or offer or sale 
the notes to any person for re-offering or resale, directly or indirectly, in Korea or to any resident of Korea, except 
pursuant to an exemption from the registration requirements of, and otherwise in compliance with, the FSCMA, the 
FETL and other relevant laws and regulations of Korea. 
Mexico. The notes have not been and will not be registered in the National Securities Registry (Registro 
Nacional de Valores). Therefore, the notes may not be offered or sold in the United Mexican States (“Mexico”) by 
any means except in circumstances which do not constitute a public offering (oferta pública) within the meaning of 
the Securities Market Law (Ley del Mercado de Valores) and its regulations. All applicable provisions of the 
Securities Market Law must be complied with in respect to anything done in relation to the notes in, from or 
otherwise involving Mexico. 
S-21 
Table of Contents
Netherlands. We do not have an authorization from the Dutch Central Bank (De Nederlandsche Bank N.V.) 
pursuant to the Dutch Financial Supervision Act (Wet op het financieel toezicht) for the pursuit of the business of a 
bank in the Netherlands and therefore do not have a license pursuant to section 2.1(1), 2.12(1), 2.13(1) or 2.20(1) of 
the Dutch Financial Supervision Act. 
Each selling agent has represented and agreed that it has not made and will not make an offer of the notes to 
the public in the Netherlands other than to qualified investors (gekwalificeerde beleggers), provided that no such 
offer of the notes will require us or any selling agent to publish a prospectus pursuant to Article 3 of the Prospectus 
Directive or supplement a prospectus pursuant to Article 16 of the Prospectus Directive. 
New Zealand. We do not intend that notes be offered for sale or subscription to the public in New Zealand in 
terms of the Securities Act 1978 of New Zealand. Accordingly, no prospectus has been or will be registered, and no 
investment statement will be prepared, under the Securities Act 1978 of New Zealand. The notes shall not be 
directly or indirectly offered for sale or transferred to any member of the public in New Zealand in breach of the 
Securities Act 1978 or the Securities Regulations 1983 of New Zealand. In particular, but without limitation, notes 
may only be offered or transferred either: 
(a) to persons whose principal business is the investment of money or to persons who, in the course of and for 
the purposes of their business, habitually invest money within the meaning of section 3(2)(a)(ii) of the 
Securities Act 1978; or 
(b) to persons who are each required to pay a minimum subscription price of at least NZ$500,000 for the notes 
(disregarding any amount lent by the offeror, us, or any associated person of the offeror or us) before the 
allotment of those notes and who have a minimum holding of the Notes of at least NZ$500,000; or 
(c) to any other persons in circumstances where there is no contravention of the Securities Act 1978, provided 
that notes shall not be offered or sold to any “eligible person” (as defined in section 5(2CC) of the 
Securities Act 1978) unless that person also satisfies the criteria in paragraphs (a) or (b) above. 
In addition, each holder of the notes is deemed to represent and agree that it will not distribute this prospectus 
supplement and the accompanying prospectus and any other document or any other advertisement (as defined in the 
Securities Act 1978) in relation to any offer of the notes in New Zealand other than to any such persons as referred 
to in paragraphs (a) to (c) above. 
Philippines. THE NOTES BEING OFFERED OR SOLD HAVE NOT BEEN REGISTERED WITH 
THE SECURITIES AND EXCHANGE COMMISSION UNDER THE SECURITIES REGULATION 
CODE. ANY FUTURE OFFER OR SALE THEREOF IS SUBJECT TO REGISTRATION 
REQUIREMENTS UNDER THE CODE UNLESS SUCH OFFER OR SALE QUALIFIES AS AN EXEMPT 
TRANSACTION.
Singapore. This prospectus supplement and the accompanying prospectus have not been registered as a 
prospectus with the Monetary Authority of Singapore. Accordingly, this prospectus supplement and the 
accompanying prospectus and any other document or material in connection with the offer or sale, or invitation for 
subscription or purchase, of the notes may not be circulated or distributed, nor may the notes be offered or sold, or 
be made the subject of an invitation for subscription or purchase, whether directly or indirectly, to persons in 
Singapore other than (a) to an institutional investor under Section 274 of the Securities and Futures Act, 
S-22 
Table of Contents
Chapter 289 of Singapore (the “SFA”), (b) to a relevant person pursuant to Section 275(1), or any person pursuant to 
Section 275(1A), and in accordance with the conditions specified in Section 275 of the SFA or (c) otherwise 
pursuant to, and in accordance with the conditions of, any other applicable provision of the SFA. 
Where the notes are subscribed or purchased under Section 275 by a relevant person which is: (a) a 
corporation (which is not an accredited investor (as defined in Section 4A of the SFA)) the sole business of which is 
to hold investments and the entire share capital of which is owned by one or more individuals, each of whom is an 
accredited investor, or (b) a trust (where the trustee is not an accredited investor) whose sole purpose is to hold 
investments and each beneficiary of the trust is an individual who is an accredited investor, then shares, debentures, 
and units of shares and debentures of that corporation or the beneficiaries’ rights and interest (howsoever described) 
in that trust shall not be transferred within six months after that corporation or that trust has acquired the notes 
pursuant to an offer made under Section 275 except: 
(1) to an institutional investor (for corporations, under Section 274 of the SFA) or to a relevant person defined 
in Section 275(2) of the SFA, or to any person pursuant to an offer that is made on terms that such shares, 
debentures, and units of shares and debentures of that corporation or such rights and interest in that trust 
are acquired at a consideration of not less than S$200,000 (or its equivalent in a foreign currency) for each 
transaction, whether such amount is to be paid for in cash or by exchange of securities or other assets, and 
further for corporations, in accordance with the conditions specified in Section 275 of the SFA; 
(2) where no consideration is or will be given for the transfer; or 
(3) where the transfer is by operation of law. 
Switzerland. The notes may not be offered or sold, directly or indirectly, in Switzerland except in 
circumstances that will not result in the offer of the notes being a public offering in Switzerland within the meaning 
of the Swiss Federal Code of Obligations (“CO”). Neither this prospectus supplement and the accompanying 
prospectus nor any other offering or marketing material relating to the notes constitutes a prospectus as that term is 
understood pursuant to Article 652a or 1156 CO, and neither this prospectus supplement and the accompanying 
prospectus nor any other offering material relating to the notes may be publicly distributed or otherwise made 
publicly available in Switzerland. The notes are not authorized by or registered with the Swiss Financial Market 
Supervisory Authority as a foreign collective investment scheme. Therefore, investors do not benefit from protection 
under the Swiss Federal Act on Collective Investment Schemes or supervision by the Swiss Financial Market 
Supervisory Authority. 
Taiwan. The notes may not be issued, sold, or offered in Taiwan. No subscription or other offer to purchase 
the notes shall be binding on us until received and accepted by us or any selling agent outside of Taiwan (the “Place 
of Acceptance”), and the purchase/sale contract arising therefrom shall be deemed a contract entered into in the 
Place of Acceptance. 
United Kingdom. Each selling agent has represented and agreed, and each further selling agent appointed in 
connection with the notes will be required to represent and agree, that: 
(a) in relation to any notes which have a maturity of less than one year (i) it is a person whose ordinary 
activities involve it in acquiring, holding, managing, or disposing of investments (as principal or agent) for 
the purposes of its business and (ii) it has not offered or sold and will not offer or sell any notes other than 
to persons whose ordinary 
S-23 
Table of Contents
activities involve them in acquiring, holding, managing, or disposing of investments (as principal or agent) 
for the purposes of their businesses or who it is reasonable to expect will acquire, hold, manage, or dispose 
of investments (as principal or agent) for the purposes of their businesses, where the issue of the notes 
would otherwise constitute a contravention of section 19 of the Financial Services and Markets Act of 2000 
(the “FSMA”) by us; 
(b) it has only communicated or caused to be communicated and will only communicate or cause to be 
communicated an invitation or inducement to engage in investment activity (within the meaning of section 
21 of the FSMA) received by it in connection with the issue or sale of any notes in circumstances in which 
section 21(1) of the FSMA does not apply to us; and 
(c) it has complied and will comply with all applicable provisions of the FSMA with respect to anything done 
by it in relation to any notes in, from, or otherwise involving the United Kingdom. 
Uruguay. The notes have not been registered under the Uruguayan Securities Market Law or recorded in the 
Uruguayan Central Bank. The notes are not available publicly in Uruguay and are offered only on a private basis. 
No action may be taken in Uruguay that would render any offering of the notes a public offering in Uruguay. No 
Uruguayan regulatory authority has approved the notes or passed on our solvency. In addition, any resale of the 
notes must be made in a manner that will not constitute a public offering in Uruguay. 
Los valores no han sido registrados bajo la Ley de Mercado de Valores de la República Oriental del Uruguay 
o registrados ante el Banco Central del Uruguay. Los valores no son ofrecidos en forma pública en Uruguay y lo 
son únicamente en forma privada. Ninguna acción puede ser adoptada en Uruguay en relación a estos valores que 
resulte en que esta oferta de valores sea una oferta pública de valores en Uruguay. Ninguna autoridad regulatoria 
del Uruguay ha aprobado estos valores o se ha manifestado sobre nuestra solvencia. Adicionalmente, cualquier 
reventa de estos valores debe ser realizada en forma tal que no constituya oferta pública de valores en el Uruguay. 
Venezuela. The notes have not been registered with the Comision Nacional de Valores de Venezuela and are 
not being publicly offered in Venezuela. No document related to the offering of the notes, including this prospectus 
supplement and the accompanying prospectus, shall be interpreted to constitute an offer of securities or an offer or 
the rendering of any investment advice, securities brokerage, and/or banking services in Venezuela. Investors 
wishing to acquire the notes may use only funds located outside of Venezuela. 
LEGAL MATTERS 
The legality of the notes will be passed upon for us by McGuireWoods LLP, Charlotte, North Carolina, and 
for the selling agents by Morrison & Foerster LLP, New York, New York. McGuireWoods LLP regularly performs 
legal services for us. Some members of McGuireWoods LLP performing those legal services own shares of our 
common stock. 
S-24 
Table of Contents
PROSPECTUS 
Debt Securities, Warrants, Units, Purchase Contracts, 
Preferred Stock, Depositary Shares, and Common Stock 
We from time to time may offer to sell debt securities, warrants, purchase contracts, preferred stock, depositary 
shares representing fractional interests in preferred stock, and common stock, as well as units comprised of two or 
more of these securities or securities of third parties. The debt securities, warrants, purchase contracts, and preferred 
stock may be convertible into or exercisable or exchangeable for our common or preferred stock or for debt or 
equity securities of one or more other entities. Our common stock is listed on the New York Stock Exchange under 
the symbol “BAC.” In addition, our common stock is listed on the London Stock Exchange, and certain shares are 
listed on the Tokyo Stock Exchange. 
This prospectus describes the general terms of these securities and the general manner in which we will offer the 
securities. When we sell a particular series of securities, we will prepare one or more supplements to this prospectus 
describing the offering and the specific terms of that series of securities. You should read this prospectus and any 
applicable supplement carefully before you invest. 
We may use this prospectus in the initial sale of these securities. In addition, Merrill Lynch, Pierce, Fenner & Smith 
Incorporated, or any of our other affiliates, may use this prospectus in a market-making transaction in any of these 
securities after their initial sale. Unless you are informed otherwise in the confirmation of sale, this prospectus is 
being used in a market-making transaction. 
Potential purchasers of our securities should consider the information set forth in the “Risk Factors” section 
beginning on page 8. 
Our securities are unsecured and are not savings accounts, deposits, or other obligations of a bank, are not 
guaranteed by Bank of America, N.A. or any other bank, are not insured by the Federal Deposit Insurance 
Corporation or any other governmental agency, and may involve investment risks, including possible loss of 
principal. 
Neither the Securities and Exchange Commission nor any state securities commission has approved or disapproved 
of these securities or passed upon the adequacy or accuracy of this prospectus. Any representation to the contrary is 
a criminal offense. 
Prospectus dated March 30, 2012 
Table of Contents
TABLE OF CONTENTS 
Page
About this Prospectus 3
Prospectus Summary 4
Risk Factors 8
Currency Risks 8
Other Risks 10
Bank of America Corporation 11
Use of Proceeds 11
Description of Debt Securities 12
General 12
The Indentures 12
Form and Denomination of Debt Securities 13
Different Series of Debt Securities 14
Fixed-Rate Notes 15
Floating-Rate Notes 15
Indexed Notes 23
Floating-Rate/Fixed-Rate/Indexed Notes 24
Original Issue Discount Notes 24
Payment of Principal, Interest, and Other Amounts Due 24
No Sinking Fund 27
Redemption 27
Repayment 27
Repurchase 27
Conversion 28
Exchange, Registration, and Transfer 28
Subordination 28
Sale or Issuance of Capital Stock of Banks 29
Limitation on Mergers and Sales of Assets 30
Waiver of Covenants 30
Modification of the Indentures 30
Meetings and Action by Securityholders 31
Events of Default and Rights of Acceleration 31
Collection of Indebtedness 31
Payment of Additional Amounts 32
Redemption for Tax Reasons 35
Defeasance and Covenant Defeasance 35
Notices 36
Concerning the Trustees 37
Governing Law 37
Description of Warrants 37
General 37
Description of Debt Warrants 37
Description of Universal Warrants 38
Modification 39
Enforceability of Rights of Warrantholders; No Trust Indenture Act Protection 39
Unsecured Obligations 40
Description of Purchase Contracts 40
General 40
Purchase Contract Property 40
Information in Supplement 41
Prepaid Purchase Contracts; Applicability of Indenture 42
Non-Prepaid Purchase Contracts; No Trust Indenture Act Protection 42
Pledge by Holders to Secure Performance 43
Settlement of Purchase Contracts That Are Part of Units 43
Failure of Holder to Perform Obligations 43
Unsecured Obligations 43
Page
Description of Units 44
General 44
Unit Agreements: Prepaid, Non-Prepaid, and Other 45
Modification 45
Enforceability of Rights of Unitholders; No Trust Indenture Act Protection 45
Unsecured Obligations 46
Description of Preferred Stock 46
General 46
The Preferred Stock 48
Description of Depositary Shares 49
General 49
Terms of the Depositary Shares 50
Withdrawal of Preferred Stock 50
Dividends and Other Distributions 50
Redemption of Depositary Shares 51
Voting the Deposited Preferred Stock 51
Amendment and Termination of the Deposit Agreement 51
Charges of Depository 52
Miscellaneous 52
Resignation and Removal of Depository 52
Description of Common Stock 52
General 52
Voting and Other Rights 53
Dividends 53
Registration and Settlement 54
Book-Entry Only Issuance 54
Certificates in Registered Form 54
Street Name Owners 55
Legal Holders 55
Special Considerations for Indirect Owners 55
Depositories for Global Securities 56
Special Considerations for Global Securities 60
Registration, Transfer, and Payment of Certificated Securities 61
U.S. Federal Income Tax Considerations 62
Taxation of Debt Securities 63
Taxation of Common Stock, Preferred Stock, and Depositary Shares 78
Taxation of Warrants 84
Taxation of Purchase Contracts 84
Taxation of Units 84
Reportable Transactions 84
Foreign Account Tax Compliance Act 85
EU Directive on the Taxation of Savings Income 86
Plan of Distribution (Conflicts of Interest) 87
Distribution Through Underwriters 87
Distribution Through Dealers 88
Distribution Through Agents 88
Direct Sales 88
General Information 88
Market-Making Transactions by Affiliates 89
Conflicts of Interest 89
ERISA Considerations 91
Where You Can Find More Information 93
Forward-Looking Statements 94
Legal Matters 94
Experts 95
2 
Table of Contents
ABOUT THIS PROSPECTUS 
This prospectus is part of a registration statement that we filed with the Securities and Exchange Commission, 
or the “SEC,” utilizing a “shelf” registration process. Under this shelf process, we may, from time to time, sell any 
combination of the securities described in this prospectus or the registration statement in one or more offerings. 
This prospectus provides you with a general description of securities we may offer. Each time we sell 
securities, we will provide one or more prospectus supplements, product supplements, pricing supplements (each of 
which we may refer to as a “term sheet”), and/or index supplements that describe the particular securities offering 
and the specific terms of the securities being offered. These documents also may add, update, or change information 
contained in this prospectus. In this prospectus, when we refer to the “applicable supplement” or the “accompanying 
supplement,” we mean the prospectus supplement or supplements, as well as any applicable pricing, product, or 
index supplements, that describe the particular securities being offered to you. If there is any inconsistency between 
the information in this prospectus and the applicable supplement, you should rely on the information in the 
applicable supplement. 
The information in this prospectus is not complete and may be changed. You should rely only on the 
information provided in or incorporated by reference in this prospectus, the accompanying supplement, or 
documents to which we otherwise refer you. We are not making an offer of these securities in any jurisdiction where 
the offer or sale is not permitted. You should assume that the information appearing in this prospectus and the 
accompanying supplement, as well as information we have filed or will file with the SEC and incorporated by 
reference in this prospectus, is accurate as of the date of the applicable document or other date referred to in that 
document. Our business, financial condition, and results of operations may have changed since that date. 
Unless we indicate otherwise or unless the context requires otherwise, all references in this prospectus to 
“Bank of America,” “we,” “us,” “our,” or similar references are to Bank of America Corporation excluding its 
consolidated subsidiaries. 
References in this prospectus to “$” and “dollars” are to the currency of the United States of America; and 
references in this prospectus to “€” and “euro” are to the currency introduced at the start of the third stage of the 
European Economic and Monetary Union pursuant to Article 109g of the Treaty establishing the European 
Community, as amended by the Treaty on European Union, as amended by the Treaty of Amsterdam. 
3 
Table of Contents
PROSPECTUS SUMMARY 
This summary section highlights selected information from this prospectus. This summary does not contain all 
the information that you should consider before investing in the securities we may offer using this prospectus. To 
fully understand the securities we may offer, you should read carefully: 
• this prospectus, which explains the general terms of the securities we may offer; 
• the applicable supplement, which explains the specific terms of the particular securities we are offering, and 
which may update or change the information in this prospectus; and 
• the documents we refer to in “Where You Can Find More Information” below for information about us, 
including our financial statements. 
Bank of America Corporation 
Bank of America Corporation is a Delaware corporation, a bank holding company, and a financial holding 
company. We provide a diversified range of banking and nonbanking financial services and products both 
domestically and internationally. Our headquarters is located at Bank of America Corporate Center, 100 North 
Tryon Street, Charlotte, North Carolina 28255 and our telephone number is (704) 386-5681. 
The Securities We May Offer 
We may use this prospectus to offer any of the following securities from time to time: 
• debt securities; 
• warrants; 
• purchase contracts; 
• preferred stock; 
• depositary shares representing fractional interests in preferred stock; 
• common stock; and 
• units, comprised of two or more of any of the securities referred to above, in any combination. 
When we use the term “securities” in this prospectus, we mean any of the securities we may offer with this 
prospectus, unless we specifically state otherwise. This prospectus, including this summary, describes the general 
terms of the securities we may offer. Each time we sell securities, we will provide you with the applicable 
supplement or supplements that will describe the offering and the specific terms of the securities being offered. A 
supplement may include a discussion of additional U.S. federal income tax consequences and any additional risk 
factors or other special considerations applicable to those particular securities. 
Debt Securities 
Our debt securities may be either senior or subordinated obligations in right of payment. Our senior and 
subordinated debt securities will be issued under separate indentures, or contracts, that 
4 
Table of Contents
we have with The Bank of New York Mellon Trust Company, N.A., as successor trustee. The particular terms of 
each series of debt securities will be described in the applicable supplement. 
Warrants 
We may offer two types of warrants: 
• warrants to purchase our debt securities; and 
• warrants to purchase or sell, or whose cash value is determined by reference to the performance, level, or 
value of, one or more of the following: 
• securities of one or more issuers, including our common or preferred stock, other securities described in 
this prospectus, or the debt or equity securities of third parties; 
• one or more currencies, currency units, or composite currencies; 
• one or more commodities; 
• any other financial, economic, or other measure or instrument, including the occurrence or nonoccurrence of any event or circumstance; and 
• one or more indices or baskets of the items described above. 
For any warrants we may offer, we will describe in the applicable supplement the underlying property, the 
expiration date, the exercise price or the manner of determining the exercise price, the amount and kind, or the 
manner of determining the amount and kind, of property to be delivered by you or us upon exercise, and any other 
specific terms of the warrants. We will issue warrants under warrant agreements that we will enter into with one or 
more warrant agents. 
Purchase Contracts 
We may offer purchase contracts requiring holders to purchase or sell, or whose cash value is determined by 
reference to the performance, level, or value of, one or more of the following: 
• securities of one or more issuers, including our common or preferred stock, other securities described in this 
prospectus, or the debt or equity securities of third parties; 
• one or more currencies, currency units, or composite currencies; 
• one or more commodities; 
• any other financial, economic, or other measure or instrument, including the occurrence or non-occurrence 
of any event or circumstance; and 
• one or more indices or baskets of the items described above. 
For any purchase contracts we may offer, we will describe in the applicable supplement the underlying 
property, the settlement date, the purchase price, or manner of determining the purchase price and whether it must be 
paid when the purchase contract is issued or at a later date, the amount and kind, or manner of determining the 
amount and kind, of property to be delivered at settlement, whether the holder will pledge property to secure the 
performance of any obligations the holder may have under the purchase contract, and any other specific terms of the 
purchase contracts. 
5 
Table of Contents
Units 
We may offer units consisting of any combination of two or more debt securities, warrants, purchase 
contracts, shares of preferred stock, depositary shares, and common stock described in this prospectus as well as 
securities of third parties. For any units we may offer, we will describe in the applicable supplement the particular 
securities that comprise each unit, whether or not the particular securities will be separable and, if they will be 
separable, the terms on which they will be separable, a description of the provisions for the payment, settlement, 
transfer, or exchange of the units, and any other specific terms of the units. We will issue units under unit 
agreements that we will enter into with one or more unit agents. 
Preferred Stock and Depositary Shares 
We may offer our preferred stock in one or more series. For any particular series we may offer, we will 
describe in the applicable supplement: 
• the specific designation; 
• the aggregate number of shares offered; 
• the dividend rate and periods, or manner of calculating the dividend rate and periods, if any; 
• the stated value and liquidation preference amount, if any; 
• the voting rights, if any; 
• the terms on which the series of preferred stock is convertible into shares of our common stock, preferred 
stock of another series, or other securities, if any; 
• the redemption terms, if any; and 
• any other specific terms of the series. 
We also may offer depositary shares, each of which will represent a fractional interest in a share or multiple 
shares of our preferred stock. We will describe in the applicable supplement any specific terms of the depositary 
shares. We will issue the depositary shares under deposit agreements that we will enter into with one or more 
depositories. 
Form of Securities 
Unless we specify otherwise in the applicable supplement, we will issue the securities, other than shares of our 
common stock, in book-entry only form through one or more depositories, such as The Depository Trust Company, 
Euroclear Bank SA/NV, or Clearstream Banking, société anonyme, Luxembourg, as identified in the applicable 
supplement. We will issue the securities only in registered form, without coupons, although we may issue the 
securities in bearer form if we so specify in the applicable supplement. The securities issued in book-entry only form 
will be represented by a global security registered in the name of the specified depository, rather than notes or 
certificates registered in the name of each individual investor. Unless we specify otherwise in the applicable 
supplement, each sale of securities in book-entry form will settle in immediately available funds through the 
specified depository. 
A global security may be exchanged for actual notes or certificates registered in the names of the beneficial 
owners only under the limited circumstances described in this prospectus. 
6 
Table of Contents
Payment Currencies 
All amounts payable in respect of the securities, including the purchase price, will be payable in U.S. dollars, 
unless we specify otherwise in the applicable supplement. 
Listing 
We will state in the applicable supplement whether the particular securities that we are offering will be listed 
or quoted on a securities exchange or quotation system. 
Distribution 
We may offer the securities under this prospectus: 
• through underwriters; 
• through dealers; 
• through agents; or 
• directly to purchasers. 
The applicable supplement will include any required information about the firms we use and the discounts or 
commissions we may pay them for their services. 
Merrill Lynch, Pierce, Fenner & Smith Incorporated, or any of our other affiliates, may be an underwriter, 
dealer, or agent for us. 
Market-Making by Our Affiliates 
Following the initial distribution of an offering of securities, Merrill Lynch, Pierce, Fenner & Smith 
Incorporated, and other affiliates of ours may offer and sell those securities in the course of their businesses as 
broker-dealers. Merrill Lynch, Pierce, Fenner & Smith Incorporated and any such other affiliates may act as a 
principal or agent in these transactions. This prospectus and the applicable supplement or supplements also will be 
used in connection with these market-making transactions. Sales in any of these market-making transactions will be 
made at varying prices related to prevailing market prices and other circumstances at the time of sale. 
If you purchase securities in a market-making transaction, you will receive information about the purchase 
price and your trade and settlement dates in a separate confirmation of sale. 
7 
Table of Contents
RISK FACTORS 
This section summarizes some specific risks and investment considerations with respect to an investment in 
our securities. This summary does not describe all of the risks and investment considerations with respect to an 
investment in our securities, including risks and considerations relating to a prospective investor’s particular 
circumstances. For information regarding risks and uncertainties that may materially affect our business and 
results, please refer to the information under the captions “Item 1A. Risk Factors” and “Item 7. Management’s 
Discussion and Analysis of Financial Condition and Results of Operations” in our annual report on Form 10-K for 
the year ended December 31, 2011, which is incorporated by reference in this prospectus. You should also review 
the risk factors that will be set forth in other documents that we will file after the date of this prospectus, together 
with the risk factors set forth in any applicable supplement. Prospective investors should consult their own financial, 
legal, tax, and other professional advisors as to the risks associated with an investment in our securities and the 
suitability of the investment for the investor. 
Currency Risks 
We may issue securities denominated in or whose principal and interest is payable in a currency other than 
U.S. dollars. We refer to these securities as “Non-U.S. Dollar-Denominated Securities.” If you intend to invest in 
any Non-U.S. Dollar-Denominated Securities, you should consult your own financial and legal advisors as to the 
currency risks related to your investment. The Non-U.S. Dollar-Denominated Securities are not an appropriate 
investment for you if you are not knowledgeable about the significant terms and conditions of the Non-U.S. DollarDenominated Securities or financial matters in general. The information in this prospectus is directed primarily to 
investors who are U.S. residents. Investors who are not U.S. residents should consult their own financial and legal 
advisors about currency-related risks arising from their investment. 
Non-U.S. Dollar-Denominated Securities have significant risks that are not associated with a similar 
investment in conventional debt securities that are payable solely in U.S. dollars. These risks include possible 
significant changes in rates of exchange between the U.S. dollar and the specified currency and the imposition or 
modification of foreign exchange controls or other conditions by either the United States or non-U.S. governments. 
These risks generally are influenced by factors over which we have no control, such as economic and political 
events and the supply of and demand for the relevant currencies in the global markets. 
Currency Exchange Rates. Exchange rates between the U.S. dollar and other currencies have been highly 
volatile. This volatility may continue and could spread to other currencies in the future. Fluctuations in currency 
exchange rates could affect adversely an investment in the Non-U.S. Dollar-Denominated Securities. Depreciation 
of the specified currency against the U.S. dollar could result in a decrease in the U.S. dollar-equivalent value of 
payments on the Non-U.S. Dollar-Denominated Securities. That in turn could cause the market value of the NonU.S. Dollar-Denominated Securities to fall. 
Changes in Foreign Currency Exchange Rates. Except as described below or in a supplement, we will not 
make any adjustment in or change to the terms of the Non-U.S. Dollar-Denominated Securities for changes in the 
foreign currency exchange rate for the specified currency, including any devaluation, revaluation, or imposition of 
exchange or other regulatory controls or taxes, or for other developments affecting the specified currency, the U.S. 
dollar, or any other currency. Consequently, you will bear the risk that your investment may be affected adversely 
by these types of events. 
Government Policy. Foreign currency exchange rates either can float or be fixed by sovereign governments. 
Governments or governmental bodies, including the European Central 
8 
Table of Contents
Bank, may intervene in their economies to alter the exchange rate or exchange characteristics of their currencies. For 
example, a central bank may intervene to devalue or revalue a currency or to replace an existing currency. In 
addition, a government may impose regulatory controls or taxes to affect the exchange rate of its currency. As a 
result, the yield or payout of a Non-U.S. Dollar-Denominated Security could be affected significantly and 
unpredictably by governmental actions. Changes in exchange rates could affect the value of the Non-U.S. DollarDenominated Securities as participants in the global currency markets move to buy or sell the specified currency or 
U.S. dollars in reaction to these developments. 
If a governmental authority imposes exchange controls or other conditions, such as taxes on the transfer of the 
specified currency, there may be limited availability of the specified currency for payment on the Non-U.S. DollarDenominated Securities at their maturity or on any other payment date. In addition, the ability of a holder to move 
currency freely out of the country in which payment in the currency is received or to convert the currency at a freely 
determined market rate could be limited by governmental actions. 
Payments in U.S. Dollars. The terms of any Non-U.S. Dollar-Denominated Securities may provide that we 
may have the right to make a payment in U.S. dollars instead of the specified currency, if at or about the time when 
the payment on the Non-U.S. Dollar-Denominated Securities comes due, the specified currency is subject to 
convertibility, transferability, market disruption, or other conditions affecting its availability because of 
circumstances beyond our control. These circumstances could include the imposition of exchange controls or our 
inability to obtain the specified currency because of a disruption in the currency markets for the specified currency. 
The exchange rate used to make payment in U.S. dollars may be based on limited information and would involve 
significant discretion on the part of our exchange rate agent. As a result, the value of the payment in U.S. dollars 
may be less than the value of the payment you would have received in the specified currency if the specified 
currency had been available. The exchange rate agent will generally not have any liability for its determinations. 
Court Judgments. Any Non-U.S. Dollar-Denominated Securities typically will be governed by New York 
law. Under Section 27 of the New York Judiciary Law, a state court in the State of New York rendering a judgment 
on the Non-U.S. Dollar-Denominated Debt Securities would be required to render the judgment in the specified 
currency. In turn, the judgment would be converted into U.S. dollars at the exchange rate prevailing on the date of 
entry of the judgment. Consequently, in a lawsuit for payment on the Non-U.S. Dollar-Denominated Securities, you 
would bear currency exchange risk until judgment is entered, which could be a long time. 
In courts outside of New York, you may not be able to obtain judgment in a specified currency other than U.S. 
dollars. For example, a judgment for money in an action based on Non-U.S. Dollar-Denominated Securities in many 
other U.S. federal or state courts ordinarily would be enforced in the United States only in U.S. dollars. The date 
used to determine the rate of conversion of the specified currency into U.S. dollars will depend on various factors, 
including which court renders the judgment. 
Information About Foreign Currency Exchange Rates. If we issue a Non-U.S. Dollar-Denominated 
Security, we may include in the applicable supplement information about historical exchange rates for the relevant 
non-U.S. dollar currency or currencies. Any information about exchange rates that we may provide will be furnished 
as a matter of information only, and you should not regard the information as indicative of the range of, or trends in, 
fluctuations in currency exchange rates that may occur in the future. 
9 
Table of Contents
Other Risks 
Possible Illiquidity of the Secondary Market. We may not list our securities on any securities exchange. We 
cannot predict how these securities will trade in the secondary market or whether that market will be liquid or 
illiquid. The number of potential buyers of our securities in any secondary market may be limited. Although any 
underwriters or agents may purchase and sell our securities in the secondary market from time to time, these 
underwriters or agents will not be obligated to do so and may discontinue making a market for the securities at any 
time without giving us notice. We cannot assure you that a secondary market for any of our securities will develop, 
or that if one develops, it will be maintained. 
Redemption. The terms of our securities may permit or require redemption of the securities prior to maturity. 
That redemption may occur at a time when prevailing interest rates are relatively low. As a result, in the case of debt 
or similar securities, a holder of the redeemed securities may not be able to invest the redemption proceeds in a new 
investment that yields a similar return. 
Credit Ratings. Our credit ratings are an assessment of our ability to pay our obligations. Consequently, real 
or anticipated changes in our credit ratings may affect the trading value of our securities. However, because the 
return on our securities generally depends upon factors in addition to our ability to pay our obligations, an 
improvement in these credit ratings will not reduce the other investment risks, if any, related to our securities. 
10 
Table of Contents
BANK OF AMERICA CORPORATION 
Bank of America Corporation is a Delaware corporation, a bank holding company, and a financial holding 
company under the Gramm-Leach-Bliley Act. Our principal executive offices are located in the Bank of America 
Corporate Center, 100 North Tryon Street, Charlotte, North Carolina 28255 and our telephone number is (704) 386-
5681. Through our banking and various nonbanking subsidiaries throughout the United States and in certain 
international markets, we provide a diversified range of banking and nonbanking financial services and products 
through six business segments: Deposits, Card Services, Consumer Real Estate Services, Global Commercial 
Banking, Global Banking & Markets and Global Wealth & Investment Management. 
USE OF PROCEEDS 
Unless we describe a different use in the applicable supplement, we will use the net proceeds from the sale of 
the securities for general corporate purposes. General corporate purposes include, but are not limited to, the 
following: 
• our working capital needs; 
• the funding of investments in, or extensions of credit to, our subsidiaries; 
• possible reductions, redemptions or repurchases of our outstanding indebtedness; 
• possible repayments on outstanding indebtedness; 
• the possible acquisitions of, or investments in, other financial institutions or other businesses of a type we 
are permitted to acquire under applicable law; and 
• other uses in the ordinary course of conducting our business. 
Until we designate the use of these net proceeds, we will invest them temporarily. From time to time, we may 
engage in additional financings as we determine appropriate based on our needs and prevailing market conditions. 
These additional financings may include the sale of other securities. 
11 
Table of Contents
DESCRIPTION OF DEBT SECURITIES 
General 
We may issue senior or subordinated debt securities. Neither the senior debt securities nor the subordinated 
debt securities will be secured by any of our property or assets. As a result, by owning a debt security, you are one 
of our unsecured creditors. 
The senior debt securities will constitute part of our senior debt, will be issued under our senior debt indenture 
described below, and will rank equally with all of our other unsecured and unsubordinated debt. 
The subordinated debt securities will constitute part of our subordinated debt, will be issued under our 
subordinated debt indenture described below, and will be subordinated in right of payment to all of our “senior 
indebtedness,” as defined in the subordinated debt indenture. Neither the senior debt indenture nor the subordinated 
debt indenture limits our ability to incur additional “senior indebtedness.” 
The Indentures 
The senior debt securities and the subordinated debt securities each are governed by a document called an 
indenture, which is a contract between us and the applicable trustee. Senior debt securities will be issued under the 
Indenture dated as of January 1, 1995 (as supplemented, the “Senior Indenture”) between us and The Bank of New 
York Mellon Trust Company, N.A., as successor trustee, and subordinated debt securities will be issued under the 
Indenture dated as of January 1, 1995 (as supplemented, the “Subordinated Indenture”) between us and The Bank of 
New York Mellon Trust Company, N.A., as successor trustee. The indentures are substantially identical, except for: 
• the covenant described below under “—Sale or Issuance of Capital Stock of Banks,” which is included only 
in the Senior Indenture; 
• the provisions relating to subordination described below under “—Subordination,” which are included only 
in the Subordinated Indenture; and 
• the events of default described below under “—Events of Default and Rights of Acceleration,” many of 
which are not included in the Subordinated Indenture. 
In this prospectus, when we refer to “debt securities,” we mean both our senior debt securities and our 
subordinated debt securities, and when we refer to the “indenture” or the “trustee” with respect to any debt 
securities, we mean the indenture under which those debt securities are issued and the trustee under that indenture. 
The trustee under each indenture has two principal functions: 
• First, the trustee can enforce your rights against us if we default. However, there are limitations on the 
extent to which the trustee may act on your behalf, which we describe below under “—Collection of 
Indebtedness.” 
• Second, the trustee performs administrative duties for us, including the delivery of interest payments and 
notices. 
Neither indenture limits the aggregate amount of debt securities that we may issue or the number of series or 
the aggregate amount of any particular series. The indentures and the debt 
12 
Table of Contents
securities also do not limit our ability to incur other indebtedness or to issue other securities. This means that we 
may issue additional debt securities and other securities at any time without your consent and without notifying you. 
In addition, neither indenture contains provisions protecting holders against a decline in our credit quality resulting 
from takeovers, recapitalizations, the incurrence of additional indebtedness, or restructuring. If our credit quality 
declines as a result of an event of this type, or otherwise, any ratings of our debt securities then outstanding may be 
withdrawn or downgraded. 
This section is a summary of the indentures and is subject to and qualified in its entirety by reference to all the 
provisions of the indentures. We have filed the indentures with the SEC as exhibits to our registration statement, and 
they are incorporated in this prospectus by reference. See “Where You Can Find More Information” below for 
information on how to obtain copies of the indentures. Whenever we refer to the defined terms of the indentures in 
this prospectus or in a supplement without defining them, the terms have the meanings given to them in the 
indentures. You must look to the indentures for the most complete description of the information summarized in this 
prospectus. 
Form and Denomination of Debt Securities 
Unless we specify otherwise in the applicable supplement, we will issue each debt security in global, or bookentry, form. Debt securities in book-entry form will be represented by a global security registered in the name of a 
depository. Accordingly, the depository will be the holder of all the debt securities represented by the global 
security. Those who own beneficial interests in a global security will do so through participants in the depository’s 
securities clearing system, and the rights of these indirect owners will be governed solely by the applicable 
procedures of the depository and its participants. We describe the procedures applicable to book-entry securities
below under the heading “Registration and Settlement.” 
Generally, all securities represented by the same global security will have the same terms. We may, however, 
issue a global security that represents multiple debt securities that have different terms and are issued at different 
times. We call this kind of global security a master global security. Your prospectus supplement will not indicate 
whether your debt securities are represented by a master global security. 
Unless we specify otherwise in the applicable supplement, we will issue our debt securities in fully registered 
form, without coupons. If we issue a debt security in bearer form, we will describe the special considerations 
applicable to bearer securities in the applicable supplement. Some of the features that we describe in this prospectus 
may not apply to the bearer securities. 
Our debt securities may be denominated, and cash payments with respect to the debt securities may be made, 
in U.S. dollars or in another currency, or in a composite currency, a basket of currencies, or a currency unit or units. 
Unless we specify otherwise in the applicable supplement, the debt securities will be denominated, and cash 
payments with respect to the debt securities will be made, in U.S. dollars, and the debt securities ordinarily will be 
issued in denominations of $1,000 and multiples of $1,000 in excess of $1,000. We may also issue debt securities 
that are denominated in units of $10. If any of the debt securities are denominated, or if principal, any premium, 
interest, and any other amounts payable on any of the debt securities is payable, in a foreign currency, or in a 
composite currency, a basket of currencies, or a currency unit or units, the specified currency, as well as any 
additional investment considerations, risk factors, restrictions, tax consequences, specific terms and other 
information relating to that issue of debt securities and the specified currency, composite currency, basket of 
currencies, or currency unit or units, may be described in the applicable supplement. We describe some of those 
investment considerations relating to securities denominated or payable in a currency other than U.S. dollars above 
under the heading “Risk Factors.” 
13 
Table of Contents
Different Series of Debt Securities 
We may issue our debt securities from time to time in one or more series with the same or different maturities. 
We also may “reopen” a series of our debt securities. This means that we can increase the principal amount of a 
series of our debt securities by selling additional debt securities with the same terms. We may do so without notice 
to the existing holders of securities of that series. However, any new securities of this kind may begin to bear 
interest at a different date. 
This section of the prospectus summarizes the material terms of the debt securities that are common to all 
series. We will describe the financial and other specific terms of the series of debt securities being offered in the 
applicable supplement. The supplement also may describe any differences from the material terms described in this 
prospectus. If there are any differences between the applicable supplement and this prospectus, the applicable 
supplement will control. 
The terms of your series of debt securities as described in the applicable supplement may include the 
following: 
• the title and type of the debt securities; 
• the principal amount of the debt securities; 
• the minimum denominations, if other than $1,000 and multiples of $1,000 in excess of $1,000; 
• the percentage of the stated principal amount at which the debt securities will be sold and, if applicable, the 
method of determining the price; 
• the person to whom interest is payable, if other than the owner of the debt securities; 
• the maturity date or dates; 
• the interest rate or rates, which may be fixed or variable, and the method used to calculate that interest; 
• any index or other reference asset or assets that will be used to determine the amounts of any payments on 
the debt securities and the manner in which those amounts will be determined; 
• the interest payment dates, the regular record dates for the interest payment dates, and the date interest will 
begin to accrue; 
• the place or places where payments on the debt securities may be made and the place or places where the 
debt securities may be presented for registration of transfer or exchange; 
• any date or dates after which the debt securities may be redeemed, repurchased, or repaid in whole or in part 
at our option or the option of the holder, and the periods, prices, terms, and conditions of that redemption, 
repurchase, or repayment; 
• if other than the full principal amount, the portion of the principal amount of the debt securities that will be 
payable if their maturity is accelerated; 
• the currency of principal, any premium, interest and any other amounts payable on the debt securities, if 
other than U.S. dollars; 
14 
Table of Contents
• if the debt securities will be issued in other than book-entry form; 
• the identification of or method of selecting any calculation agents, exchange rate agents, or any other agents 
for the debt securities; 
• any provisions for the discharge of our obligations relating to the debt securities by the deposit of funds or 
U.S. government obligations; 
• any provisions relating to the extension or renewal of the maturity date of the debt securities; 
• whether the debt securities will be listed on any securities exchange; or 
• any other terms of the debt securities that are permitted under the applicable indenture. 
Fixed-Rate Notes 
General. We may issue debt securities that bear interest at one or more fixed rates of interest, as specified in 
the applicable supplement. We refer to these as “fixed-rate notes.” Unless we specify otherwise in the applicable 
supplement, each fixed-rate note will bear interest from its original issue date or from the most recent date to which 
interest on the note has been paid or made available for payment. Interest will accrue on the principal of a fixed-rate 
note at the fixed annual rate stated in the applicable supplement, until the principal is paid or made available for 
payment or the note is converted or exchanged. 
Unless we specify otherwise in the applicable supplement, we will pay interest on any fixed-rate note 
quarterly, semi-annually, or annually, as applicable, in arrears, on the days set forth in the applicable supplement 
(each such day being an “interest payment date”) and at maturity. Each interest payment due on an interest payment 
date or the maturity date will include interest accrued from and including the most recent interest payment date to 
which interest has been paid, or, if no interest has been paid, from the original issue date, to but excluding the next 
interest payment date or the maturity date, as the case may be. Unless we specify otherwise in the applicable 
supplement, interest on fixed-rate notes will be computed and paid on the basis of a 360-day year consisting of 
twelve 30-day months. We will make payments on fixed-rate notes as described below under the heading “—
Payment of Principal, Interest, and Other Amounts Due.” 
Amortizing Notes. We also may issue amortizing notes, which are fixed-rate notes for which combined 
principal and interest payments are made in installments over the life of the debt security. Payments on amortizing 
notes are applied first to interest due and then to the reduction of the unpaid principal amount. The supplement for 
an amortizing note will include a table setting forth repayment information. 
Floating-Rate Notes 
General. We may issue debt securities that will bear interest at a floating rate of interest determined by 
reference to one or more interest rate bases, or by reference to one or more interest rate formulae, referred to as the 
“base rate.” We refer to these debt securities as “floating-rate notes.” The base rate may be one or more of the 
following: 
• the federal funds rate, in which case the debt security will be a “federal funds rate note”; 
• the London interbank offered rate, in which case the debt security will be a “LIBOR note”; 
15 
Table of Contents
• the euro interbank offered rate, in which case the debt security will be a “EURIBOR note”; 
• the prime rate, in which case the debt security will be a “prime rate note”; 
• the treasury rate, in which case the debt security will be a “treasury rate note”; or 
• any other interest rate formula as may be specified in the applicable supplement. 
The interest rate for a floating-rate note will be determined by reference to: 
• the specified base rate based on the index maturity; 
• plus or minus the spread, if any; and/or 
• multiplied by the spread multiplier, if any. 
For any floating-rate note, the “index maturity” is the period to maturity of the instrument for which the 
interest rate basis is calculated and will be specified in the applicable supplement. The “spread” is the number of 
basis points we specify on the floating-rate note to be added to or subtracted from the base rate. The “spread 
multiplier” is the percentage we may specify on the floating-rate note by which the base rate is multiplied in order to 
calculate the applicable interest rate. 
A floating-rate note also may be subject to: 
• a maximum interest rate limit, or ceiling, on the interest that may accrue during any interest period; 
• a minimum interest rate limit, or floor, on the interest that may accrue during any interest period; or 
• both. 
Unless we specify otherwise in the applicable supplement, each floating-rate note will bear interest from its 
original issue date or from the most recent date to which interest on the note has been paid or made available for 
payment. Interest will accrue on the principal of a floating-rate note at the annual rate determined according to the 
interest rate formula stated in the applicable supplement, until the principal is paid or made available for payment. 
Unless we specify otherwise in the applicable supplement, we will pay interest on any floating-rate note monthly, 
quarterly, semi-annually, or annually, as applicable, in arrears, on the days set forth in the applicable supplement. 
Unless we specify otherwise in the applicable supplement, each interest payment due on an interest payment date or 
the maturity date will include interest accrued from and including the most recent interest payment date to which 
interest has been paid, or, if no interest has been paid, from the original issue date, to but excluding the next interest 
payment date or the maturity date, as the case may be. We will make payments on floating-rate notes as described 
below under the heading “—Payment of Principal, Interest, and Other Amounts Due.” 
How Interest Is Reset. The interest rate in effect from the date of issue to the first interest reset date for a 
floating-rate note will be the initial interest rate determined as described in the applicable supplement. The interest 
rate of each floating-rate note may be reset daily, weekly, monthly, quarterly, semi-annually, or annually, as we 
specify in the applicable supplement. We refer to the period during which an interest rate is effective as an “interest 
period,” and the first day of each interest period as the “interest reset date.” 
16
Table of Contents
The “interest determination date” for any interest reset date is the day the calculation agent will refer to when 
determining the new interest rate at which a floating rate will reset. Unless we specify otherwise in the applicable 
supplement, the interest determination date for an interest reset date will be: 
• for a federal funds rate note or a prime rate note, the business day immediately preceding the interest reset 
date; 
• for a LIBOR note, the second London Banking Day (as defined below) preceding the interest reset date 
unless the index currency is pounds sterling, in which case the interest determination date will be the interest 
reset date; 
• for a EURIBOR note, the second TARGET Settlement Date (as defined below) preceding the interest reset 
date; 
• for a treasury rate note, the day of the week in which the interest reset date falls on which Treasury bills (as 
described below) of the applicable index maturity would normally be auctioned; and 
• for a floating-rate note with two or more base rates, the interest determination date will be the most recent 
business day that is at least two business days prior to the applicable interest reset date on which each 
applicable base rate is determinable. 
Treasury bills usually are sold at auction on Monday of each week, unless that day is a legal holiday, in which 
case the auction usually is held on the following Tuesday, except that the auction may be held on the preceding 
Friday. If, as a result of a legal holiday, an auction is held on the preceding Friday, that preceding Friday will be the 
interest determination date pertaining to the interest reset date occurring in the next succeeding week. The treasury 
rate will be determined as of that date, and the applicable interest rate will take effect on the applicable interest reset 
date. 
We will specify the interest reset dates in the applicable supplement. If any interest reset date for any floatingrate note falls on a day that is not a business day for the floating-rate note, the interest reset date for the floating-rate 
note will be postponed to the next day that is a business day for the floating-rate note. If Treasury bills are sold at an 
auction that falls on a day that is an interest reset date, that interest reset date will be the next following business 
day. However, unless we specify otherwise in the applicable supplement, in the case of a LIBOR note or a 
EURIBOR note, if the next business day is in the next succeeding calendar month, the interest reset date will be the 
immediately preceding business day. 
Calculation of Interest. Calculations relating to floating-rate notes will be made by the calculation agent, 
which will be an institution that we appoint as our agent for this purpose. The calculation agent may be one of our 
affiliates, including Bank of America, N.A., Merrill Lynch, Pierce, Fenner & Smith Incorporated, Merrill Lynch 
Commodities, Inc., or Merrill Lynch Capital Services, Inc. and may also be The Bank of New York Mellon Trust 
Company, N.A. We will identify in the applicable supplement the calculation agent we have appointed for a 
particular series of debt securities as of its original issue date. We may appoint different calculation agents from time 
to time after the original issue date of a floating-rate note without your consent and without notifying you of the 
change. Absent manifest error, all determinations of the calculation agent will be final and binding on you, the 
trustee and us. 
For each floating-rate note, the calculation agent will determine, on the corresponding calculation or interest 
determination date, the interest rate for the applicable interest period. In addition, the calculation agent will calculate 
the amount of interest that has accrued during each interest period. Unless we specify otherwise in the applicable 
supplement, the calculation date for 
17 
Table of Contents
any interest determination date will be the date by which the calculation agent computes the amount of interest owed 
on a floating-rate note for the related interest period. Unless we specify otherwise in the applicable supplement, the 
calculation date pertaining to an interest determination date will be the earlier of: 
• the tenth calendar day after that interest determination date or, if that day is not a business day, the next 
succeeding business day; or 
• the business day immediately preceding the applicable interest payment date, the maturity date, or the date 
of redemption or prepayment, as the case may be. 
Accrued interest on a floating-rate note is calculated by multiplying the principal amount of a note by an 
accrued interest factor. This accrued interest factor is the sum of the interest factors calculated for each day in the 
period for which accrued interest is being calculated. Unless we specify otherwise in the applicable supplement, the 
accrued interest factor will be computed and interest will be paid (including payments for partial periods) as follows: 
• for federal funds rate notes, LIBOR notes, EURIBOR notes, prime rate notes, or any other floating-rate 
notes other than treasury rate notes, the daily interest factor will be computed by dividing the interest rate in 
effect on that day by 360; and 
• for treasury rate notes, the daily interest factor will be computed by dividing the interest rate in effect on that 
day by 365 or 366, as applicable. 
All amounts used in or resulting from any calculation on floating-rate notes will be rounded to the nearest 
cent, in the case of U.S. dollars, or to the nearest corresponding hundredth of a unit, in the case of a currency other 
than U.S. dollars, with one-half cent or one-half of a corresponding hundredth of a unit or more being rounded 
upward. Unless we specify otherwise in the applicable supplement, all percentages resulting from any calculation 
with respect to a floating-rate note will be rounded, if necessary, to the nearest one hundred-thousandth of a percent, 
with five one-millionths of a percentage point rounded upwards, e.g., 9.876545% (or .09876545) being rounded to 
9.87655% (or .0987655). 
In determining the base rate that applies to a floating-rate note during a particular interest period, the 
calculation agent may obtain rate quotes from various banks or dealers active in the relevant market, as described in 
the descriptions of the base rates below and/or in the applicable supplement. Those reference banks and dealers may 
include the calculation agent itself and its affiliates, as well as any underwriter, dealer, or agent participating in the 
distribution of the relevant floating-rate notes and its affiliates, and they may include our affiliates. 
At the request of the holder of any floating-rate note, the calculation agent will provide the interest rate then in 
effect for that floating-rate note and, if already determined, the interest rate that is to take effect on the next interest 
reset date. 
LIBOR Notes. Each LIBOR note will bear interest at the LIBOR base rate, adjusted by any spread or spread 
multiplier, as specified in the applicable supplement. The LIBOR base rate will be the London interbank offered rate 
for deposits in U.S. dollars or any index currency, as specified in the applicable supplement. 
LIBOR for any interest determination date will be the arithmetic mean of the offered rates for deposits in the 
relevant index currency having the index maturity described in the applicable supplement, commencing on the 
related interest reset date, as the rates appear on the Reuters LIBOR screen page designated in the applicable 
supplement as of 11:00 A.M., London time, on that 
18 
Table of Contents
interest determination date, if at least two offered rates appear on the designated LIBOR page, except that, if the 
designated Reuters LIBOR screen page only provides for a single rate, that single rate will be used. 
If fewer than two of the rates described above appear on that page or no rate appears on any page on which 
only one rate normally appears, then the calculation agent will determine LIBOR as follows: 
• The calculation agent will select four major banks in the London interbank market, after consultation with 
us. On the interest determination date, those four banks will be requested to provide their offered quotations 
for deposits in the relevant index currency having an index maturity specified in the applicable supplement 
commencing on the interest reset date to prime banks in the London interbank market at approximately 
11:00 A.M., London time. 
• If at least two quotations are provided, the calculation agent will determine LIBOR as the arithmetic mean 
of those quotations. 
• If fewer than two quotations are provided, the calculation agent will select, after consultation with us, three 
major banks in New York City. On the interest determination date, those three banks will be requested to 
provide their offered quotations for loans in the relevant index currency having an index maturity specified 
in the applicable supplement commencing on the interest reset date to leading European banks at 
approximately 11:00 A.M., New York time. The calculation agent will determine LIBOR as the average of 
those quotations. 
• If fewer than three New York City banks selected by the calculation agent are quoting rates, LIBOR for that 
interest period will remain LIBOR then in effect on the interest determination date. 
EURIBOR Notes. Each EURIBOR note will bear interest at the EURIBOR base rate, adjusted by any spread 
or spread multiplier, as specified in the applicable supplement. 
EURIBOR, for any interest determination date, will mean the rate for deposits in euro as sponsored, 
calculated, and published jointly by the European Banking Federation and ACI—The Financial Market Association, 
or any company established by the joint sponsors for purposes of compiling and publishing those rates, having the 
index maturity specified in the applicable supplement, as that rate appears on the display on Reuters, or any 
successor service, on page EURIBOR01 or any other page as may replace such page, referred to as “Reuters Page 
EURIBOR01,” as of 11:00 A.M., Brussels time. 
The following procedures will be followed if EURIBOR cannot be determined as described above: 
• If no offered rate appears on Reuters Page EURIBOR01 on an interest determination date at approximately 
11:00 A.M., Brussels time, then the calculation agent, after consultation with us, will select four major 
banks in the Eurozone interbank market to provide a quotation of the rate at which deposits in euro having 
the index maturity specified in the applicable supplement are offered to prime banks in the Eurozone 
interbank market, and in a principal amount not less than the equivalent of €1,000,000, that is representative 
of a single transaction in euro in that market at that time. If at least two quotations are provided, EURIBOR 
will be the average of those quotations. 
• If fewer than two quotations are provided, then the calculation agent, after consultation with us, will select 
four major banks in the Eurozone interbank market to provide a quotation of the rate offered by them, at 
approximately 11:00 A.M., Brussels time, on the 
19 
Table of Contents
interest determination date, for loans in euro to prime banks in the Eurozone interbank market for a period 
of time equivalent to the index maturity specified in the applicable supplement commencing on that interest 
reset date and in a principal amount not less than the equivalent of €1,000,000, that is representative of a 
single transaction in euro in that market at that time. If at least three quotations are provided, EURIBOR will 
be the average of those quotations. 
• If three quotations are not provided, EURIBOR for that interest determination date will be equal to 
EURIBOR for the immediately preceding interest period. 
“Eurozone” means the region comprised of member states of the European Union that adopted the single 
currency in accordance with the Treaty establishing the European Community (signed in Rome on March 25, 1957), 
as amended by the Treaty on European Union (signed in Maastricht on February 7, 1992) and the Treaty of 
Amsterdam (signed in Amsterdam on October 2, 1997). 
Treasury Rate Notes. Each treasury rate note will bear interest at the treasury rate, adjusted by any spread or 
spread multiplier, as specified in the applicable supplement. 
The “treasury rate” for any interest determination date will be the rate set at the auction of direct obligations of 
the United States, referred to as “Treasury bills,” having the index maturity described in the applicable supplement, 
as specified under the caption “Investment Rate” on Reuters screen page USAUCTION 10 or page USAUCTION 
11, or any successor service or page. 
The following procedures will be followed if the treasury rate cannot be determined as described above: 
• If the rate is not displayed on the Reuters pages described above by 3:00 P.M., New York City time, on the 
related calculation date, the treasury rate will be the rate of Treasury bills as published in H.15 Daily 
Update, or another recognized electronic source used for the purpose of displaying the applicable rate, under 
the caption “U.S. Government Securities/Treasury Bills/Auction High.” 
• If the alternative rate described in the paragraph immediately above is not published by 3:00 P.M., New 
York City time, on the related calculation date, the treasury rate will be the bond equivalent yield, as defined 
below, of the auction rate of the applicable Treasury bills as announced by the U.S. Department of the 
Treasury. 
• If the alternative rate described in the paragraph immediately above is not announced by the U.S. 
Department of the Treasury, or if the auction is not held, the treasury rate will be the bond equivalent yield 
of the rate on the particular interest determination date of the applicable Treasury bills as published in 
H.15(519) under the caption “U.S. Government Securities/Treasury Bills/Secondary Market.” 
• If the alternative rate described in the paragraph immediately above is not published by 3:00 P.M., New 
York City time, on the related calculation date, the treasury rate will be the rate on the particular interest 
determination date of the applicable Treasury bills as published in H.15 Daily Update, or another recognized 
electronic source used for the purpose of displaying the applicable rate, under the caption “U.S. Government 
Securities/Treasury Bills/Secondary Market.” 
• If the alternative rate described in the paragraph immediately above is not published by 3:00 P.M., New 
York City time, on the related calculation date, the treasury rate will be the rate on the particular interest 
determination date calculated by the calculation agent as the 
20 
Table of Contents
bond equivalent yield of the arithmetic mean of the secondary market bid rates, as of approximately 3:30 
P.M., New York City time, on that interest determination date, of three primary U.S. government securities 
dealers, selected by the calculation agent, after consultation with us, for the issue of Treasury bills with a 
remaining maturity closest to the particular index maturity. 
• If the dealers selected by the calculation agent are not quoting as described in the paragraph immediately 
above, the treasury rate will be the treasury rate in effect on the particular interest determination date. 
The bond equivalent yield will be calculated using the following formula: 
Bond equivalent yield = D x N x 
360-(D x M) 100
where “D” refers to the applicable annual rate for Treasury bills quoted on a bank discount basis and expressed as a 
decimal, “N” refers to 365 or 366, as the case may be, and “M” refers to the actual number of days in the applicable 
interest period. 
“H.15(519)” means the weekly statistical release designated as H.15(519), or any successor publication, 
published by the Board of Governors of the Federal Reserve System. 
“H.15 Daily Update” means the daily update of H.15(519), available through the website of the Board of 
Governors of the Federal Reserve System at www.federalreserve.gov/releases/h15/update, or any successor site or 
publication. 
Federal Funds Rate Notes. Each federal funds rate note will bear interest at the federal funds rate, adjusted by 
any spread or spread multiplier, as specified in the applicable supplement. 
If “Federal Funds (Effective) Rate” is specified in the applicable supplement, the federal funds rate for any 
interest determination date will be the rate on that date for U.S. dollar federal funds, as published in H.15(519) under 
the heading “Federal Funds (Effective)” and displayed on Reuters, or any successor service, on page FEDFUNDS1 
or any other page as may replace the specified page on that service, referred to as “Reuters Page FedFunds1.” If this 
rate is not published in H.15 Daily Update by 3:00 P.M., New York City time, on the related calculation date, or 
does not appear on Reuters Page FedFunds1, the federal funds rate will be the rate on that interest determination 
date as published in H.15 Daily Update, or any other recognized electronic source for the purposes of displaying the 
applicable rate, under the caption “Federal Funds (Effective) Rate.” If this alternate rate is not published in H.15 
Daily Update, or other recognized electronic source for the purpose of displaying the applicable rate, by 3:00 P.M., 
New York City time, on the related calculation date, then the calculation agent will determine the federal funds rate 
to be the average of the rates for the last transaction in overnight U.S. dollar federal funds quoted prior to 9:00 A.M., 
New York City time, on the business day following that interest determination date, by each of three leading brokers 
of U.S. dollar federal funds transactions in New York City, selected by the calculation agent, after consultation with 
us. If fewer than three brokers selected by the calculation agent are so quoting, the federal funds rate will be the 
federal funds rate in effect on that interest determination date. 
If “Federal Funds Open Rate” is specified in the applicable supplement, the federal funds rate will be the rate 
on that interest determination date set forth under the heading “Federal Funds” opposite the caption “Open” and 
displayed on Reuters, or any successor service, on page 5 or any other page as may replace the specified page on 
that service, referred to as “Reuters Page 5,” or if that rate does not appear on Reuters Page 5 by 3:00 P.M., New 
York City time, on the related calculation date, the federal funds rate will be the rate on that date displayed on 
FFPREBON Index 
21 
Table of Contents
page on Bloomberg L.P. (“Bloomberg”), which is the Fed Funds Opening Rate as reported by Prebon Yamane (or a 
successor) on Bloomberg. If the alternate rate described in the preceding sentence is not displayed on FFPREBON 
Index page on Bloomberg, or any other recognized electronic source for the purpose of displaying the applicable 
rate, by 3:00 P.M., New York City time, on the related calculation date, then the calculation agent will determine the 
federal funds rate to be the average of the rates for the last transaction in overnight U.S. dollar federal funds, quoted 
prior to 9:00 A.M., New York City time, on that interest determination date, by each of three leading brokers of U.S. 
dollar federal funds transactions in New York City, selected by the calculation agent, after consultation with us. If 
fewer than three brokers selected by the calculation agent are quoting as described above, the federal funds rate will 
be the federal funds rate in effect on that interest determination date. 
If “Federal Funds Target Rate” is specified in the applicable supplement, the federal funds rate will be the rate 
on that interest determination date for U.S. dollar federal funds displayed on the FDTR Index page on Bloomberg. If 
that rate does not appear on the FDTR Index page on Bloomberg by 3:00 P.M., New York City time, on the 
calculation date, the federal funds rate for the applicable interest determination date will be the rate for that day 
appearing on Reuters, or any successor service, on page USFFTARGET= or any other page as may replace the 
specified page on that service, referred to as “Reuters Page USFFTARGET=.” If that rate does not appear on the 
FDTR Index page on Bloomberg or is not displayed on Reuters Page USFFTARGET= by 3:00 P.M., New York 
City time, on the applicable date, then the calculation agent will determine the federal funds rate to be the average of 
the rates for the last transaction in overnight U.S. dollar federal funds, quoted prior to 9:00 A.M., New York City 
time, on that interest determination date, by each of three leading brokers of U.S. dollar federal funds transactions in 
New York City, selected by the calculation agent, after consultation with us. If fewer than three brokers selected by 
the calculation agent are quoting as described above, the federal funds rate will be the federal funds rate in effect on 
that interest determination date. 
Prime Rate Notes. Each prime rate note will bear interest at the prime rate, as adjusted by any spread or spread 
multiplier, as specified in the applicable supplement. 
The “prime rate” for any interest determination date will be the prime rate or base lending rate on that date, as 
published in H.15(519) prior to 3:00 P.M., New York City time, on the related calculation date, under the heading 
“Bank Prime Loan.” 
The following procedures will be followed if the prime rate cannot be determined as described above: 
• If the rate is not published in H.15(519) by 3:00 P.M., New York City time, on the related calculation date, 
then the prime rate will be the rate as published in H.15 Daily Update, or any other recognized electronic 
source used for the purpose of displaying the applicable rate, under the caption “Bank Prime Loan.” 
• If the alternative rate described above is not published in H.15 Daily Update or another recognized 
electronic source by 3:00 P.M., New York City time, on the related calculation date, then the calculation 
agent will determine the prime rate to be the arithmetic mean of the rates of interest publicly announced by 
each bank that appears on the Reuters screen US PRIME 1, as defined below, as that bank’s prime rate or 
base lending rate as in effect as of 11:00 A.M., New York City time, on that interest determination date. 
• If fewer than four rates appear on the Reuters screen US PRIME 1 for that interest determination date, by 
3:00 P.M., New York City time, then the calculation agent will determine the prime rate to be the average of 
the prime rates or base lending rates furnished in New York City by three substitute banks or trust 
companies (all organized
22 
Table of Contents
under the laws of the United States or any of its states and having total equity capital of at least 
$500,000,000) selected by the calculation agent, after consultation with us. 
• If the banks selected by the calculation agent are not quoting as described above, the prime rate will remain 
the prime rate then in effect on the interest determination date. 
“Reuters screen US PRIME 1” means the display designated as page “US PRIME 1” on the Reuters Monitor 
Money Rates Service (or any other page as may replace the US PRIME 1 page on that service for the purpose of 
displaying prime rates or base lending rates of major U.S. banks). 
Indexed Notes 
We may issue debt securities that provide that the rate of return, including the principal, premium (if any), 
interest, or other amounts payable (if any), is determined by reference, either directly or indirectly, to the price or 
performance of one or more securities, currencies or composite currencies, commodities, interest rates, stock 
indices, commodity indices or other indices, formulae, or measure, in each case as specified in the applicable 
supplement. We refer to these as “indexed notes.” 
Holders of indexed notes may receive an amount at maturity that is greater than or less than the face amount 
of the notes, depending upon the formula used to determine the amount payable and the relative value at maturity of 
the reference asset or underlying obligation. The value of the applicable index will fluctuate over time. 
An indexed note may provide either for cash settlement or for physical settlement by delivery of the indexed 
note or securities, or other securities of the types listed above. An indexed note also may provide that the form of 
settlement may be determined at our option or the holder’s option. Some indexed notes may be convertible, 
exercisable, or exchangeable prior to maturity, at our option or the holder’s option, for the related securities. 
We will specify in the applicable supplement the method for determining the principal, premium (if any), 
interest, or other amounts payable (if any) in respect of particular indexed notes, as well as certain historical 
information with respect to the specified index or indexed items, specific risk factors relating to that particular type 
of indexed note, and tax considerations associated with an investment in the indexed notes. 
The applicable supplement for any particular indexed notes also will identify the calculation agent that will 
calculate the amounts payable with respect to the indexed note. The calculation agent may be one of our affiliates, 
including Bank of America, N.A., Merrill Lynch, Pierce, Fenner & Smith Incorporated, Merrill Lynch 
Commodities, Inc., or Merrill Lynch Capital Services, Inc. We may appoint different calculation agents from time to 
time after the original issue date of an indexed note without your consent and without notifying you of the change. 
Absent manifest error, all determinations of the calculation agent will be final and binding on you, the trustee and 
us. Upon request of the holder of an indexed note, the calculation agent will provide, if applicable, information 
relating to the current principal, premium (if any), rate of interest, interest payable, or other amounts payable (if any) 
in connection with the indexed note. 
We also may offer “indexed amortizing notes,” the rate of amortization and final maturity of which are subject 
to periodic adjustment based upon the degree to which an objective base or index rate such as LIBOR, called a 
“reference rate,” coincides with a specified “target rate.” Indexed amortizing notes may provide for adjustment of 
the amortization rate either on every interest payment date, or only on interest payment dates that occur after a 
specified “lockout date.” Each 
23 
Table of Contents
indexed amortizing note will include an amortization table, specifying the rate at which the principal of the note is to 
be amortized following any applicable interest payment date, based upon the difference between the reference rate 
and the target rate. The specific terms of, and any additional considerations relating to, indexed amortizing notes 
will be set forth in the applicable supplement. 
Floating-Rate/Fixed-Rate/Indexed Notes 
We may issue a debt security with elements of each of the fixed-rate, floating-rate and indexed notes 
described above. For example, a debt security may bear interest at a fixed rate for some periods and at a floating rate 
in others. Similarly, a debt security may provide for a payment of principal at maturity linked to an index and also 
may bear interest at a fixed or floating rate. We will describe the determination of interest for any of these debt 
securities in the applicable supplement. 
Original Issue Discount Notes 
A fixed-rate note, a floating-rate note, or an indexed note may be an original issue discount note. Original 
issue discount notes are debt securities that are issued at a price lower than their stated principal amount or lower 
than their minimum guaranteed repayment amount at maturity. Original issue discount notes may bear no interest 
(“zero coupon rate notes”) or may bear interest at a rate that is below market rates at the time of issuance. Upon an 
acceleration of the maturity of an original issue discount note, the amount of interest payable will be determined in 
accordance with the terms of the note, as described in the applicable supplement. That amount normally is less than 
the amount payable at the maturity date. A note issued at a discount to its principal may, for U.S. federal income tax 
purposes, be considered an original issue discount note, regardless of the amount payable upon redemption or 
acceleration of maturity. See “U.S. Federal Income Tax Considerations—Taxation of Debt Securities” below for a 
summary of the U.S. federal income tax consequences of owning an original issue discount note. 
Payment of Principal, Interest, and Other Amounts Due 
Paying Agents. We may appoint one or more financial institutions to act as our paying agents. Unless we 
specify otherwise in the applicable supplement, the trustee will act as our sole paying agent, security registrar and 
transfer agent with respect to the debt securities through the trustee’s office. That office is currently located at 101 
Barclay Street, New York, New York 10286. In addition, in the case of some of our debt securities, such as debt 
securities denominated in euro, that office is expected to be 48th Floor, One Canada Square, London, E14 5AL. At 
any time, we may rescind the designation of a paying agent, appoint a successor paying agent, or approve a change 
in the office through which any successor paying agent acts in accordance with the applicable indenture. In addition, 
we may decide to act as our own paying agent with respect to some or all of the debt securities, and the paying agent 
may resign. 
Payments to Holders and Record Dates for Interest. We refer to each date on which interest is payable on a 
debt security as an “interest payment date.” Unless we specify otherwise in the applicable supplement, the 
provisions described in this section will apply to payments on the debt securities. 
Interest payments on the debt securities will be made on each interest payment date applicable to, and at the 
maturity date of, the debt securities. Interest payable at any interest payment date other than the maturity date will be 
paid to the registered holder of the debt security on the regular record date for that interest payment date, as
described below. However, unless we specify otherwise in the applicable supplement, the initial interest payment on 
a debt security issued 
24 
Table of Contents
between a regular record date and the interest payment date immediately following the regular record date will be 
made on the second interest payment date following the original issue date to the holder of record on the regular 
record date preceding the second interest payment date. The principal and interest payable at maturity will be paid to 
the holder of the debt security at the time of payment by the paying agent. 
Unless we specify otherwise in the applicable supplement, the record date for any interest payment for a debt 
security in book-entry only form generally will be the business day prior to the payment date. If the debt security is 
in a form that is other than book-entry only, and unless we specify otherwise in the applicable supplement, the 
regular record date for an interest payment date will be the last day of the calendar month preceding the interest 
payment date or the fifteenth day of the calendar month in which the interest payment date occurs, as specified in the 
supplement, whether or not that date is a business day. 
Unless we specify otherwise in the applicable supplement, if any interest payment date or the maturity date of 
a debt security falls on a day that is not a business day, we will make the required payment on the next business day, 
and no additional interest will accrue in respect of the payment made on the next business day. However, unless we 
specify otherwise in the applicable supplement, for LIBOR notes or EURIBOR notes, if an interest payment date 
falls on a date that is not a business day, and the next business day is in the next calendar month, the interest 
payment date will be the immediately preceding business day. 
Unless we specify otherwise in the applicable supplement, the term “business day” means, for any debt 
security, a day that meets all the following applicable requirements: 
• for all debt securities, is any weekday that is not a legal holiday in New York, New York, Charlotte, North 
Carolina, or any other place of payment of the debt security, and is not a date on which banking institutions 
in those cities are authorized or required by law or regulation to be closed; 
• for any LIBOR note, also is a day on which commercial banks are open for business (including dealings in 
the index currency specified in the applicable supplement) in London, England (a “London Banking Day”); 
• for any debt security denominated in euro or any EURIBOR note, also is a day on which the TransEuropean 
Automated Real-Time Gross Settlement Express Transfer, or “TARGET,” System or any successor is 
operating (a “TARGET Settlement Date”); and 
• for any debt security that has a specified currency other than U.S. dollars or euro, also is not a day on which 
banking institutions generally are authorized or obligated by law, regulation, or executive order to close in 
the principal financial center of the country of the specified currency. 
Unless we specify otherwise in the applicable supplement, for purposes of this determination, the “principal 
financial center” is: 
• the capital city of the country issuing the specified currency, except for U.S. dollars, Australian dollars, 
Canadian dollars, South African rand and Swiss francs, for which the “principal financial center” is New 
York, Sydney and Melbourne, Toronto, Johannesburg and Zurich, respectively; or 
• the capital city of the country to which the index currency relates, except for U.S. dollars, Australian dollars, 
Canadian dollars, South African rand and Swiss francs, for which the “principal financial center” is New 
York, Sydney, Toronto, Johannesburg and Zurich, respectively. 
25 
Table of Contents
Payments Due in U.S. Dollars. Unless we specify otherwise in the applicable supplement, we will follow the 
practices described in this subsection when we pay amounts that are due in U.S. dollars. 
We will make payments on debt securities in book-entry form in accordance with arrangements then in place 
between the paying agent and the depository or its nominee, as holder. An indirect owner’s right to receive those 
payments will be governed by the rules and practices of the depository and its participants, as described below under 
the heading “Registration and Settlement.” 
We will pay any interest on debt securities in certificated form on each interest payment date other than the 
maturity date by, in our discretion, wire transfer of immediately available funds or check mailed to holders of the 
debt securities on the applicable record date at the address appearing on our records. We will pay any principal, 
premium (if any), interest, and other amounts payable (if any) at the maturity date of a debt security in certificated 
form by wire transfer of immediately available funds upon surrender of the debt security at the corporate trust office 
of the applicable trustee or paying agent. 
Book-entry and other indirect owners should contact their banks or brokers for information on how they will 
receive payments on their debt securities. 
Payments Due in Other Currencies. Unless we specify otherwise in the applicable supplement, we will follow 
the practices described in this subsection when we pay amounts that are due in a currency other than U.S. dollars. 
Unless we specify otherwise in the applicable supplement, holders are not entitled to receive payments in U.S. 
dollars of an amount due in another currency, either on a global debt security or a debt security in certificated form. 
We will make payments on Non-U.S. Dollar Denominated Debt Securities in book-entry form in the 
applicable specified currency in accordance with arrangements then in place between the paying agent and the 
depository or its nominee, as holder. An indirect owner’s right to receive those payments will be governed by the 
rules and practices of the depository and its participants, as described below under the heading “Registration and 
Settlement.” 
We will pay any interest on Non-U.S. Dollar-Denominated Debt Securities in certificated form by check 
mailed to holders of the debt securities on the applicable record date at the address appearing on our records. We 
will pay any principal, premium (if any), interest and other amounts payable (if any) at the maturity date of a NonU.S. Dollar-Denominated Debt Security in certificated form by wire transfer of immediately available funds upon 
surrender of the debt security at the corporate trust office of the applicable trustee or paying agent. 
If we issue a debt security in a specified currency other than U.S. dollars, unless we specify otherwise in the 
applicable supplement, we will appoint a financial institution to act as the exchange rate agent. The exchange rate 
agent will determine the applicable rate of exchange that would apply to a payment made in U.S. dollars, if the 
currency in which we otherwise would be required to make the applicable payment is not available. The exchange 
rate agent may be one of our affiliates. We will identify in the applicable supplement the exchange rate agent that we 
have appointed for a particular debt security as of its original issue date. We may appoint different exchange rate 
agents from time to time after the original issue date of the debt security without your consent and without notifying 
you of the change. All determinations made by the exchange rate agent will be in its sole discretion unless we state 
in the applicable supplement that any determination requires our approval. Absent manifest error, those 
determinations will be final and binding on you and us. 
26 
Table of Contents
Book-entry and other indirect owners of a debt security with a specified currency other than U.S. dollars 
should contact their banks or brokers for information about how to receive payments in the specified currency or in 
U.S. dollars. 
No Sinking Fund 
Unless we specify otherwise in the applicable supplement, our debt securities will not be entitled to the benefit 
of any sinking fund. This means that we will not deposit money on a regular basis into any separate custodial 
account to repay the debt securities. 
Redemption 
The applicable supplement will indicate whether we may redeem the debt securities prior to their maturity 
date. If we may redeem the debt securities prior to maturity, the applicable supplement will indicate the redemption 
price, the method for redemption and the date or dates upon which we may redeem the debt securities. Unless we 
specify otherwise in the applicable supplement, we may redeem debt securities only on an interest payment date, and 
the redemption price will be 100% of the principal amount of the debt securities to be redeemed, plus any accrued 
and unpaid interest. 
Unless we specify otherwise in the applicable supplement, we may exercise our right to redeem debt securities 
by giving notice to the trustee under the applicable indenture at least 10 business days but not more than 60 calendar 
days before the specified redemption date. The notice will take the form of a certificate signed by us specifying: 
• the date fixed for redemption; 
• the redemption price; 
• the CUSIP number of the debt securities to be redeemed; 
• the amount to be redeemed, if less than all of a series of debt securities is to be redeemed; 
• the place of payment for the debt securities to be redeemed; and 
• that on and after the date fixed for redemption, interest will cease to accrue on the debt securities to be 
redeemed. 
So long as a depository is the record holder of the applicable debt securities to be redeemed, we will deliver 
any notice of our election to exercise our redemption right only to that depository. 
Repayment 
The applicable supplement will indicate whether the debt securities can be repaid at the holder’s option prior 
to their maturity date. If the debt securities may be repaid prior to maturity, the applicable supplement will indicate 
the applicable repayment price or prices, the procedures for repayment and the date or dates on or after which the 
holder can request repayment. 
Repurchase 
We may purchase at any time and from time to time, including through a subsidiary or affiliate of ours, 
outstanding debt securities by tender, in the open market, or by private agreement. We, or our affiliates, have the 
discretion to hold or resell any repurchased debt securities. We also have the discretion to cancel any repurchased 
debt securities. 
27 
Table of Contents
Conversion 
We may issue debt securities that are convertible into, or exercisable or exchangeable for, at either our option 
or the holder’s option, our preferred stock, depositary shares, common stock, or other debt securities, or debt or 
equity securities of one or more third parties. The applicable supplement will describe the terms of any conversion, 
exercise, or exchange features, including: 
• the periods during which conversion, exercise, or exchange, as applicable, may be elected; 
• the conversion, exercise, or exchange price payable and the number of shares or amount of our preferred 
stock, depositary shares, common stock, or other debt securities, or debt or equity securities of a third party, 
that may be issued upon conversion, exercise, or exchange, and any adjustment provisions; and 
• the procedures for electing conversion, exercise, or exchange, as applicable. 
Exchange, Registration, and Transfer 
Subject to the terms of the applicable indenture, debt securities of any series in certificated form may be 
exchanged at the option of the holder for other debt securities of the same series and of an equal aggregate principal 
amount and type in any authorized denominations. 
Debt securities in certificated form may be presented for registration of transfer at the office of the security 
registrar or at the office of any transfer agent that we designate and maintain. The security registrar or the transfer 
agent will make the transfer or registration only if it is satisfied with the documents of title and identity of the person 
making the request. There will not be a service charge for any exchange or registration of transfer of debt securities, 
but we may require payment of a sum sufficient to cover any tax or other governmental charge that may be imposed 
in connection with the exchange. Unless we specify otherwise in the applicable supplement, The Bank of New York 
Mellon Trust Company, N.A. will be the authenticating agent, registrar, and transfer agent for the debt securities 
issued under the respective indentures. We may change the security registrar or the transfer agent or approve a 
change in the location through which any security registrar or transfer agent acts at any time, except that we will be 
required to maintain a security registrar and transfer agent in each place of payment for each series of debt 
securities. At any time, we may designate additional transfer agents for any series of debt securities. 
We will not be required to (1) issue, exchange, or register the transfer of any debt security of any series to be 
redeemed for a period of 15 days before those debt securities were selected for redemption, or (2) exchange or 
register the transfer of any debt security that was selected, called, or is being called for redemption, except the 
unredeemed portion of any debt security being redeemed in part. 
For a discussion of restrictions on the exchange, registration, and transfer of book-entry securities, see 
“Registration and Settlement” below. 
Subordination 
Our subordinated debt securities are subordinated in right of payment to all of our “senior indebtedness.” The 
Subordinated Indenture defines “senior indebtedness” as any indebtedness for money borrowed, including all of our 
indebtedness for borrowed and purchased money, all of our obligations arising from off-balance sheet guarantees 
and direct credit substitutes, and our obligations associated with derivative products such as interest and foreign 
exchange rate contracts and commodity contracts, that was outstanding on the date we executed the Subordinated 
28 
Table of Contents
Indenture, or was created, incurred, or assumed after that date, for which we are responsible or liable as obligor, 
guarantor, or otherwise, and all deferrals, renewals, extensions, and refundings of that indebtedness or obligations, 
other than the debt securities issued under the Subordinated Indenture or any other indebtedness that by its terms is 
subordinate in right of payment to any of our other indebtedness. Each supplement for a series of subordinated debt 
securities will indicate the aggregate amount of our senior indebtedness outstanding at that time and any limitation 
on the issuance of additional senior indebtedness. 
If there is a default or event of default under any senior indebtedness that would allow acceleration of maturity 
of the senior indebtedness and that default or event of default is not remedied, and we and the trustee of the 
Subordinated Indenture receive notice of this default from the holders of at least 10% in principal amount of any 
kind or category of any senior indebtedness or if the trustee of the Subordinated Indenture receives notice from us, 
then we will not be able to make any principal, premium, interest, or other payments on the subordinated debt 
securities or repurchase our subordinated debt securities. 
If any subordinated debt security is declared due and payable before the required date or upon a payment or 
distribution of our assets to creditors pursuant to a dissolution, winding up, liquidation, or reorganization, we are 
required to pay all principal, premium, interest, or other payments to holders of senior indebtedness before any 
holders of subordinated debt are paid. In addition, if any amounts previously were paid to the holders of 
subordinated debt or the trustee of the Subordinated Indenture, the holders of senior indebtedness will have first 
rights to the amounts previously paid. 
Subject to the payment in full of all our senior indebtedness, the holders of our subordinated debt securities 
will be subrogated to the rights of the holders of our senior indebtedness to receive payments or distributions of our 
assets applicable to the senior indebtedness until our subordinated debt securities are paid in full. For purposes of 
this subrogation, the subordinated debt securities will be subrogated equally and ratably with all our other 
indebtedness that by its terms ranks equally with our subordinated debt securities and is entitled to like rights of 
subrogation. 
Sale or Issuance of Capital Stock of Banks 
The Senior Indenture prohibits the issuance, sale, or other disposition of capital stock, or securities convertible 
into or options, warrants, or rights to acquire capital stock, of any Principal Subsidiary Bank (as defined below) or of 
any subsidiary which owns shares of capital stock, or securities convertible into or options, warrants, or rights to 
acquire capital stock, of any Principal Subsidiary Bank, with the following exceptions: 
• sales of directors’ qualifying shares; 
• sales or other dispositions for fair market value, if, after giving effect to the disposition and to conversion of 
any shares or securities convertible into capital stock of a Principal Subsidiary Bank, we would own at least 
80% of each class of the capital stock of that Principal Subsidiary Bank; 
• sales or other dispositions made in compliance with an order of a court or regulatory authority of competent 
jurisdiction; 
• any sale by a Principal Subsidiary Bank of additional shares of its capital stock, securities convertible into 
shares of its capital stock, or options, warrants, or rights to subscribe for or purchase shares of its capital 
stock, to its stockholders at any price, so long as before that sale we owned, directly or indirectly, securities 
of the same class and immediately after the 
29 
Table of Contents
sale, we owned, directly or indirectly, at least as great a percentage of each class of securities of the 
Principal Subsidiary Bank as we owned before the sale of additional securities; and 
• any issuance of shares of capital stock, or securities convertible into or options, warrants, or rights to 
subscribe for or purchase shares of capital stock, of a Principal Subsidiary Bank or any subsidiary which 
owns shares of capital stock, or securities convertible into or options, warrants, or rights to acquire capital 
stock, of any Principal Subsidiary Bank, to us or our wholly owned subsidiary. 
A “Principal Subsidiary Bank” is defined in the Senior Indenture as any bank with total assets equal to more 
than 10% of our total consolidated assets. As of the date of this prospectus, Bank of America, N.A. is our only 
Principal Subsidiary Bank. 
Limitation on Mergers and Sales of Assets 
Each indenture generally permits a consolidation or merger between us and another entity. It also permits the 
sale or transfer by us of all or substantially all of our assets. These transactions are permitted if: 
• the resulting or acquiring entity, if other than us, is organized and existing under the laws of the United 
States or any state or the District of Columbia and expressly assumes all of our obligations under that 
indenture; and 
• immediately after the transaction, we (or any successor company) are not in default in the performance of 
any covenant or condition under that indenture. 
Upon any consolidation, merger, sale, or transfer of this kind, the resulting or acquiring entity will be 
substituted for us in the applicable indenture with the same effect as if it had been an original party to that indenture. 
As a result, the successor entity may exercise our rights and powers under the indenture. 
Waiver of Covenants 
The holders of a majority in principal amount of the debt securities of all affected series then outstanding 
under the indenture may waive compliance with some of the covenants or conditions of that indenture. 
Modification of the Indentures 
We and the trustee may modify the applicable indenture and the rights of the holders of the debt securities 
with the consent of the holders of at least 66 2/3% of the aggregate principal amount of all series of debt securities 
under that indenture affected by the modification. However, no modification may extend the fixed maturity of, 
reduce the principal amount or redemption premium of, or reduce the rate of, or extend the time of payment of, 
interest on, any debt security without the consent of each holder affected by the modification. No modification may 
reduce the percentage of debt securities that is required to consent to modification of an indenture without the 
consent of all holders of the debt securities outstanding under that indenture. 
In addition, we and the trustee may execute supplemental indentures in some circumstances without the 
consent of any holders of outstanding debt securities. 
For purposes of determining the aggregate principal amount of the debt securities outstanding at any time in 
connection with any request, demand, authorization, direction, notice, consent, or 
30 
Table of Contents
waiver under the applicable indenture, (1) the principal amount of any debt security issued with original issue 
discount is that amount that would be due and payable at that time upon an event of default, and (2) the principal 
amount of a debt security denominated in a foreign currency or currency unit is the U.S. dollar equivalent on the 
date of original issuance of the debt security. 
Meetings and Action by Securityholders 
The trustee may call a meeting in its discretion, or upon request by us or the holders of at least 10% in 
principal amount of a series of outstanding debt securities, by giving notice. If a meeting of holders is duly held, any 
resolution raised or decision taken in accordance with the indenture will be binding on all holders of debt securities 
of that series. 
Events of Default and Rights of Acceleration 
The Senior Indenture defines an event of default for a series of senior debt securities as any one of the 
following events: 
• our failure to pay principal or any premium when due on any securities of that series; 
• our failure to pay interest on any securities of that series, within 30 calendar days after the interest becomes 
due; 
• our breach of any of our other covenants contained in the senior debt securities of that series or in the Senior 
Indenture, that is not cured within 90 calendar days after written notice to us by the trustee of the Senior 
Indenture, or to us and the trustee of the Senior Indenture by the holders of at least 25% in principal amount 
of all senior debt securities then outstanding under the Senior Indenture and affected by the breach; and 
• specified events involving our bankruptcy, insolvency, or liquidation. 
The Subordinated Indenture defines an event of default only as our bankruptcy under U.S. federal bankruptcy 
laws. 
If an event of default occurs and is continuing, either the trustee or the holders of 25% in principal amount of 
the debt securities outstanding under the applicable indenture (or, in the case of an event of default under the Senior 
Indenture with respect to a series of senior debt securities, the holders of 25% in principal amount of the outstanding 
debt securities of all series affected) may declare the principal amount, or, if the debt securities are issued with 
original issue discount, a specified portion of the principal amount, of all debt securities (or the debt securities of all 
series affected, as the case may be) to be due and payable immediately. The holders of a majority in principal 
amount of the debt securities then outstanding (or of the series affected, as the case may be), in some circumstances, 
may annul the declaration of acceleration and waive past defaults. 
Payment of principal of the subordinated debt securities may not be accelerated in the case of a default in the 
payment of principal, any premium, interest, or any other amounts or the performance of any of our other covenants. 
Collection of Indebtedness 
If we fail to pay the principal of (or, under the Senior Indenture, any premium on) any debt securities, or if we 
are over 30 calendar days late on an interest payment on the debt securities, the applicable trustee can demand that 
we pay to it, for the benefit of the holders of those debt securities, the amount which is due and payable on those 
debt securities, including any interest incurred because of our failure to make that payment. If we fail to pay the 
required amount on demand, the trustee may take appropriate action, including instituting judicial proceedings 
against us. 
31 
Table of Contents
In addition, a holder of a debt security also may file suit to enforce our obligation to make payment of 
principal, any premium, interest, or other amounts due on that debt security regardless of the actions taken by the 
trustee. 
The holders of a majority in principal amount of each series of the debt securities then outstanding under an 
indenture may direct the time, method, and place of conducting any proceeding for any remedy available to the 
trustee under that indenture, but the trustee will be entitled to receive from the holders a reasonable indemnity 
against expenses and liabilities. 
We are required periodically to file with the trustees a certificate stating that we are not in default under any of 
the terms of the indentures. 
Payment of Additional Amounts 
If we so specify in the applicable supplement, and subject to the exceptions and limitations set forth below, we 
will pay to the beneficial owner of any debt security that is a “non-U.S. person” additional amounts to ensure that 
every net payment on that debt security will not be less, due to the payment of U.S. withholding tax, than the amount 
then otherwise due and payable. For this purpose, a “net payment” on a debt security means a payment by us or any 
paying agent, including payment of principal and interest, after deduction for any present or future tax, assessment, 
or other governmental charge of the United States (other than a territory or possession). These additional amounts 
will constitute additional interest on the debt security. For this purpose, U.S. withholding tax means a withholding 
tax of the United States, other than a territory or possession. 
However, notwithstanding our obligation, if so specified, to pay additional amounts, we will not be required to 
pay additional amounts in any of the circumstances described in items (1) through (15) below, unless we specify 
otherwise in the applicable supplement. 
(1) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld solely by reason of the beneficial 
owner of the debt security: 
• having a relationship with the United States as a citizen, resident, or otherwise; 
• having had such a relationship in the past; or 
• being considered as having had such a relationship. 
(2) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld solely by reason of the beneficial 
owner of the debt security: 
• being treated as present in or engaged in a trade or business in the United States; 
• being treated as having been present in or engaged in a trade or business in the United States in the 
past; 
• having or having had a permanent establishment in the United States; or 
• having or having had a qualified business unit which has the U.S. dollar as its functional currency. 
32 
Table of Contents
(3) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld solely by reason of the beneficial 
owner of the debt security being or having been a: 
• personal holding company; 
• foreign personal holding company; 
• private foundation or other tax-exempt organization; 
• passive foreign investment company; 
• controlled foreign corporation; or 
• corporation which has accumulated earnings to avoid U.S. federal income tax. 
(4) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld solely by reason of the beneficial 
owner of the debt security owning or having owned, actually or constructively, 10% or more of the total 
combined voting power of all classes of our stock entitled to vote. 
(5) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld solely by reason of the beneficial 
owner of the debt security being a bank extending credit under a loan agreement entered into in the 
ordinary course of business. 
For purposes of items (1) through (5) above, “beneficial owner” includes, without limitation, a holder 
and a fiduciary, settlor, partner, member, shareholder, or beneficiary of the holder if the holder is an estate, 
trust, partnership, limited liability company, corporation, or other entity, or a person holding a power over an 
estate or trust administered by a fiduciary holder. 
(6) Additional amounts will not be payable to any beneficial owner of a debt security that is: 
• A fiduciary; 
• A partnership; 
• A limited liability company;
• Another fiscally transparent entity; or 
• Not the sole beneficial owner of the debt security, or any portion of the debt security. 
However, this exception to the obligation to pay additional amounts will apply only to the extent that a 
beneficiary or settlor in relation to the fiduciary, or a beneficial owner, partner, or member of the partnership, 
limited liability company, or other fiscally transparent entity, would not have been entitled to the payment of 
an additional amount had the beneficiary, settlor, beneficial owner, partner, or member received directly its 
beneficial or distributive share of the payment. 
(7) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld 
33 
Table of Contents
solely by reason of the failure of the beneficial owner of the debt security or any other person to comply 
with applicable certification, identification, documentation, or other information reporting requirements. 
This exception to the obligation to pay additional amounts will apply only if compliance with such 
requirements is required as a precondition to exemption from such tax, assessment, or other governmental 
charge by statute or regulation of the United States or by an applicable income tax treaty to which the 
United States is a party. 
(8) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is collected or imposed by any method other than by 
withholding from a payment on a debt security by us or any paying agent. 
(9) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld by reason of a change in law, 
regulation, or administrative or judicial interpretation that becomes effective more than 15 days after the 
payment becomes due or is duly provided for, whichever occurs later. 
(10) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld by reason of the presentation by the 
beneficial owner of a debt security for payment more than 30 days after the date on which such payment 
becomes due or is duly provided for, whichever occurs later. 
(11) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any: 
• estate tax; 
• inheritance tax; 
• gift tax; 
• sales tax; 
• excise tax; 
• transfer tax; 
• wealth tax; 
• personal property tax; or 
• any similar tax, assessment, or other governmental charge. 
(12) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge required to be withheld by any paying agent from a payment of 
principal or interest on the applicable security if such payment can be made without such withholding by 
any other paying agent. 
(13) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld 
34 
Table of Contents
by reason of the application of Section 1471 (or any successor provision) or Section 1472 (or any 
successor provision) of the U.S. Internal Revenue Code of 1986, as amended, or any related 
administrative regulation or pronouncement. 
(14) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any tax, 
assessment, or other governmental charge that is imposed or withheld by reason of the payment being 
treated as a dividend or dividend equivalent for U.S. tax purposes. 
(15) Additional amounts will not be payable if a payment on a debt security is reduced as a result of any 
combination of items (1) through (14) above. 
Except as specifically provided in this section, we will not be required to make any payment of any tax, 
assessment, or other governmental charge imposed by any government, political subdivision, or taxing authority of 
that government. 
For purposes of determining whether the payment of additional amounts is required, the term “U.S. person” 
means any individual who is a citizen or resident of the United States; any corporation, partnership, or other entity 
created or organized in or under the laws of the United States; any estate if the income of such estate falls within the 
federal income tax jurisdiction of the United States regardless of the source of that income; and any trust if a U.S. 
court is able to exercise primary supervision over its administration and one or more U.S. persons have the authority 
to control all of the substantial decisions of the trust. Additionally, for this purpose, “non-U.S. person” means a 
person who is not a U.S. person, and “United States” means the United States of America, including each state of 
the United States and the District of Columbia, its territories, its possessions, and other areas within its jurisdiction. 
Redemption for Tax Reasons 
If we so specify in the applicable supplement, we may redeem the debt securities in whole, but not in part, at 
any time before maturity, after giving not less than 30 nor more than 60 calendar days’ notice to the trustee under 
the applicable indenture and to the holders of the debt securities, if we have or will become obligated to pay 
additional amounts, as described above under “—Payment of Additional Amounts,” as a result of any change in, or 
amendment to, the laws or regulations of the United States or any political subdivision or any authority of the United 
States having power to tax, or any change in the application or official interpretation of such laws or regulations, 
which change or amendment becomes effective on or after the date of the applicable supplement for the issuance of 
those debt securities. 
Before we publish any notice of redemption for tax reasons, we will deliver to the trustee under the indenture 
a certificate signed by our chief financial officer or a senior vice president stating that we are entitled to redeem the 
debt securities and that the conditions precedent to redemption have occurred. 
Unless we specify otherwise in the applicable supplement, any debt securities redeemed for tax reasons will be 
redeemed at 100% of their principal amount together with interest accrued up to, but excluding, the redemption date. 
Defeasance and Covenant Defeasance 
If we so specify in the applicable supplement, the provisions for full defeasance and covenant defeasance 
described below will apply to the debt securities if certain conditions are satisfied. 
35 
Table of Contents
Full Defeasance. If there is a change in the U.S. federal tax law, as described below, we can legally release 
ourselves from all payment and other obligations on any debt securities. This is called full defeasance. For us to do 
so, each of the following must occur: 
• We must deposit in trust for the benefit of the holders of those debt securities a combination of money and 
U.S. government or U.S. government agency notes or bonds that will generate enough cash to make interest, 
principal, and any other payments on those debt securities at their due dates; 
• There must be a change in current U.S. federal tax law or an Internal Revenue Service ruling that lets us 
make the above deposit without causing the holders to be taxed on the debt securities any differently than if 
we did not make the deposit and repaid the debt securities ourselves. Under current U.S. federal tax law, the 
deposit, and our legal release from your debt security, would be treated as though we took back your debt 
security and gave you your share of the cash and notes or bonds deposited in trust. In that event, you could 
recognize gain or loss on your debt security; and 
• We must deliver to the trustee under the indenture a legal opinion of our counsel confirming the tax law 
treatment described above. 
If we ever fully defeased your debt security, you would have to rely solely on the trust deposit for payments 
on your debt security. You would not be able to look to us for payment in the event of any shortfall. 
Covenant Defeasance. Under current U.S. federal tax law, we can make the same type of deposit described 
above and be released from any restrictive covenants relating to your debt security. This is called covenant 
defeasance. In that event, you would lose the protection of those restrictive covenants. In order to achieve covenant 
defeasance for the debt securities, we must do both of the following: 
• We must deposit in trust for the benefit of the holders of those debt securities a combination of money and 
U.S. government or U.S. government agency notes or bonds that will generate enough cash to make interest, 
principal, and any other payments on those debt securities on their due dates; and 
• We must deliver to the trustee under the indenture a legal opinion of our counsel confirming that under 
current U.S. federal income tax law we may make the above deposit without causing the holders to be taxed 
on the debt securities any differently than if we did not make the deposit and repaid the debt securities 
ourselves. 
If we achieve covenant defeasance with respect to your debt security, you can still look to us for repayment of 
your debt security in the event of any shortfall in the trust deposit. You should note, however, that if one of the 
remaining events of default occurred, such as our bankruptcy, and your debt security became immediately due and 
payable, there may be a shortfall. Depending on the event causing the default, you may not be able to obtain 
payment of the shortfall. 
Notices 
We or the trustee on our behalf, if so requested, will provide the holders with any required notices by firstclass mail to the addresses of the holders as they appear in the security register. So long as a depository is the record 
holder of a series of debt securities with respect to which a notice is given, we or the trustee, if so requested, will 
deliver the notice only to that depository. 
36 
Table of Contents
Concerning the Trustees 
We and certain of our affiliates have from time to time maintained deposit accounts and conducted other 
banking transactions with The Bank of New York Mellon Trust Company, N.A. and its affiliates in the ordinary 
course of business. We expect to continue these business transactions. The Bank of New York Mellon Trust 
Company, N.A. and its affiliates also serve as trustee for a number of series of outstanding indebtedness of us and 
our affiliates under other indentures. 
Governing Law 
The indentures and the debt securities will be governed by New York law. 
DESCRIPTION OF WARRANTS 
General 
We may issue warrants, including debt warrants and universal warrants. We may offer warrants separately or 
as part of a unit, as described below under the heading “Description of Units.” 
We may issue warrants in any amounts or in as many distinct series as we determine. We will issue each series 
of debt warrants and universal warrants under a separate warrant agreement to be entered into between us and a 
warrant agent to be designated in the applicable supplement. When we refer to a series of warrants, we mean all 
warrants issued as part of the same series under the applicable warrant agreement. 
This section describes some of the general terms and provisions of the warrants. We will describe the specific 
terms of a series of warrants and the applicable warrant agreement in the applicable supplement. The following 
description and any description of the warrants in the applicable supplement may not be complete and is subject to 
and qualified in its entirety by reference to the terms and provisions of the applicable warrant agreement. A form of 
the warrant agreement reflecting the particular terms and provisions of a series of offered warrants will be filed with 
the SEC in connection with the offering and incorporated by reference in the registration statement and this 
prospectus. See “Where You Can Find More Information” below for information on how to obtain copies of any 
warrant agreements. 
Description of Debt Warrants 
Debt warrants are rights to purchase our debt securities. If debt warrants are offered, the supplement will 
describe the terms of the debt warrants and the warrant agreement relating to the debt warrants, including the 
following: 
• the offering price; 
• the designation, aggregate stated principal amount, and terms of the debt securities purchasable upon 
exercise of the debt warrants; 
• the currency, currency unit, or composite currency in which the price for the debt warrants is payable; 
• if applicable, the designation and terms of the debt securities with which the debt warrants are issued, and 
the number of debt warrants issued with each security; 
37 
Table of Contents
• if applicable, the date on and after which the debt warrants and the related debt securities will be separately 
transferable; 
• the principal amount of debt securities purchasable upon exercise of a debt warrant and the price at which, 
and the currency, currency units, or composite currency based on or relating to currencies in which, the 
principal amount of debt securities may be purchased upon exercise; 
• the dates the right to exercise the debt warrants will commence and expire and, if the debt warrants are not 
continuously exercisable, any dates on which the debt warrants are not exercisable; 
• any circumstances that will cause the debt warrants to be deemed to be automatically exercised; 
• if applicable, a discussion of the U.S. federal income tax consequences; 
• whether the debt warrants or related securities will be listed on any securities exchange; 
• whether the debt warrants will be issued in global or certificated form; 
• the name of the warrant agent; 
• a description of the terms of any warrant agreement to be entered into between us and a bank or trust 
company, as warrant agent, governing the debt warrants; and 
• any other terms of the debt warrants which are permitted under the warrant agreement. 
Description of Universal Warrants 
Universal warrants are rights to purchase or sell, or our delivery obligations are determined by reference to the 
performance, level, or value of, one or more of the following: 
• securities of one or more issuers, including our common or preferred stock or other securities described in 
this prospectus, or the debt or equity securities of third parties; 
• one or more currencies, currency units, or composite currencies; 
• one or more commodities; 
• any other financial, economic, or other measure or instrument, including the occurrence or non-occurrence 
of any event or circumstance; and 
• one or more indices or baskets of the items described above. 
We refer to each type of property described above as “warrant property.” 
We may satisfy our obligations, if any, and the holder of a universal warrant may satisfy its obligations, if any, 
with respect to any universal warrants by delivering the assets described in the applicable supplement, and in some 
cases, cash. 
38 
Table of Contents
If universal warrants are offered, the applicable supplement will describe the terms of the universal warrants 
and the warrant agreement, including the following: 
• the offering price; 
• the title and aggregate number of the universal warrants; 
• the nature and amount of the warrant property that the universal warrants represent the right to buy or sell; 
• whether the universal warrants are put warrants or call warrants, including in either case, the method by 
which the warrants may be settled; 
• the price at which the warrant property may be purchased or sold, the currency, and the procedures and 
conditions relating to exercise; 
• the method of exercising the universal warrants, the method of paying the exercise price, and the method of 
settling the warrant; 
• the dates on which the right to exercise the universal warrants will commence and expire; 
• if applicable, a discussion of the U.S. federal income tax consequences; 
• whether the universal warrants or underlying securities will be listed on any securities exchange; 
• whether the universal warrants will be issued in global or certificated form; 
• the name of the warrant agent; 
• a description of the terms of any warrant agreement to be entered into between us and a bank or trust 
company, as warrant agent, governing the universal warrants; and 
• any other terms of the universal warrants which are permitted under the warrant agreement. 
Modification 
We and the warrant agent may amend the terms of any warrant agreement and the warrants without the 
consent of the holders of the warrants to cure any ambiguity, to correct any inconsistent provision, or in any other 
manner we deem necessary or desirable and which will not affect adversely the interests of the holders. In addition, 
we may amend the warrant agreement and the terms of the warrants with the consent of the holders of a majority of 
the outstanding unexercised warrants affected. However, any modification to the warrants cannot change the 
exercise price, reduce the amounts receivable upon exercise, cancellation, or expiration, shorten the time period 
during which the warrants may be exercised, or otherwise materially and adversely affect the rights of the holders of 
the warrants or reduce the percentage of outstanding warrants required to modify or amend the warrant agreement or 
the terms of the warrants, without the consent of the affected holders. 
Enforceability of Rights of Warrantholders; No Trust Indenture Act Protection 
The warrant agent will act solely as our agent and will not assume any obligation or relationship of agency or 
trust with the holders of the warrants. Any record holder or beneficial 
39 
Table of Contents
owner of a warrant, without anyone else’s consent, may enforce by appropriate legal action, on his or her own 
behalf, his or her right to exercise the warrant in accordance with its terms. A holder of a warrant will not be entitled 
to any of the rights of a holder of the debt securities or other securities or warrant property purchasable upon the 
exercise of the warrant, including any right to receive payments on those securities or warrant property or to enforce 
any covenants or rights in the relevant indenture or any other agreement, before exercising the warrant. 
No warrant agreement will be qualified as an indenture, and no warrant agent under any warrant agreement 
will be required to qualify as a trustee, under the Trust Indenture Act of 1939. Therefore, holders of warrants issued 
under a warrant agreement will not have the protection of the Trust Indenture Act of 1939 with respect to their
warrants. 
Unsecured Obligations 
Any warrants we issue will be our unsecured contractual obligations. Because Bank of America Corporation is 
a holding company, our right to participate in any distribution of assets of any subsidiary upon such subsidiary’s 
liquidation or reorganization or otherwise is subject to the prior claims of creditors of that subsidiary, except to the 
extent we may ourselves be recognized as a creditor of that subsidiary. Accordingly, our obligations under our 
warrants will be structurally subordinated to all existing and future liabilities of our subsidiaries, and claimants 
should look only to Bank of America Corporation’s assets for payments. 
DESCRIPTION OF PURCHASE CONTRACTS 
General 
We may issue purchase contracts in any amounts and in as many distinct series as we determine. We may 
offer purchase contracts separately or as part of a unit, as described below under the heading “Description of Units.” 
When we refer to a series of purchase contracts, we mean all purchase contracts issued as part of the same series 
under the applicable purchase contract. 
This section describes some of the general terms and provisions applicable to all purchase contracts. We will 
describe the specific terms of a series of purchase contracts in the applicable supplement. The following description 
and any description of the purchase contracts in the applicable supplement may not be complete and is subject to and 
qualified in its entirety by reference to the terms and provisions of the applicable purchase contract. A form of the 
purchase contract reflecting the particular terms and provisions of a series of offered purchase contracts will be filed 
with the SEC in connection with the offering and incorporated by reference in the registration statement and this 
prospectus. See “Where You Can Find More Information” below for information on how to obtain copies of any 
purchase contracts. 
Purchase Contract Property 
We may issue purchase contracts for the purchase or sale of, or whose cash value is determined by reference 
or linked to the performance, level, or value of, one or more of the following: 
• securities of one or more issuers, including our common or preferred stock, other securities described in this 
prospectus, or the debt or equity securities of third parties; 
• one or more currencies, currency units, or composite currencies; 
• one or more commodities; 
40 
Table of Contents
• any other financial, economic, or other measure or instrument, including the occurrence or non-occurrence 
of any event or circumstance; and 
• one or more indices or baskets of the items described above. 
We refer to each type of property described above as a “purchase contract property.” 
Each purchase contract will obligate: 
• the holder to purchase or sell, and us to sell or purchase, on specified dates, one or more purchase contract 
properties at a specified price or prices; or 
• the holder or us to settle the purchase contract with a cash payment determined by reference to the value, 
performance, or level of one or more purchase contract properties, on specified dates and at a specified price 
or prices. 
No holder of a purchase contract will, as such, have any rights of a holder of the purchase contract property 
purchasable under or referenced in the contract, including any rights to receive payments on that property. 
Information in Supplement 
If we offer purchase contracts, the applicable supplement will describe the terms of the purchase contracts, 
including the following: 
• the purchase date or dates; 
• if other than U.S. dollars, the currency or currency unit in which payment will be made; 
• the specific designation and aggregate number of, and the price at which we will issue, the purchase 
contracts; 
• whether the purchase contract obligates the holder to purchase or sell, or both purchase and sell, one or more 
purchase contract properties, and the nature and amount of each of those properties, or the method of 
determining those amounts; 
• the purchase contract property or cash value, and the amount or method for determining the amount of 
purchase contract property or cash value, deliverable under each purchase contract; 
• whether the purchase contract is to be prepaid or not and the governing document for the contract; 
• the price at which the purchase contract is settled, and whether the purchase contract is to be settled by 
delivery of, or by reference or linkage to the value, performance, or level of, the purchase contract 
properties; 
• any acceleration, cancellation, termination, or other provisions relating to the settlement of the purchase 
contract; 
• if the purchase contract property is an index, the method of providing for a substitute index or indices or 
otherwise determining the amount payable; 
41 
Table of Contents
• if the purchase contract property is an index or a basket of securities, a description of the index or basket of 
securities; 
• whether, following the occurrence of a market disruption event or force majeure event (as defined in the 
applicable supplement), the settlement delivery obligation or cash settlement value of a purchase contract 
will be determined on a different basis than under normal circumstances; 
• whether the purchase contract will be issued as part of a unit and, if so, the other securities comprising the 
unit and whether any unit securities will be subject to a security interest in our favor as described below; 
• if applicable, a discussion of the U.S. federal income tax consequences; 
• the identities of any depositories and any paying, transfer, calculation, or other agents for the purchase 
contracts; 
• whether the purchase contract will be issued in global or certificated form; 
• any securities exchange or quotation system on which the purchase contracts or any securities deliverable in 
settlement of the purchase contracts may be listed; and 
• any other terms of the purchase contracts and any terms required by or advisable under applicable laws and 
regulations. 
Prepaid Purchase Contracts; Applicability of Indenture 
Purchase contracts may require holders to satisfy their obligations under the purchase contracts at the time 
they are issued. We refer to these contracts as “prepaid purchase contracts.” 
In certain circumstances, our obligation to settle a prepaid purchase contract on the relevant settlement date 
may constitute our senior debt securities or our subordinated debt securities. Accordingly, prepaid purchase 
contracts may be issued under the Senior Indenture or the Subordinated Indenture, which are described above under 
the heading “Description of Debt Securities.” 
Non-Prepaid Purchase Contracts; No Trust Indenture Act Protection 
Some purchase contracts do not require holders to satisfy their obligations under the purchase contracts until 
settlement. We refer to these contracts as “non-prepaid purchase contracts.” The holder of a non-prepaid purchase 
contract may remain obligated to perform under the contract for a substantial period of time. 
Non-prepaid purchase contracts will be issued under a unit agreement, if they are issued in units, or under 
some other document, if they are not. We describe unit agreements generally under the heading “Description of 
Units” below. We will describe the particular governing document that applies to your non-prepaid purchase 
contracts in the applicable supplement. 
Non-prepaid purchase contracts will not be our senior debt securities or subordinated debt securities and will 
not be issued under one of our indentures, unless we specify otherwise in the applicable supplement. Consequently, 
no governing documents for non-prepaid purchase contracts will be qualified as indentures, and no third party will 
be required to qualify as a trustee with regard to those contracts, under the Trust Indenture Act of 1939. Therefore, 
holders of non-prepaid purchase contracts will not have the protection of the Trust Indenture Act of 1939. 
42 
Table of Contents
Pledge by Holders to Secure Performance 
If we so specify in the applicable supplement, the holder’s obligations under the purchase contract and 
governing document will be secured by collateral. In that case, the holder, acting through the unit agent as its 
attorney-in-fact, if applicable, will pledge the items described below to a collateral agent that we will identify in the 
applicable supplement, which will hold them, for our benefit, as collateral to secure the holder’s obligations. We 
refer to this as the “pledge” and all the items described below as the “pledged items.” Unless we specify otherwise in 
the applicable supplement, the pledge will create a security interest in the holder’s entire interest in and to: 
• any other securities included in the unit, if the purchase contract is part of a unit, and/or any other property 
specified in the applicable supplement; 
• all additions to and substitutions for the pledged items; 
• all income, proceeds, and collections received in respect of the pledged items; and 
• all powers and rights owned or acquired later with respect to the pledged items. 
The collateral agent will forward all payments and proceeds from the pledged items to us, unless the payments 
and proceeds have been released from the pledge in accordance with the purchase contract and the governing 
document. We will use the payments and proceeds from the pledged items to satisfy the holder’s obligations under 
the purchase contract. 
Settlement of Purchase Contracts that Are Part of Units 
Unless we specify otherwise in the applicable supplement, where purchase contracts issued together with debt 
securities as part of a unit require the holders to buy purchase contract property, the unit agent may apply principal 
payments from the debt securities in satisfaction of the holders’ obligations under the related purchase contract as 
specified in the applicable supplement. The unit agent will not so apply the principal payments if the holder has 
delivered cash to meet its obligations under the purchase contract. If the holder is permitted to settle its obligations 
by cash payment, the holder may be permitted to do so by delivering the debt securities in the unit to the unit agent 
as provided in the governing document. If the holder settles its obligations in cash rather than by delivering the debt 
security that is part of the unit, that debt security will remain outstanding, if the maturity extends beyond the relevant 
settlement date and, as more fully described in the applicable supplement, the holder will receive that debt security 
or an interest in the relevant global debt security. 
Book-entry and other indirect owners should consult their banks or brokers for information on how to settle 
their purchase contracts. 
Failure of Holder to Perform Obligations 
If the holder fails to settle its obligations under a non-prepaid purchase contract as required, the holder will not 
receive the purchase contract property or other consideration to be delivered at settlement. Holders that fail to make 
timely settlement also may be obligated to pay interest or other amounts. 
Unsecured Obligations 
The purchase contracts are our unsecured contractual obligations. Because Bank of America Corporation is a 
holding company, our right to participate in any distribution of assets of any subsidiary upon such subsidiary’s 
liquidation or reorganization or otherwise is subject to the prior 
43 
Table of Contents
claims of creditors of that subsidiary, except to the extent we may ourselves be recognized as a creditor of that 
subsidiary. Accordingly, our obligations under our purchase contracts will be structurally subordinated to all 
existing and future liabilities of Bank of America Corporation’s subsidiaries, and claimants should look only to our 
assets for payments. 
DESCRIPTION OF UNITS 
General 
We may issue units from time to time in such amounts and in as many distinct series as we determine. 
We will issue each series of units under a unit agreement to be entered into between us and a unit agent to be 
designated in the applicable supplement. When we refer to a series of units, we mean all units issued as part of the 
same series under the applicable unit agreement. 
This section describes some of the general terms and provisions applicable to all the units. We will describe 
the specific terms of a series of units and the applicable unit agreement in the applicable supplement. The following 
description and any description of the units in the applicable supplement may not be complete and is subject to and 
qualified in its entirety by reference to the terms and provisions of the applicable unit agreement. A form of the unit 
agreement reflecting the particular terms and provisions of a series of offered units will be filed with the SEC in 
connection with the offering and incorporated by reference in the registration statement and this prospectus. See 
“Where You Can Find More Information” below for information on how to obtain copies of any unit agreements. 
We may issue units consisting of any combination of two or more securities described in this prospectus or 
securities of third parties, in any combination. Each unit will be issued so that the holder of the unit is also the holder 
of each security included in the unit. Thus, the holder of a unit will have the rights and obligations of a holder of 
each included security. The unit agreement under which a unit is issued may provide that the securities included in 
the unit may not be held or transferred separately, at any time or at any time before a specified date. 
If units are offered, the applicable supplement will describe the terms of the units, including the following: 
• the designation and aggregate number of, and the price at which we will issue, the units; 
• the terms of the units and of the securities comprising the units, including whether and under what 
circumstances the securities comprising the units may or may not be held or transferred separately; 
• the name of the unit agent; 
• a description of the terms of any unit agreement to be entered into between us and a bank or trust company, 
as unit agent, governing the units; 
• if applicable, a discussion of the U.S. federal income tax consequences; 
• whether the units will be listed on any securities exchange; and 
• a description of the provisions for the payment, settlement, transfer, or exchange of the units. 
44 
Table of Contents
Unit Agreements: Prepaid, Non-Prepaid, and Other 
If a unit includes one or more purchase contracts, and all those purchase contracts are prepaid purchase 
contracts, we will issue the unit under a “prepaid unit agreement.” Prepaid unit agreements will reflect the fact that 
the holders of the related units have no further obligations under the purchase contracts included in their units. If a 
unit includes one or more non-prepaid purchase contracts, we will issue the unit under a “non-prepaid unit 
agreement.” Non-prepaid unit agreements will reflect the fact that the holders have payment or other obligations 
under one or more of the purchase contracts comprising their units. We may also issue units under other kinds of 
unit agreements, which will be described in the applicable supplement, if applicable. 
Each holder of units issued under a non-prepaid unit agreement will: 
• be bound by the terms of each non-prepaid purchase contract included in the holder’s units and by the terms 
of the unit agreement with respect to those contracts; and 
• appoint the unit agent as its authorized agent to execute, deliver, and perform on the holder’s behalf each 
non-prepaid purchase contract included in the holder’s units. 
Any unit agreement for a unit that includes a non-prepaid purchase contract also will include provisions 
regarding the holder’s pledge of collateral and special settlement provisions. These are described above under the 
heading “Description of Purchase Contracts.” 
A unit agreement also may serve as the governing document for a security included in a unit. For example, a 
non-prepaid purchase contract that is part of a unit may be issued under and governed by the relevant unit 
agreement. 
Modification 
We and the unit agent may amend the terms of any unit agreement and the units without the consent of the 
holders to cure any ambiguity, to correct any inconsistent provision, or in any other manner we deem necessary or 
desirable and which will not affect adversely the interests of the holders. In addition, we may amend the unit 
agreement and the terms of the units with the consent of the holders of a majority of the outstanding unexpired units 
affected. However, any modification to the units that materially and adversely affects the rights of the holders of the 
units, or reduces the percentage of outstanding units required to modify or amend the unit agreement or the terms of 
the units, requires the consent of the affected holders. 
Enforceability of Rights of Unitholders; No Trust Indenture Act Protection 
The unit agent will act solely as our agent and will not assume any obligation or relationship of agency or trust 
with the holders of the units. Except as described below, any record holder of a unit, without anyone else’s consent, 
may enforce his or her rights as holder under any security included in the unit, in accordance with the terms of the 
included security and the indenture, warrant agreement, unit agreement, or purchase contract under which that 
security is issued. We describe these terms in other sections of this prospectus relating to debt securities, warrants, 
and purchase contracts. 
Notwithstanding the foregoing, a unit agreement may limit or otherwise affect the ability of a holder of units 
issued under that agreement to enforce his or her rights, including any right to bring legal action, with respect to 
those units or any included securities, other than debt securities. We will describe any limitations of this kind in the 
applicable supplement. 
45 
Table of Contents
No unit agreement will be qualified as an indenture, and no unit agent will be required to qualify as a trustee 
under the Trust Indenture Act of 1939. Therefore, holders of units issued under a unit agreement will not have the 
protection of the Trust Indenture Act of 1939 with respect to their units. 
Unsecured Obligations 
The units are our unsecured contractual obligations. Because Bank of America Corporation is a holding 
company, our right to participate in any distribution of assets of any subsidiary upon such subsidiary’s liquidation or 
reorganization or otherwise is subject to the prior claims of creditors of that subsidiary, except to the extent we may 
ourselves be recognized as a creditor of that subsidiary. Accordingly, our obligations under our units will be 
structurally subordinated to all existing and future liabilities of our subsidiaries, and claimants should look only to 
Bank of America Corporation’s assets for payments. 
DESCRIPTION OF PREFERRED STOCK 
General 
As of the date of this prospectus, under our Amended and Restated Certificate of Incorporation, we have 
authority to issue 100,000,000 shares of preferred stock, par value $.01 per share. We may issue preferred stock in 
one or more series, each with the preferences, designations, limitations, conversion rights, and other rights as we 
may determine. Of our authorized and outstanding preferred stock, as of March 15, 2012: 
• 35,045 shares were designated as 7% Cumulative Redeemable Preferred Stock, Series B, having a 
liquidation preference of $100 per share, 7,571 shares of which were issued and outstanding; 
• 34,500 shares were designated as 6.204% Non-Cumulative Preferred Stock, Series D, having a liquidation 
preference of $25,000 per share, 26,174 shares of which were issued and outstanding; 
• 85,100 shares were designated as Floating Rate Non-Cumulative Preferred Stock, Series E, having a 
liquidation preference of $25,000 per share, 12,691 shares of which were issued and outstanding; 
• 7,001 shares were designated as Floating Rate Non-Cumulative Preferred Stock, Series F, having a 
liquidation preference of $100,000 per share, 1,409.22 shares of which were issued and outstanding; 
• 8,501 shares were designated as Adjustable Rate Non-Cumulative Preferred Stock, Series G, having a 
liquidation preference of $100,000 per share, 4,925.37 shares of which were issued and outstanding; 
• 124,200 shares were designated as 8.20% Non-Cumulative Preferred Stock, Series H, having a liquidation 
preference of $25,000 per share, 114,483 shares of which were issued and outstanding; 
• 25,300 shares were designated as 6.625% Non-Cumulative Preferred Stock, Series I, having a liquidation 
preference of $25,000 per share, 14,584 shares of which were issued and outstanding; 
46 
Table of Contents
• 41,400 shares were designated as 7.25% Non-Cumulative Preferred Stock, Series J, having a liquidation 
preference of $25,000 per share, 38,053 shares of which were issued and outstanding; 
• 240,000 shares were designated as Fixed-to-Floating Rate Non-Cumulative Preferred Stock, Series K, 
having a liquidation preference of $25,000 per share, 61,773 shares of which were issued and outstanding; 
• 6,900,000 shares were designated as 7.25% Non-Cumulative Perpetual Convertible Preferred Stock, Series 
L, having a liquidation preference of $1,000 per share, 3,080,182 shares of which were issued and 
outstanding; 
• 160,000 shares were designated as Fixed-to-Floating Rate Non-Cumulative Preferred Stock, Series M, 
having a liquidation preference of $25,000 per share, 52,399 shares of which were issued and outstanding;
• 50,000 shares were designated as 6% Cumulative Perpetual Preferred Stock, Series T, having a liquidation 
preference of $100,000 per share, 50,000 shares of which were issued and outstanding; 
• 21,000 shares were designated as Floating Rate Non-Cumulative Preferred Stock, Series 1, having a 
liquidation preference of $30,000 per share, 3,318 shares of which were issued and outstanding; 
• 37,000 shares were designated as Floating Rate Non-Cumulative Preferred Stock, Series 2, having a 
liquidation preference of $30,000 per share, 10,227 shares of which were issued and outstanding; 
• 27,000 shares were designated as 6.375% Non-Cumulative Preferred Stock, Series 3, having a liquidation 
preference of $30,000 per share, 21,773 shares of which were issued and outstanding; 
• 20,000 shares were designated as Floating Rate Non-Cumulative Preferred Stock, Series 4, having a 
liquidation preference of $30,000 per share, 7,061 shares of which were issued and outstanding; 
• 50,000 shares were designated as Floating Rate Non-Cumulative Preferred Stock, Series 5, having a 
liquidation preference of $30,000 per share, 14,181 shares of which were issued and outstanding; 
• 65,000 shares were designated as 6.70% Noncumulative Perpetual Preferred Stock, Series 6, having a 
liquidation preference of $1,000 per share, 59,388 shares of which were issued and outstanding; 
• 50,000 shares were designated as 6.25% Noncumulative Perpetual Preferred Stock, Series 7, having a 
liquidation preference of $1,000 per share, 16,596 shares of which were issued and outstanding; and 
• 89,100 shares were designated as 8.625% Non-Cumulative Preferred Stock, Series 8, having a liquidation 
preference of $30,000 per share, 89,100 shares of which were issued and outstanding. 
47 
Table of Contents
In addition, as of March 15, 2012, the following series of preferred stock were designated, but no shares of 
any of these series were outstanding: 
• 3 million shares of ESOP Convertible Preferred Stock, Series C; 
• 20 million shares of $2.50 Cumulative Convertible Preferred Stock, Series BB; 
• 600,000 shares of Fixed Rate Cumulative Perpetual Preferred Stock, Series N; 
• 400,000 shares of Fixed Rate Cumulative Perpetual Preferred Stock, Series Q; 
• 800,000 shares of Fixed Rate Cumulative Perpetual Preferred Stock, Series R; and 
• 1,286,000 shares of Common Equivalent Junior Preferred Stock, Series S. 
We refer to all of our preferred stock summarized above as our existing preferred stock. This brief summary 
does not purport to be complete and is qualified in its entirety by reference to the description of these securities 
contained in our Amended and Restated Certificate of Incorporation and the respective certificates of designation for 
each series of our existing preferred stock. In addition, for a more complete description of our outstanding preferred 
stock, see the information contained in our current reports on Form 8-K filed with the SEC on April 20, 2009 and 
September 1, 2011, which are incorporated by reference in this prospectus. We may update this description of some 
or all of our existing preferred stock from time to time in reports that we file under the Exchange Act. 
The Preferred Stock 
General. Any preferred stock sold under this prospectus will have the general dividend, voting, and 
liquidation preference rights stated below unless we specify otherwise in the applicable supplement. The applicable 
supplement for a series of preferred stock will describe the specific terms of those shares, including, where 
applicable: 
• the title and stated value of the preferred stock; 
• the aggregate number of shares of preferred stock offered; 
• the offering price or prices of the preferred stock; 
• the dividend rate or rates or method of calculation, the dividend period, and the dates dividends will be 
payable; 
• whether dividends are cumulative or noncumulative, and, if cumulative, the date the dividends will begin to 
cumulate; 
• the dividend and liquidation preference rights of the preferred stock relative to any existing or future series 
of our preferred stock; 
• the dates the preferred stock become subject to redemption at our option, and any redemption terms; 
• any redemption or sinking fund provisions; 
• whether the preferred stock will be issued in other than book-entry form; 
48 
Table of Contents
• whether the preferred stock will be listed on any securities exchange; 
• any rights on the part of the stockholder or us to convert the preferred stock into shares of our common 
stock or any other security; and 
• any additional voting, liquidation, preemptive, and other rights, preferences, privileges, limitations, and 
restrictions. 
Dividends. The holders of our preferred stock will be entitled to receive when, as, and if declared by our board 
of directors, cash dividends at those rates as will be fixed by our board of directors, subject to the terms of our 
Amended and Restated Certificate of Incorporation. All dividends will be paid out of funds that are legally available 
for this purpose. Unless we specify otherwise in the applicable supplement, whenever dividends on any non-voting 
preferred stock are in arrears for six quarterly dividend periods (whether or not consecutive), holders of the nonvoting preferred stock will have the right to elect two additional directors to serve on our board of directors, and 
these two additional directors will continue to serve until full dividends on such non-voting preferred stock have 
been paid regularly for at least four quarterly dividend periods. 
Voting. The holders of our preferred stock will have no voting rights except: 
• as required by applicable law; or 
• as specifically approved by us for that particular series. 
Under regulations adopted by the Board of Governors of the Federal Reserve System (the “Federal Reserve 
Board”), if the holders of any series of our preferred stock become entitled to vote for the election of directors 
because dividends on that series are in arrears, that series may then be deemed a “class of voting securities.” In such 
a case, a holder of 25% or more of the series, or a holder of 5% or more if that holder would also be considered to 
exercise a “controlling influence” over us, may then be subject to regulation as a bank holding company in 
accordance with the The Bank Holding Company Act of 1956. In addition, (1) any other bank holding company may 
be required to obtain the prior approval of the Federal Reserve Board to acquire or retain 5% or more of that series, 
and (2) any person other than a bank holding company may be required to obtain the approval of the Federal 
Reserve Board to acquire or retain 10% or more of that series. 
Liquidation Preference. In the event of our voluntary or involuntary dissolution, liquidation, or winding up, 
the holders of any series of our preferred stock will be entitled to receive, after distributions to holders of any series 
or class of our capital stock ranking superior, an amount equal to the stated or liquidation value of the shares of the
series plus an amount equal to accrued and unpaid dividends. If the assets and funds to be distributed among the 
holders of our preferred stock will be insufficient to permit full payment to the holders, then the holders of our 
preferred stock will share ratably in any distribution of our assets in proportion to the amounts that they otherwise 
would receive on their shares of our preferred stock if the shares were paid in full. 
DESCRIPTION OF DEPOSITARY SHARES 
General 
We may offer depositary receipts evidencing depositary shares, each of which will represent a fractional 
interest in shares of preferred stock, rather than full shares of these securities. We will deposit shares of preferred 
stock of each series represented by depositary shares under a deposit agreement between us and a U.S. bank or trust 
company that we will select (the “depository”). 
49 
Table of Contents
This section describes some of the general terms and provisions applicable to all depositary shares. We will 
describe the specific terms of a series of depositary shares and the deposit agreement in the applicable supplement. 
The following description and any description of the depositary shares in the applicable supplement may not be 
complete and is subject to and qualified in its entirety by reference to the terms and provisions of the applicable 
deposit agreement and depositary receipts. Forms of the deposit agreement and depositary receipts reflecting the 
particular terms and provisions of a series of offered depositary shares will be filed with the SEC in connection with 
the offering and incorporated by reference in the registration statement and this prospectus. See “Where You Can 
Find More Information” below for information on how to obtain copies of any deposit agreements and depositary 
receipts. 
Terms of the Depositary Shares 
Depositary receipts issued under the deposit agreement will evidence the depositary shares. Depositary 
receipts will be distributed to those persons purchasing depositary shares representing fractional shares of preferred 
stock in accordance with the terms of the offering. Subject to the terms of the deposit agreement, each holder of a 
depositary share will be entitled, in proportion to the fractional interest of a share of preferred stock represented by 
the applicable depositary share, to all the rights and preferences of the preferred stock being represented, including 
dividend, voting, redemption, conversion, and liquidation rights, all as will be set forth in the applicable supplement 
relating to the depositary shares being offered. 
Pending the preparation of definitive depositary receipts, the depository, upon our written order, may issue 
temporary depositary receipts. The temporary depositary receipts will be substantially identical to, and will have all 
the rights of, the definitive depositary receipts, but will not be in definitive form. Definitive depositary receipts will 
be prepared thereafter and temporary depositary receipts will be exchanged for definitive depositary receipts at our 
expense. 
Withdrawal of Preferred Stock 
Unless the depositary shares have been called for redemption, a holder of depositary shares may surrender his 
or her depositary receipts at the principal office of the depository, pay any charges, and comply with any other terms 
as provided in the deposit agreement for the number of shares of preferred stock underlying the depositary shares. A 
holder of depositary shares who withdraws shares of preferred stock will be entitled to receive whole shares of 
preferred stock on the basis set forth in the applicable supplement relating to the depositary shares being offered. 
However, unless we specify otherwise in the applicable supplement, holders of whole shares of preferred 
stock will not be entitled to deposit those shares under the deposit agreement or to receive depositary receipts for 
those shares after the withdrawal. If the depositary shares surrendered by the holder in connection with the 
withdrawal exceed the number of depositary shares that represent the number of whole shares of preferred stock to 
be withdrawn, the depository will deliver to the holder at the same time a new depositary receipt evidencing the 
excess number of depositary shares. 
Dividends and Other Distributions 
The depository will distribute all cash dividends or other cash distributions received in respect of the preferred 
stock to the record holders of depositary shares relating to that preferred stock in proportion to the number of 
depositary shares owned by those holders. However, the depository will distribute only the amount that can be 
distributed without attributing to any holder of depositary 
50 
Table of Contents
shares a fraction of one cent. Any balance that is not distributed will be added to and treated as part of the next sum 
received by the depository for distribution to record holders. 
If there is a distribution other than in cash, the depository will distribute property it receives to the record 
holders of depositary shares who are entitled to that property. However, if the depository determines that it is not 
feasible to make this distribution of property, the depository, with our approval, may sell that property and distribute 
the net proceeds to the holders of the depositary shares. 
Redemption of Depositary Shares 
If a series of preferred stock which relates to depositary shares is redeemed, the depositary shares will be 
redeemed from the proceeds received by the depository from the redemption, in whole or in part, of that series of 
preferred stock. Unless we specify otherwise in the applicable supplement, the depository will mail notice of 
redemption at least 30 and not more than 45 calendar days before the date fixed for redemption to the record holders 
of the depositary shares to be redeemed at their addresses appearing in the depository’s books. The redemption price 
per depositary share will be equal to the applicable fraction of the redemption price per share payable on that series 
of the preferred stock. 
Whenever we redeem preferred stock held by the depository, the depository will redeem as of the same 
redemption date the number of depositary shares representing the preferred stock redeemed. If less than all of the 
depositary shares are redeemed, the depositary shares redeemed will be selected by lot or pro rata or by any other 
equitable method as the depository may decide. 
After the date fixed for redemption, the depositary shares called for redemption will no longer be deemed to 
be outstanding. At that time, all rights of the holder of the depositary shares will cease, except the right to receive 
any money or other property they become entitled to receive upon surrender to the depository of the depositary 
receipts. 
Voting the Deposited Preferred Stock 
Any voting rights of holders of the depositary shares are directly dependent on the voting rights of the 
underlying preferred stock, which customarily have limited voting rights. Upon receipt of notice of any meeting at 
which the holders of the preferred stock held by the depository are entitled to vote, the depository will mail the 
information contained in the notice of meeting to the record holders of the depositary shares relating to the preferred 
stock. Each record holder of depositary shares on the record date, which will be the same date as the record date for 
the preferred stock, will be entitled to instruct the depository as to the exercise of the voting rights pertaining to the 
amount of preferred stock underlying the holder’s depositary shares. The depository will endeavor, insofar as 
practicable, to vote the amount of preferred stock underlying the depositary shares in accordance with these 
instructions. We will agree to take all action which may be deemed necessary by the depository to enable the 
depository to do so. The depository will not vote any shares of preferred stock except to the extent it receives 
specific instructions from the holders of depositary shares representing that number of shares of preferred stock. 
Amendment and Termination of the Deposit Agreement 
The form of depositary receipt evidencing the depositary shares and any provision of the deposit agreement 
may be amended by agreement between us and the depository. However, any amendment which materially and 
adversely alters the rights of the existing holders of depositary 
51 
Table of Contents
shares will not be effective unless the amendment has been approved by the record holders of at least a majority of 
the depositary shares then outstanding. Either we or the depository may terminate a deposit agreement if all of the 
outstanding depositary shares have been redeemed or if there has been a final distribution in respect of our preferred 
stock in connection with our liquidation, dissolution, or winding up. 
Charges of Depository 
We will pay all transfer and other taxes, assessments, and governmental charges arising solely from the 
existence of the depository arrangements. We will pay the fees of the depository in connection with the initial 
deposit of the preferred stock and any redemption of the preferred stock. Holders of depositary receipts will pay 
transfer and other taxes, assessments, and governmental charges and any other charges as are expressly provided in 
the deposit agreement to be for their accounts. The depository may refuse to effect any transfer of a depositary 
receipt or any withdrawals of preferred stock evidenced by a depositary receipt until all taxes, assessments, and 
governmental charges with respect to the depositary receipt or preferred stock are paid by their holders. 
Miscellaneous 
The depository will forward to the holders of depositary shares all of our reports and communications which 
are delivered to the depository and which we are required to furnish to the holders of our preferred stock. 
Neither we nor the depository will be liable if we are prevented or delayed by law or any circumstance beyond 
our control in performing our obligations under the deposit agreement. All of our obligations as well as the 
depository’s obligations under the deposit agreement are limited to performance in good faith of our respective 
duties set forth in the deposit agreement, and neither of us will be obligated to prosecute or defend any legal 
proceeding relating to any depositary shares or preferred stock unless provided with satisfactory indemnity. We, and 
the depository, may rely upon written advice of counsel or accountants, or information provided by persons 
presenting preferred stock for deposit, holders of depositary shares, or other persons believed to be competent and 
on documents believed to be genuine. 
Resignation and Removal of Depository 
The depository may resign at any time by delivering to us notice of its election to do so, and we may remove 
the depository at any time. Any resignation or removal will take effect only upon the appointment of a successor 
depository and the successor depository’s acceptance of the appointment. Any successor depository must be a U.S. 
bank or trust company. 
DESCRIPTION OF COMMON STOCK 
The following summary of our common stock is qualified in its entirety by reference to the description of the 
common stock incorporated by reference in this prospectus. 
General 
As of the date of this prospectus, under our Amended and Restated Certificate of Incorporation, we are 
authorized to issue twelve billion eight hundred million (12,800,000,000) shares of common 
52 
Table of Contents
stock, par value $.01 per share, of which approximately 10.77 billion shares were outstanding on March 15, 2012. 
Our common stock trades on the New York Stock Exchange under the symbol “BAC.” Our common stock also is 
listed on the London Stock Exchange, and certain shares are listed on the Tokyo Stock Exchange. As of March 15, 
2012, approximately 1.91 billion shares were reserved for issuance in connection with our various employee and 
director benefit plans, the conversion of outstanding securities convertible into shares of our common stock, and for 
other purposes. After taking into account the reserved shares, there were approximately 114 million authorized 
shares of our common stock available for issuance as of March 15, 2012. 
Voting and Other Rights 
Holders of our common stock are entitled to one vote per share. There are no cumulative voting rights. In 
general, a majority of votes cast on a matter is sufficient to take action upon routine matters, including the election 
of directors in an uncontested election. However, (1) amendments to our Amended and Restated Certificate of 
Incorporation generally must be approved by the affirmative vote of the holders of a majority of the voting power of 
the outstanding stock, and (2) a merger, dissolution, or the sale of all or substantially all of our assets generally must 
be approved by the affirmative vote of the holders of a majority of the voting power of the outstanding stock. 
In the event of our liquidation, holders of our common stock will be entitled to receive pro rata any assets 
legally available for distribution to stockholders, subject to any prior rights of any preferred stock then outstanding. 
Our common stock does not have any preemptive rights, redemption privileges, sinking fund privileges, or 
conversion rights. All the outstanding shares of our common stock are, and upon proper conversion of any 
convertible securities, all of the shares of our common stock into which those securities are converted will be, 
validly issued, fully paid, and nonassessable. 
Computershare Trust Company, N.A. is the transfer agent and registrar for our common stock. 
Dividends 
Subject to the preferential rights of any holders of any outstanding series of preferred stock, the holders of our 
common stock are entitled to receive dividends or distributions, whether payable in cash or otherwise, as our board 
of directors may declare out of funds legally available for payments. Stock dividends, if any are declared, may be 
paid from our authorized but unissued shares of common stock. 
53 
Table of Contents
REGISTRATION AND SETTLEMENT 
Unless we specify otherwise in the applicable supplement, we will issue the securities in registered, and not 
bearer, form. This means that our obligation runs to the holder of the security named on the face of the security. 
Each debt security, warrant, purchase contract, unit, share of preferred stock, and depositary share issued in 
registered form will be represented either by a certificate issued in definitive form to a particular investor or by one 
or more global securities representing the entire issuance of securities. 
We refer to those persons who have securities registered in their own names, on the books that we or the 
trustee, warrant agent, or other agent maintain for this purpose, as the “holders” of those securities. These persons 
are the legal holders of the securities. We refer to those who, indirectly through others, own beneficial interests in 
securities that are not registered in their own names as indirect owners of those securities. As we discuss below, 
indirect owners are not legal holders, and investors in securities issued in global, or book-entry, form or in street 
name will be indirect owners. 
Book-Entry Only Issuance 
Unless we specify otherwise in the applicable supplement, we will issue each security other than our common 
stock in global, or book-entry, form. This means that we will not issue actual notes or certificates to investors. 
Instead, we will issue global securities in registered form representing the entire issuance of securities. Each global 
security will be registered in the name of a financial institution or clearing system that holds the global security as 
depository on behalf of other financial institutions that participate in that depository’s book-entry system. These 
participating institutions, in turn, hold beneficial interests in the global securities on behalf of themselves or their 
customers. 
Because securities issued in global form are registered in the name of the depository, we will recognize only 
the depository as the holder of the securities. This means that we will make all payments on the securities, including 
deliveries of any property other than cash, to the depository. The depository passes along the payments it receives 
from us to its participants, which in turn pass the payments along to their customers who are the beneficial owners. 
The depository and its participants are not obligated to pass these payments along under the terms of the securities. 
Instead, they do so under agreements they have made with one another or with their customers. 
As a result, investors will not own securities issued in book-entry form directly. Instead, they will own 
beneficial interests in a global security, through a bank, broker, or other financial institution that participates in the 
depository’s book-entry system or holds an interest through a participant in the depository’s book-entry system. As 
long as the securities are issued in global form, investors will be indirect owners, and not holders, of the securities. 
The depository will not have knowledge of the actual beneficial owners of the securities. 
Certificates in Registered Form 
In the future, we may cancel a global security or we may issue securities initially in non-global, or certificated, 
form. We do not expect to exchange global securities for actual notes or certificates registered in the names of the 
beneficial owners of the global securities representing the securities unless: 
• the depository notifies us that it is unwilling or unable to continue as depository for the global securities, or 
we become aware that the depository has ceased to be a clearing agency registered under the Securities 
Exchange Act of 1934, and in any case we fail to appoint a successor to the depository within 60 calendar 
days; or 
54 
Table of Contents
• we, in our sole discretion, determine that the global securities will be exchangeable for certificated 
securities. 
Street Name Owners 
When we issue actual notes or certificates registered in the names of the beneficial owners, investors may 
choose to hold their securities in their own names or in street name. Securities held by an investor in street name 
would be registered in the name of a bank, broker, or other financial institution that the investor chooses, and the 
investor would hold only a beneficial interest in those securities through an account that he or she maintains at that 
institution. 
For securities held in street name, we will recognize only the intermediary banks, brokers, and other financial 
institutions in whose names the securities are registered as the holders of those securities, and we will make all 
payments on those securities, including deliveries of any property other than cash, to them. These institutions pass 
along the payments they receive to their customers who are the beneficial owners, but only because they agree to do 
so in their customer agreements or because they are legally required to do so. Investors who hold securities in street 
name will be indirect owners, not holders, of those securities. 
Legal Holders 
Our obligations, as well as the obligations of the trustee under any indenture and the obligations, if any, of any 
warrant agents, unit agents, depository for depositary shares, and any other third parties employed by us, the trustee, 
or any of those agents, run only to the holders of the securities. We do not have obligations to investors who hold 
beneficial interests in global securities, who hold the securities in street name, or who hold the securities by any 
other indirect means. This will be the case whether an investor chooses to be an indirect owner of a security or has 
no choice because we are issuing the securities only in global form. For example, once we make a payment or give a 
notice to the holder, we have no further responsibility for that payment or notice even if that holder is required, 
under agreements with depository participants or customers or by law, to pass it along to the indirect owners, but 
does not do so. Similarly, if we want to obtain the approval of the holders for any purpose, such as to amend the 
indenture for a series of debt securities or the warrant agreement for a series of warrants or the unit agreement for a 
series of units or to relieve us of the consequences of a default or of our obligation to comply with a particular 
provision of an indenture, we would seek the approval only from the holders, and not the indirect owners, of the 
relevant securities. Whether and how the holders contact the indirect owners is up to the holders. 
When we refer to “you” in this prospectus, we mean those who invest in the securities being offered by this 
prospectus, whether they are the holders or only indirect owners of those securities. When we refer to “your 
securities” in this prospectus, we mean the securities in which you will hold a direct or indirect interest. 
Special Considerations for Indirect Owners 
If you hold securities through a bank, broker, or other financial institution, either in book-entry form or in 
street name, you should check with your own institution to find out: 
• how it handles payments on your securities and notices; 
• whether you can provide contact information to the registrar to receive copies of notices directly; 
55 
Table of Contents
• whether it imposes fees or charges; 
• whether and how you can instruct it to exercise any rights to purchase or sell warrant property under a 
warrant or purchase contract property under a purchase contract or to exchange or convert a security for or 
into other property; 
• how it would handle a request for the holders’ consent, if required; 
• whether and how you can instruct it to send you the securities registered in your own name so you can be a 
holder, if that is permitted at any time; 
• how it would exercise rights under the securities if there were a default or other event triggering the need for 
holders to act to protect their interests; and 
• if the securities are in book-entry form, how the depository’s rules and procedures will affect these matters. 
Depositories for Global Securities 
Each security issued in book-entry form will be represented by a global security that we deposit with and 
register in the name of one or more financial institutions or clearing systems, or their nominees, which we will 
select. A financial institution or clearing system that we select for this purpose is called the “depository” for that 
security. A security usually will have only one depository, but it may have more. 
Each series of securities will have one or more of the following as the depositories: 
• The Depository Trust Company, New York, New York, which is known as “DTC”; 
• a financial institution holding the securities on behalf of Euroclear Bank SA/NV, which is known as 
“Euroclear”; 
• a financial institution holding the securities on behalf of Clearstream Banking, société anonyme, 
Luxembourg, which is known as “Clearstream, Luxembourg”; and 
• any other clearing system or financial institution that we identify in the applicable supplement. 
The depositories named above also may be participants in one another’s clearing systems. For example, if 
DTC is the depository for a global security, investors may hold beneficial interests in that security through Euroclear 
or Clearstream, Luxembourg as DTC participants. 
We will name the depository or depositories for your securities in the applicable supplement. If no depository 
is named, the depository will be DTC. 
The Depository Trust Company 
The following is based on information furnished to us by DTC: 
DTC will act as securities depository for the securities. The securities will be issued as fully-registered 
securities registered in the name of Cede & Co. (DTC’s partnership nominee) or any other name as may be 
requested by an authorized representative of DTC. One fully-registered 
56 
Table of Contents
security certificate will be issued for each issue of the securities, each in the aggregate principal amount of the 
issue, and will be deposited with DTC. If, however, the aggregate principal amount of any issue exceeds $500 
million, one certificate will be issued with respect to each $500 million of principal amount, and an additional 
certificate will be issued with respect to any remaining principal amount of the issue. We may also issue one 
or more global securities that represent multiple series of debt securities. 
DTC, the world’s largest securities depository, is a limited-purpose trust company organized under the 
New York Banking Law, a “banking organization” within the meaning of the New York Banking Law, a 
member of the Federal Reserve System, a “clearing corporation” within the meaning of the New York 
Uniform Commercial Code, and a “clearing agency” registered under Section 17A of the Securities Exchange 
Act of 1934. DTC holds and provides asset servicing for over 3.5 million issues of U.S. and non-U.S. equity 
issues, corporate and municipal debt issues, and money market instruments (from over 100 countries) that 
DTC’s direct participants deposit with DTC. DTC also facilitates the post-trade settlement among direct 
participants of sales and other securities transactions in deposited securities, through electronic computerized 
book-entry transfers and pledges between direct participants’ accounts. This eliminates the need for physical 
movement of securities certificates. Direct participants include both U.S. and non-U.S. securities brokers and 
dealers, banks, trust companies, clearing corporations, and certain other organizations. DTC is a whollyowned subsidiary of The Depository Trust & Clearing Corporation (“DTCC”). DTCC is the holding company 
for DTC, National Securities Clearing Corporation and Fixed Income Clearing Corporation, all of which are 
registered clearing agencies. DTCC is owned by the users of its regulated subsidiaries. Access to the DTC 
system is also available to others such as both U.S. and non-U.S. securities brokers and dealers, banks, trust 
companies, and clearing corporations that clear through or maintain a custodial relationship with a direct 
participant, either directly or indirectly (“indirect participants”). The DTC rules applicable to its participants 
are on file with the SEC. More information about DTC can be found at www.dtcc.com. Information on that 
website is not included or incorporated by reference herein. 
Purchases of the securities under the DTC system must be made by or through direct participants, which 
will receive a credit for the securities on DTC’s records. The ownership interest of each actual purchaser of 
each security (“beneficial owner”) is in turn to be recorded on the direct and indirect participants’ records. 
Beneficial owners will not receive written confirmation from DTC of their purchase. Beneficial owners are, 
however, expected to receive written confirmations providing details of the transaction, as well as periodic 
statements of their holdings, from the direct or indirect participant through which the beneficial owner entered 
into the transaction. Transfers of ownership interests in the securities are to be accomplished by entries made 
on the books of direct and indirect participants acting on behalf of beneficial owners. Beneficial owners will 
not receive certificates representing their ownership interests in the securities, except in the event that use of 
the book-entry system for the securities is discontinued. 
To facilitate subsequent transfers, all securities deposited by direct participants with DTC are registered 
in the name of DTC’s partnership nominee, Cede & Co., or such other name as may be requested by an 
authorized representative of DTC. The deposit of securities with DTC and their registration in the name of 
Cede & Co. or such other DTC nominee do not effect any change in beneficial ownership. DTC has no 
knowledge of the actual beneficial owners of the securities; DTC’s records reflect only the identity of the 
direct participants to whose accounts such securities are credited, which may or may not be the beneficial 
owners. The direct and indirect participants will remain responsible for keeping account of their holdings on 
behalf of their customers. 
57 
Table of Contents
Conveyance of notices and other communications by DTC to direct participants, by direct participants to 
indirect participants, and by direct and indirect participants to beneficial owners will be governed by 
arrangements among them, subject to any statutory or regulatory requirements as may be in effect from time 
to time. Beneficial owners of securities may wish to take certain steps to augment the transmission to them of 
notices of significant events with respect to the securities, such as redemptions, tenders, defaults, and proposed 
amendments to the security documents. For example, beneficial owners of securities may wish to ascertain 
that the nominee holding the securities for its benefit has agreed to obtain and transmit notices to beneficial 
owners. In the alternative, beneficial owners may wish to provide their names and addresses to the registrar 
and request that copies of notices be provided directly to them. 
Neither DTC nor Cede & Co. (nor any other DTC nominee) will consent or vote with respect to 
securities unless authorized by a direct participant in accordance with DTC’s Money Market Instrument 
(“MMI”) procedures. Under its usual procedures, DTC mails an omnibus proxy to us as soon as possible after 
the record date. The omnibus proxy assigns Cede & Co.’s consenting or voting rights to those direct 
participants to whose accounts the securities are credited on the record date (identified in a listing attached to 
the omnibus proxy). 
We will make dividend payments or any payments of principal, any premium, interest, or other amounts 
on the securities in immediately available funds directly to Cede & Co., or any other nominee as may be 
requested by an authorized representative of DTC. DTC’s practice is to credit direct participants’ accounts 
upon DTC’s receipt of funds and corresponding detail information from us, on the applicable payment date in 
accordance with their respective holdings shown on DTC’s records. Payments by participants to beneficial 
owners will be governed by standing instructions and customary practices, as is the case with securities held 
for the accounts of customers in bearer form or registered in “street name,” and will be the responsibility of 
these participants and not of DTC or its nominee, us, the trustee, or any other agent or party, subject to any 
statutory or regulatory requirements that may be in effect from time to time. Payment of dividends or principal 
and any premium or interest to Cede & Co. (or any other nominee as may be requested by an authorized 
representative of DTC) is our responsibility. Disbursement of the payments to direct participants is the 
responsibility of DTC, and disbursement of the payments to the beneficial owners is the responsibility of the 
direct or indirect participants. 
We will send any redemption notices to DTC. If less than all of the securities of a series are being 
redeemed, DTC’s practice is to determine by lot the amount of the interest of each direct participant in the 
issue to be redeemed. 
A beneficial owner shall give notice to elect to have its securities repurchased through the participant 
through which it holds its beneficial interest in the security to the applicable trustee or tender agent. The 
beneficial owner shall effect delivery of its securities by causing the direct participant to transfer its interest in 
the securities on DTC’s records. The requirement for physical delivery of securities in connection with an 
optional tender or a mandatory purchase will be deemed satisfied when the ownership rights in the securities 
are transferred by the direct participant on DTC’s records and followed by a book-entry credit of tendered 
securities to the applicable trustee or agent’s DTC account. 
DTC may discontinue providing its services as depository for the securities at any time by giving us 
reasonable notice. If this occurs, and if a successor securities depository is not obtained, we will print and 
deliver certificated securities. 
We may decide to discontinue use of the system of book-entry only transfers through DTC (or a 
successor securities depository). In that event, we will print and delivery certificated securities to DTC. 
58 
Table of Contents
The information in this section concerning DTC and DTC’s book-entry system has been obtained from 
sources that we believe to be reliable, but we take no responsibility for its accuracy. 
Euroclear and Clearstream, Luxembourg 
Euroclear and Clearstream, Luxembourg each hold securities for their customers and facilitate the 
clearance and settlement of securities transactions by electronic book-entry transfer between their respective 
account holders (each such account holder, a “participant” and collectively, the “participants”). Euroclear and 
Clearstream, Luxembourg provide various services including safekeeping, administration, clearance and 
settlement of internationally traded securities and securities lending and borrowing. Euroclear and 
Clearstream, Luxembourg also deal with domestic securities markets in several countries through established 
depository and custodial relationships. Euroclear and Clearstream, Luxembourg have established an electronic 
bridge between their two systems across which their respective participants may settle trades with each other. 
Euroclear is incorporated under the laws of Belgium and Clearstream, Luxembourg is incorporated under the 
laws of Luxembourg. 
Euroclear and Clearstream, Luxembourg customers are world-wide financial institutions, including 
underwriters, securities brokers and dealers, banks, trust companies, and clearing corporations. Indirect access 
to Euroclear and Clearstream, Luxembourg is available to other institutions that clear through or maintain a 
custodial relationship with a participant of either system. 
The address of Euroclear is Euroclear Bank SA/NV, 1 Boulevard du Roi Albert II, B-1210 Brussels and 
the address of Clearstream, Luxembourg is Clearstream Banking, 42 Avenue JF Kennedy, L-1855, 
Luxembourg. 
Euroclear and Clearstream, Luxembourg may be depositories for a global security sold or traded outside 
the United States. In addition, if DTC is the depository for a global security, Euroclear and Clearstream, 
Luxembourg may hold interests in the global security as participants in DTC. As long as any global security is 
held by Euroclear or Clearstream, Luxembourg as depository, you may hold an interest in the global security 
only through an organization that participates, directly or indirectly, in Euroclear or Clearstream, 
Luxembourg. If Euroclear or Clearstream, Luxembourg is the depository for a global security and there is no 
depository in the United States, you will not be able to hold interests in that global security through any 
securities clearing system in the United States. 
Payments, deliveries, transfers, exchanges, notices, and other matters relating to the securities made 
through Euroclear or Clearstream, Luxembourg must comply with the rules and procedures of those clearing 
systems. Those clearing systems could change their rules and procedures at any time. We have no control over 
those clearing systems or their participants, and we take no responsibility for their activities. Transactions 
between participants in Euroclear or Clearstream, Luxembourg, on one hand, and participants in DTC, on the
other hand, when DTC is the depository, also would be subject to DTC’s rules and procedures. 
Investors will be able to make and receive through Euroclear and Clearstream, Luxembourg payments, 
deliveries, transfers, exchanges, notices, and other transactions involving any securities held through those 
clearing systems only on days when those clearing systems are open for business. Those clearing systems may 
not be open for business on days when banks, brokers, and other institutions are open for business in the 
United States. In addition, because of time-zone differences, U.S. investors who hold their interests in the 
securities through these clearing systems and wish to transfer their interests, or to receive or 
59 
Table of Contents
make a payment or delivery or exercise any other right with respect to their interests, on a particular day may 
find that the transaction will not be effected until the next business day in Brussels or Luxembourg, as 
applicable. Thus, investors who wish to exercise rights that expire on a particular day may need to act before 
the expiration date. In addition, investors who hold their interests through both DTC and Euroclear or 
Clearstream, Luxembourg may need to make special arrangements to finance any purchases or sales of their 
interests between the United States and European clearing systems, and those transactions may settle later than 
would be the case for transactions within one clearing system. 
Special Considerations for Global Securities 
As an indirect owner, an investor’s rights relating to a global security will be governed by the account rules of 
the depository and those of the investor’s financial institution or other intermediary through which it holds its 
interest (e.g., Euroclear or Clearstream, Luxembourg, if DTC is the depository), as well as general laws relating to 
securities transfers. We do not recognize this type of investor or any intermediary as a holder of securities. Instead, 
we deal only with the depository that holds the global security. 
If securities are issued only in the form of a global security, an investor should be aware of the following: 
• an investor cannot cause the securities to be registered in his or her own name, and cannot obtain physical 
certificates for his or her interest in the securities, except in the special situations described above; 
• an investor will be an indirect holder and must look to his or her own bank or broker for payments on the 
securities and protection of his or her legal rights relating to the securities, as we describe above under “—
Legal Holders”; 
• under existing industry practices, if we or the applicable trustee request any action of owners of beneficial 
interests in any global security or if an owner of a beneficial interest in any global security desires to give 
instructions or take any action that a holder of an interest in a global security is entitled to give or take under 
the applicable indenture, Euroclear or Clearstream, Luxembourg, as the case may be, would authorize the 
participants owning the relevant beneficial interests to give instructions or take such action, and such 
participants would authorize indirect holders to give or take such action or would otherwise act upon the 
instructions of such indirect holders; 
• an investor may not be able to sell interests in the securities to some insurance companies and other 
institutions that are required by law to own their securities in certificated form; 
• an investor may not be able to pledge his or her interest in a global security in circumstances where 
certificates representing the securities must be delivered to the lender or other beneficiary of the pledge in 
order for the pledge to be effective; furthermore, as Euroclear and Clearstream, Luxembourg act on behalf 
of their respective participants only, who in turn may act on behalf of their respective clients, the ability of 
beneficial owners who are not participants with Euroclear or Clearstream, Luxembourg to pledge interests in 
any global security to persons or entities that are not participants with Euroclear or Clearstream, 
Luxembourg or otherwise take action in respect of interests in any global security, may be limited; 
• the depository’s policies will govern payments, deliveries, transfers, exchanges, notices, and other matters 
relating to an investor’s interest in a global security, and those policies may change from time to time; 
60 
Table of Contents
• we, the trustee, any warrant agents, and any unit or other agents will not be responsible for any aspect of the 
depository’s policies, actions, or records of ownership interests in a global security; 
• we, the trustee, any warrant agents, and any unit or other agents do not supervise the depository in any way; 
• the depository will require that those who purchase and sell interests in a global security within its bookentry system use immediately available funds, and your broker or bank may require you to do so as well; 
and 
• financial institutions that participate in the depository’s book-entry system and through which an investor 
holds its interest in the global securities, directly or indirectly, also may have their own policies affecting 
payments, deliveries, transfers, exchanges, notices, and other matters relating to the securities. Those 
policies may change from time to time. For example, if you hold an interest in a global security through 
Euroclear or Clearstream, Luxembourg when DTC is the depository, Euroclear or Clearstream, 
Luxembourg, as applicable, will require those who purchase and sell interests in that security through them 
to use immediately available funds and comply with other policies and procedures, including deadlines for 
giving instructions as to transactions that are to be effected on a particular day. There may be more than one 
financial intermediary in the chain of ownership for an investor. We do not monitor and are not responsible 
for the policies or actions or records of ownership interests of any of those intermediaries. 
Registration, Transfer, and Payment of Certificated Securities 
If we ever issue securities in certificated form, those securities may be presented for registration of transfer at 
the office of the registrar or at the office of any transfer agent we designate and maintain. The registrar or transfer 
agent will make the transfer or registration only if it is satisfied with the documents of title and identity of the person 
making the request. There will not be a service charge for any exchange or registration of transfer of the securities, 
but we may require payment of a sum sufficient to cover any tax or other governmental charge that may be imposed 
in connection with the exchange. At any time we may change transfer agents or approve a change in the location 
through which any transfer agent acts. We also may designate additional transfer agents for any securities at any 
time. 
We will not be required to issue, exchange, or register the transfer of any security to be redeemed for a period 
of 15 calendar days before the selection of the securities to be redeemed. In addition, we will not be required to 
exchange or register the transfer of any security that was selected, called, or is being called for redemption, except 
the unredeemed portion of any security being redeemed in part. 
We will pay amounts payable on any certificated securities at the offices of the paying agents we may 
designate from time to time. 
61 
Table of Contents
U.S. FEDERAL INCOME TAX CONSIDERATIONS 
The following summary of the material U.S. federal income tax considerations of the acquisition, ownership, 
and disposition of certain of the debt securities, preferred stock, depositary shares representing fractional interests in 
preferred stock, and common stock that we are offering, is based upon the advice of Morrison & Foerster LLP, our 
tax counsel. The following discussion is not exhaustive of all possible tax considerations. This summary is based 
upon the Internal Revenue Code of 1986, as amended (the “Code”), regulations promulgated under the Code by the 
U.S. Treasury Department (“Treasury”) (including proposed and temporary regulations), rulings, current 
administrative interpretations and official pronouncements of the Internal Revenue Service (the “IRS”), and judicial 
decisions, all as currently in effect and all of which are subject to differing interpretations or to change, possibly 
with retroactive effect. No assurance can be given that the IRS would not assert, or that a court would not sustain, a 
position contrary to any of the tax consequences described below. 
This summary is for general information only, and does not purport to discuss all aspects of U.S. federal 
income taxation that may be important to a particular holder in light of its investment or tax circumstances or to 
holders subject to special tax rules, such as: partnerships, subchapter S corporations, or other pass-through entities, 
any government (or instrumentality or agency thereof), banks, financial institutions, tax-exempt entities, insurance 
companies, regulated investment companies, real estate investment trusts, trusts and estates, dealers in securities or 
currencies, traders in securities that have elected to use the mark-to-market method of accounting for their securities, 
persons holding the debt securities, preferred stock, depositary shares, or common stock as part of an integrated 
investment, including a “straddle,” “hedge,” “constructive sale,” or “conversion transaction,” persons (other than 
Non-U.S. Holders) whose functional currency for tax purposes is not the U.S. dollar, and persons subject to the 
alternative minimum tax provisions of the Code. This summary does not include any description of the tax laws of 
any state or local governments, or of any foreign government, that may be applicable to a particular holder. This 
summary also may not apply to all forms of debt securities, preferred stock, depositary shares, or common stock that 
we may issue. If the tax consequences associated with a particular form of debt security, preferred stock, depositary 
shares or common stock are different than those described below, they will be described in the applicable 
supplement. 
This summary is directed solely to holders that, except as otherwise specifically noted, will purchase the debt 
securities, preferred stock, depositary shares, or common stock offered in this prospectus upon original issuance and 
will hold such securities as capital assets within the meaning of Section 1221 of the Code, which generally means as 
property held for investment. 
You should consult your own tax advisor concerning the U.S. federal income tax consequences to you of 
acquiring, owning, and disposing of these securities, as well as any tax consequences arising under the laws of any 
state, local, foreign, or other tax jurisdiction and the possible effects of changes in U.S. federal or other tax laws. 
As used in this prospectus, the term “U.S. Holder” means a beneficial owner of the debt securities, preferred 
stock, depositary shares, or common stock offered in this prospectus that is for U.S. federal income tax purposes: 
• a citizen or resident of the United States; 
• a corporation (including an entity treated as a corporation for U.S. federal income tax purposes) created or 
organized in or under the laws of the United States or of any state of the United States or the District of 
Columbia; 
62 
Table of Contents
• an estate the income of which is subject to U.S. federal income taxation regardless of its source; or 
• any trust if a court within the United States is able to exercise primary supervision over the administration of 
the trust and one or more United States persons have the authority to control all substantial decisions of the 
trust. 
Notwithstanding the preceding paragraph, to the extent provided in Treasury regulations, some trusts in 
existence on August 20, 1996, and treated as United States persons prior to that date, that elect to continue to be 
treated as United States persons also will be U.S. Holders. As used in this prospectus, the term “Non-U.S. Holder” is 
a holder that is not a U.S. Holder. 
If an entity or arrangement treated as a partnership for U.S. federal income tax purposes holds the debt 
securities, preferred stock, depositary shares, or common stock offered in this prospectus, the U.S. federal income 
tax treatment of a partner generally will depend upon the status of the partner and the activities of the partnership 
and accordingly, this summary does not apply to partnerships. A partner of a partnership holding the debt securities, 
preferred stock, depositary shares, or common stock should consult its own tax advisor regarding the U.S. federal 
income tax consequences to the partner of the acquisition, ownership, and disposition by the partnership of the debt 
securities, preferred stock, depositary shares, or common stock. 
Taxation of Debt Securities 
This subsection describes the material U.S. federal income tax consequences of the acquisition, ownership, 
and disposition of the debt securities offered in this prospectus, other than the debt securities described below under 
“—Convertible, Renewable, Extendible, Indexed, and Other Debt Securities,” which will be described in the 
applicable supplement. This subsection is directed solely to holders that, except as otherwise specifically noted, will 
purchase the debt securities offered in this prospectus upon original issuance at the issue price, as defined below. 
Consequences to U.S. Holders 
The following is a summary of the material U.S. federal income tax consequences that will apply to U.S. 
Holders of debt securities. 
Payment of Interest. Except as described below in the case of interest on a debt security issued with original 
issue discount, as defined below under “—Consequences to U.S. Holders— Original Issue Discount,” interest on a 
debt security generally will be included in the income of a U.S. Holder as interest income at the time it is accrued or 
is received in accordance with the U.S. Holder’s regular method of accounting for U.S. federal income tax purposes 
and will be ordinary income. 
Original Issue Discount. Some of our debt securities may be issued with original issue discount (“OID”). U.S. 
Holders of debt securities issued with OID, other than short-term debt securities with a maturity of one year or less 
from its date of issue, will be subject to special tax accounting rules, as described in greater detail below. For tax 
purposes, OID is the excess of the “stated redemption price at maturity” of a debt instrument over its “issue price.” 
The “stated redemption price at maturity” of a debt security is the sum of all payments required to be made on the 
debt security other than “qualified stated interest” payments, as defined below. The “issue price” of a debt security is 
generally the first offering price to the public at which a substantial amount of the issue was sold (ignoring sales to 
bond houses, brokers, or similar persons or organizations acting in the capacity of underwriters, placement agents, or 
wholesalers). The term “qualified stated interest” 
63 
Table of Contents
generally means stated interest that is unconditionally payable in cash or property (other than debt instruments of the 
issuer), or that is treated as constructively received, at least annually at a single fixed rate or, under certain 
circumstances, at a variable rate. If a debt security bears interest during any accrual period at a rate below the rate 
applicable for the remaining term of the debt security (for example, debt securities with teaser rates or interest 
holidays), then some or all of the stated interest may not be treated as qualified stated interest. 
A U.S. Holder of a debt security with a maturity of more than one year from its date of issue that has been 
issued with OID (an “OID debt security”) is generally required to include any qualified stated interest payments in 
income as interest at the time such interest is accrued or is received in accordance with the U.S. Holder’s regular 
accounting method for tax purposes, as described above under “—Consequences to U.S. Holders—Payment of 
Interest.” A U.S. Holder of an OID debt security is generally required to include in income the sum of the daily 
accruals of the OID for the debt security for each day during the taxable year (or portion of the taxable year) in 
which the U.S. Holder held the OID debt security, regardless of such holder’s regular method of accounting. Thus, a 
U.S. Holder may be required to include OID in income in advance of the receipt of some or all of the related cash 
payments. The daily portion is determined by allocating the OID for each day of the accrual period. An accrual 
period may be of any length and the accrual periods may even vary in length over the term of the OID debt security, 
provided that each accrual period is no longer than one year and each scheduled payment of principal or interest 
occurs either on the first day of an accrual period or on the final day of an accrual period. The amount of OID 
allocable to an accrual period is equal to the excess of: (1) the product of the “adjusted issue price” of the OID debt 
security at the beginning of the accrual period and its yield to maturity (computed generally on a constant yield 
method and compounded at the end of each accrual period, taking into account the length of the particular accrual 
period) over (2) the amount of any qualified stated interest allocable to the accrual period. OID allocable to a final 
accrual period is the difference between the amount payable at maturity, other than a payment of qualified stated 
interest, and the adjusted issue price at the beginning of the final accrual period. Special rules will apply for 
calculating OID for an initial short accrual period. The “adjusted issue price” of an OID debt security at the 
beginning of any accrual period is the sum of the issue price of the OID debt security plus the amount of OID 
allocable to all prior accrual periods reduced by any payments received on the OID debt security that were not 
qualified stated interest. Under these rules, a U.S. Holder generally will have to include in income increasingly 
greater amounts of OID in successive accrual periods. 
If the excess of the “stated redemption price at maturity” of a debt security over its “issue price” is less than 
1/4 of 1% of the debt instrument’s stated redemption price at maturity multiplied by the number of complete years 
from its issue date to its maturity, or weighted average maturity in the case of debt securities with more than one
principal payment (“de minimis OID”), the debt security is not treated as issued with OID. A U.S. Holder generally 
must include the de minimis OID in income at the time payments, other than qualified stated interest, on the debt 
securities are made in proportion to the amount paid (unless the U.S. Holder makes the election described below 
under “—Consequences to U.S. Holders—Election to Treat All Interest as Original Issue Discount”). Any amount 
of de minimis OID that is included in income in this manner will be treated as capital gain. 
Additional rules applicable to debt securities with OID that are denominated in or determined by reference to a 
currency other than the U.S. dollar are described under “—Consequences to U.S. Holders—Non-U.S. Dollar 
Denominated Debt Securities” below. 
Variable Rate Debt Securities. In the case of a debt security that is a variable rate debt security, special rules 
apply. A debt security will qualify as a “variable rate debt instrument” under Treasury regulations if (i) the debt 
security’s issue price does not exceed the total noncontingent principal payments by more than the lesser of: (a) 
0.015 multiplied by the product of the total 
64 
Table of Contents
noncontingent principal payments and the number of complete years to maturity from the issue date, or (b) 15% of 
the total noncontingent principal payments; and (ii) the debt security provides for stated interest, compounded or 
paid at least annually, only at one or more qualified floating rates, a single fixed rate and one or more qualified 
floating rates, a single objective rate, or a single fixed rate and a single objective rate that is a qualified inverse 
floating rate. 
Generally, a rate is a qualified floating rate if: (i) (a) variations in the value of the rate can reasonably be 
expected to measure contemporaneous variations in the cost of newly borrowed funds in the currency in which the 
debt security is denominated; or (b) the rate is equal to such a rate multiplied by either a fixed multiple that is greater 
than 0.65 but not more than 1.35 or a fixed multiple greater than 0.65 but not more than 1.35 increased or decreased 
by a fixed rate, and (ii) the value of the rate on any date during the term of the debt security is set no earlier than 
three months prior to the first day on which that value is in effect and no later than one year following that first day. 
If a debt security provides for two or more qualified floating rates that are within 0.25 percentage points of each 
other on the issue date or can reasonably be expected to have approximately the same values throughout the term of 
the debt security, the qualified floating rates together constitute a single qualified floating rate. A debt security will 
not have a variable rate that is a qualified floating rate, however, if the variable rate of interest is subject to one or 
more minimum or maximum rate floors or ceilings or one or more governors limiting the amount of increase or 
decrease unless such floor, ceiling, or governor is fixed throughout the term of the debt security or is not reasonably 
expected as of the issue date to significantly affect the yield on the debt security. 
Generally, an objective rate is a rate that is (i) not a qualified floating rate, (ii) is determined using a single 
fixed formula that is based on objective financial or economic information that is not within the control of the issuer 
or a related party, and (iii) the value of the rate on any date during the term of the debt security is set no earlier than 
three months prior to the first day on which that value is in effect and no later than one year following that first day. 
If it is reasonably expected that the average value of the variable rate during the first half of the term of a debt 
security will be either significantly less than or significantly greater than the average value of the rate during the 
final half of the term of the debt security, then the debt security will not have a variable rate that is an objective rate. 
An objective rate is a qualified inverse floating rate if that rate is equal to a fixed rate minus a qualified floating rate 
and variations in the rate can reasonably be expected to inversely reflect contemporaneous variations in the qualified 
floating rate. 
A debt security will also have a variable rate that is a single qualified floating rate or an objective rate if 
interest on the debt security is stated at a fixed rate for an initial period of one year or less followed by either a 
qualified floating rate or an objective rate for a subsequent period, and either: (i) the fixed rate and the qualified 
floating rate or objective rate have values on the issue date of the debt security that do not differ by more than 0.25 
percentage points, or (ii) the value of the qualified floating rate or objective rate is intended to approximate the fixed 
rate. 
In the case of a debt security that provides for stated interest that is unconditionally payable at least annually at 
a variable rate that is a single qualified floating rate or objective rate, or one of those rates after a single fixed rate 
for an initial period, all stated interest on the debt security is treated as qualified stated interest. In that case, both the 
debt security’s yield to maturity and qualified stated interest will be determined, solely for purposes of calculating 
the accrual of OID, if any, as though the debt security will bear interest in all periods throughout its term (in the case 
of a single qualified floating rate or qualified inverse floating rate) at a fixed rate generally equal to the value of the 
rate on the issue date or, in the case of an objective rate (other than a qualified inverse floating rate), the rate that 
reflects the yield to maturity that is reasonably expected for the debt security (the “fixed rate substitute”). A U.S. 
holder should then recognize OID, if any, that is 
65 
Table of Contents
calculated based on the debt security’s assumed yield to maturity. If the interest actually accrued or paid during an 
accrual period exceeds or is less than the assumed fixed interest, the qualified stated interest allocable to that period 
is increased or decreased, as applicable. 
If a debt security does not provide for stated interest at a single qualified floating rate or a single objective 
rate, and also does not provide for interest payable at a fixed rate other than a single fixed rate for an initial period, 
the interest and OID accruals on the debt security must be determined by (i) determining a fixed rate substitute for 
each variable rate provided under the debt security (as described above), (ii) constructing the equivalent fixed rate 
debt instrument, using the fixed rate substitutes, (iii) determining the amount of qualified stated interest and OID 
with respect to the equivalent fixed rate debt instrument, and (iv) making appropriate adjustments to qualified stated 
interest or OID for actual variable rates during the applicable accrual period. 
In the case of a debt security that provides for stated interest either at one or more qualified floating rates or at 
a qualified inverse floating rate and also provides for stated interest at a single fixed rate other than at a single fixed 
rate for an initial period (as described above), the interest and OID accruals on the debt security must be determined 
by using the method described above. However, the debt security will be treated, for purposes of the first three steps 
of the determination, as if the debt security had provided for a qualified floating rate, or a qualified inverse floating 
rate, rather than the fixed rate. The qualified floating rate, or qualified inverse floating rate, that replaces the fixed 
rate must be such that the fair market value of the debt security as of the issue date approximates the fair market 
value of an otherwise identical debt instrument that provides for the qualified floating rate, or qualified inverse 
floating rate, rather than the fixed rate. 
Acquisition Premium. If a U.S. Holder purchases an OID debt security for an amount greater than its adjusted 
issue price (as determined above) at the purchase date and less than or equal to the sum of all amounts, other than 
qualified stated interest, payable on the OID debt security after the purchase date, the excess is “acquisition 
premium.” Under these rules, in general, the amount of OID which must be included in income for the debt security 
for any taxable year (or any portion of a taxable year in which the debt security is held) will be reduced (but not 
below zero) by the portion of the acquisition premium allocated to the period. The amount of acquisition premium 
allocated to each period is determined by multiplying the OID that otherwise would have been included in income 
by a fraction, the numerator of which is the excess of the cost over the adjusted issue price of the OID debt security 
and the denominator of which is the excess of the OID debt security’s stated redemption price at maturity over its 
adjusted issue price. 
If a U.S. Holder purchases an OID debt security for an amount less than its adjusted issue price (as determined 
above) at the purchase date, any OID accruing with respect to that OID debt security will be required to be included 
in income and, to the extent of the difference between the purchase amount and the OID debt security’s adjusted 
issue price, the OID debt security will be treated as having “market discount.” See “—Consequences to U.S. 
Holders—Market Discount” below. 
Amortizable Bond Premium. If a U.S. Holder purchases a debt security (including an OID debt security) for an 
amount in excess of the sum of all amounts payable on the debt security after the purchase date, other than qualified 
stated interest, such holder will be considered to have purchased such debt security with “amortizable bond 
premium” equal in amount to such excess. A U.S. Holder may elect to amortize such premium as an offset to 
interest income using a constant yield method over the remaining term of the debt security based on the U.S. 
Holder’s yield to maturity with respect to the debt security. 
66 
Table of Contents
A U.S. Holder generally may use the amortizable bond premium allocable to an accrual period to offset 
interest required to be included in the U.S. Holder’s income under its regular method of accounting with respect to 
the debt security in that accrual period. If the amortizable bond premium allocable to an accrual period exceeds the 
amount of interest allocable to such accrual period, such excess would be allowed as a deduction for such accrual 
period, but only to the extent of the U.S. Holder’s prior interest inclusions on the debt security that have not been 
offset previously by bond premium. Any excess is generally carried forward and allocable to the next accrual period. 
If a debt security may be redeemed by us prior to its maturity date, the amount of amortizable bond premium 
will be based on the amount payable at the applicable redemption date, but only if use of the redemption date (in lieu 
of the stated maturity date) results in a smaller amortizable bond premium for the period ending on the redemption 
date. In addition, special rules limit the amortization of bond premium in the case of convertible debt securities. 
An election to amortize bond premium applies to all taxable debt obligations held by the U.S. Holder at the 
beginning of the first taxable year to which the election applies and thereafter acquired by the U.S. Holder and may 
be revoked only with the consent of the IRS. Generally, a holder may make an election to include in income its 
entire return on a debt security (i.e., the excess of all remaining payments to be received on the debt security over 
the amount paid for the debt security by such holder) in accordance with a constant yield method based on the 
compounding of interest, as discussed below under “—Consequences to U.S. Holders—Election to Treat All Interest 
as Original Issue Discount.” If a holder makes such an election for a debt security with amortizable bond premium, 
such election will result in a deemed election to amortize bond premium for all of the holder’s debt instruments with 
amortizable bond premium and may be revoked only with the permission of the IRS. 
A U.S. Holder that elects to amortize bond premium will be required to reduce its tax basis in the debt security 
by the amount of the premium amortized during its holding period. OID debt securities purchased at a premium will 
not be subject to the OID rules described above. If a U.S. Holder does not elect to amortize bond premium, the 
amount of bond premium will be included in its tax basis in the debt security. Therefore, if a U.S. Holder does not 
elect to amortize bond premium and it holds the debt security to maturity, the premium generally will be treated as 
capital loss when the debt security matures. 
Market Discount. If a U.S. Holder purchases a debt security for an amount that is less than its stated 
redemption price at maturity, or, in the case of an OID debt security, its adjusted issue price, such holder will be 
considered to have purchased the debt security with “market discount.” Any payment, other than qualified stated 
interest, or any gain on the sale, exchange, retirement, or other disposition of a debt security with market discount 
generally will be treated as ordinary interest income to the extent of the market discount not previously included in 
income that accrued on the debt security during such holder’s holding period. In general, market discount is treated 
as accruing on a straight-line basis over the term of the debt security unless an election is made to accrue the market 
discount under a constant yield method. In addition, a U.S. Holder may be required to defer, until the maturity of the 
debt security or its earlier disposition in a taxable transaction, the deduction of a portion of the interest paid on any 
indebtedness incurred or maintained to purchase or carry the debt security in an amount not exceeding the accrued 
market discount on the debt security. 
A U.S. Holder may elect to include market discount in income currently as it accrues (on either a straight-line 
or constant yield basis), in lieu of treating a portion of any gain realized on a sale, exchange, retirement, or other 
disposition of the debt security as ordinary income. If an election is made to include market discount on a current 
basis, the interest deduction deferral rule described 
67 
Table of Contents
above will not apply. If a U.S. Holder makes such an election, it will apply to all market discount debt instruments 
acquired by such holder on or after the first day of the first taxable year to which the election applies. The election 
may not be revoked without the consent of the IRS. U.S. Holders should consult with their own tax advisors before 
making this election. 
If the difference between the stated redemption price at maturity of a debt security or, in the case of an OID 
debt security, its adjusted issue price, and the amount paid for the debt security is less than 1/4 of 1% of the debt 
instrument’s stated redemption price at maturity or, in the case of an OID debt security, its adjusted issue price, 
multiplied by the number of remaining complete years to the debt security’s maturity (“de minimis market 
discount”), the debt security is not treated as issued with market discount. 
Generally, a holder may make an election to include in income its entire return on a debt security (i.e., the 
excess of all remaining payments to be received on the debt security over the amount paid for the debt security by 
such holder) in accordance with a constant yield method based on the compounding of interest, as discussed below 
under “—Consequences to U.S. Holders—Election to Treat All Interest as Original Issue Discount.” If a holder 
makes such an election for a debt security with market discount, the holder will be required to include market 
discount in income currently as it accrues on a constant yield basis for all market discount debt instruments acquired 
by such holder on or after the first day of the first taxable year to which the election applies, and such election may 
be revoked only with the permission of the IRS. 
Election to Treat All Interest as Original Issue Discount. A U.S. Holder may elect to include in income all 
interest that accrues on a debt security using the constant-yield method applicable to OID described above, subject 
to certain limitations and exceptions. For purposes of this election, interest includes stated interest, acquisition 
discount, OID, de minimis OID, market discount, de minimis market discount, and unstated interest, as adjusted by 
any amortizable bond premium or acquisition premium, each as described herein. If this election is made for a debt 
security, then, to apply the constant-yield method: (i) the issue price of the debt security will equal its cost, (ii) the 
issue date of the debt security will be the date it was acquired, and (iii) no payments on the debt security will be 
treated as payments of qualified stated interest. A U.S. Holder must make this election for the taxable year in which 
the debt security was acquired, and may not revoke the election without the consent of the IRS. U.S. Holders should 
consult with their own tax advisors before making this election. 
Debt Securities That Trade “Flat.” We expect that certain debt securities will trade in the secondary market 
with accrued interest. However, we may issue debt securities with terms and conditions that would make it likely 
that such debt securities would trade “flat” in the secondary market, which means that upon a sale of a debt security 
a U.S. Holder would not be paid an amount that reflects the accrued but unpaid interest with respect to such debt 
security. Nevertheless, for U.S. federal income tax purposes, a portion of the sales proceeds equal to the interest 
accrued with respect to such debt security from the last interest payment date to the sale date must be treated as 
interest income rather than as an amount realized upon the sale. Accordingly, a U.S. Holder that sells such a debt 
security between interest payment dates would be required to recognize interest income and, in certain 
circumstances, would recognize a capital loss (the deductibility of which is subject to limitations) on the sale of the 
debt security. Concurrently, a U.S. Holder that purchases such a debt security between interest payment dates would 
not be required to include in income that portion of any interest payment received that is attributable to interest that 
accrued prior to the purchase. Such payment is treated as a return of capital which reduces the U.S. Holder’s 
remaining cost basis in the debt security. However, interest that accrues after the purchase date is included in income 
in the year received or accrued (depending on the U.S. Holder’s accounting method). U.S. Holders that purchase 
such debt securities between interest 
68 
Table of Contents
payment dates should consult their own tax advisors concerning such holder’s adjusted tax basis in the debt security 
and whether such debt securities should be treated as having been purchased with market discount, as described 
above. 
Short-Term Debt Securities. Some of our debt securities may be issued with maturities of one year or less 
from the date of issue, which we refer to as short-term debt securities. Treasury regulations provide that no 
payments of interest on a short-term debt security are treated as qualified stated interest. Accordingly, in 
determining the amount of discount on a short-term debt security, all interest payments, including stated interest, are 
included in the short-term debt security’s stated redemption price at maturity. 
In general, individual and certain other U.S. Holders using the cash basis method of tax accounting are not 
required to include accrued discount on short-term debt securities in income currently unless they elect to do so, but 
they may be required to include any stated interest in income as the interest is received. However, a cash basis U.S. 
Holder will be required to treat any gain realized on a sale, exchange, or retirement of the short-term debt security as 
ordinary income to the extent such gain does not exceed the discount accrued with respect to the short-term debt 
security, which will be determined on a straight-line basis unless the holder makes an election to accrue the discount 
under the constant-yield method, through the date of sale or retirement. In addition, a cash basis U.S. Holder that 
does not elect to currently include accrued discount in income will not be allowed to deduct any of the interest paid 
or accrued on any indebtedness incurred or maintained to purchase or carry a short-term debt security (in an amount 
not exceeding the deferred income), but instead will be required to defer deductions for such interest until the 
deferred income is realized upon the maturity of the short-term debt security or its earlier disposition in a taxable 
transaction. Notwithstanding the foregoing, a cash-basis U.S. Holder of a short-term debt security may elect to 
include accrued discount in income on a current basis. If this election is made, the limitation on the deductibility of 
interest described above will not apply. 
A U.S. Holder using the accrual method of tax accounting and some cash basis holders (including banks, 
securities dealers, regulated investment companies, and certain trust funds) generally will be required to include 
accrued discount on a short-term debt security in income on a current basis, on either a straight-line basis or, at the 
election of the holder, under the constant-yield method based on daily compounding. 
Regardless of whether a U.S. Holder is a cash-basis or accrual-basis holder, the holder of a short-term debt 
security may elect to include accrued “acquisition discount” with respect to the short-term debt security in income 
on a current basis. Acquisition discount is the excess of the remaining redemption amount of the short-term debt 
security at the time of acquisition over the purchase price. Acquisition discount will be treated as accruing on a 
straight-line basis or, at the election of the holder, under a constant yield method based on daily compounding. If a 
U.S. Holder elects to include accrued acquisition discount in income, the rules for including OID will not apply. In 
addition, the market discount rules described above will not apply to short-term debt securities. 
Sale, Exchange, or Retirement of Debt Securities. Upon the sale, exchange, retirement, or other disposition of 
a debt security, a U.S. Holder will recognize gain or loss equal to the difference between the amount realized upon 
the sale, exchange, retirement, or other disposition (less an amount equal to any accrued interest not previously 
included in income if the debt security is disposed of between interest payment dates, which will be included in 
income as interest income for U.S. federal income tax purposes) and the U.S. Holder’s adjusted tax basis in the debt 
security. The amount realized by the U.S. Holder will include the amount of any cash and the fair market value of 
any other property received for the debt security. A U.S. Holder’s adjusted tax basis in a debt security generally will 
be the cost of the debt security to such U.S. Holder, increased by any OID, market discount, de minimis OID, de 
minimis market discount, or any discount with respect to a 
69 
Table of Contents
short-term debt security previously included in income with respect to the debt security, and decreased by the 
amount of any premium previously amortized to reduce interest on the debt security and the amount of any payment 
(other than a payment of qualified stated interest) received in respect of the debt security. 
Except as discussed above with respect to market discount, or as described below with respect to debt 
securities subject to contingencies and Non-U.S. Dollar Denominated Debt Securities, gain or loss realized on the 
sale, exchange, retirement, or other disposition of a debt security generally will be capital gain or loss and will be 
long-term capital gain or loss if the debt security has been held for more than one year. Net long-term capital gain 
recognized by an individual U.S. Holder is generally taxed at preferential rates. The ability of U.S. Holders to deduct 
capital losses is subject to limitations under the Code. 
Reopenings. Treasury regulations provide specific rules regarding whether additional debt instruments issued 
in a reopening will be considered part of the same issue, with the same issue price and yield to maturity, as the 
original debt instruments for U.S. federal income tax purposes. Except as provided otherwise in an applicable 
supplement, we expect that additional debt securities issued by us in any reopening will be issued such that they will 
be considered part of the original issuance to which they relate. 
Debt Securities Subject to Contingencies. Certain of the debt securities may provide for an alternative 
payment schedule or schedules applicable upon the occurrence of a contingency or contingencies, other than a 
remote or incidental contingency, whether such contingency relates to payments of interest or of principal. In 
addition, certain of the debt securities may contain provisions permitting them to be redeemed prior to their stated 
maturity at our option and/or at the option of the holder. Debt securities containing these features may be 
characterized as “contingent payment debt instruments” for U.S. federal income tax purposes. 
If the debt securities are properly characterized as contingent payment debt instruments for U.S. federal 
income tax purposes, such debt securities generally will be subject to Treasury regulations governing contingent 
payment debt instruments. Under those regulations, a U.S. Holder will be required to report OID or interest income 
based on a “comparable yield” and a “projected payment schedule,” both as described below, established by us for 
determining interest accruals and adjustments with respect to a note. A U.S. Holder which does not use the 
“comparable yield” and follow the “projected payment schedule” to calculate its OID and interest income on a debt 
security must timely disclose and justify the use of other estimates to the IRS. 
A “comparable yield” with respect to a debt security generally is the yield at which we could issue a fixed-rate 
debt instrument with terms similar to those of the debt security (taking into account for this purpose the level of 
subordination, term, timing of payments, and general market conditions, but ignoring any adjustments for liquidity 
or the riskiness of the contingencies with respect to the debt security). Notwithstanding the foregoing, a comparable 
yield must not be less than the applicable U.S. federal rate based on the overall maturity of the debt security. 
A “projected payment schedule” with respect to a debt security generally is a series of projected payments, the 
amount and timing of which would produce a yield to maturity on that debt security equal to the comparable yield. 
This projected payment schedule will consist of a projection for tax purposes of each non-contingent and contingent 
payment. 
Based on the comparable yield and the projected payment schedule of the debt securities, a U.S. Holder of a 
note (regardless of accounting method) generally will be required to accrue as OID the sum of the daily portions of 
interest on the debt security for each day in the taxable year on which the holder held the debt security, adjusted 
upward or downward to reflect the difference, if 
70 
Table of Contents
any, between the actual and projected amount of any contingent payments on the debt security, as set forth below. 
The daily portions of interest for a debt security are determined by allocating to each day in an accrual period the 
ratable portion of interest on the debt security that accrues in the accrual period. The amount of interest on the debt 
security that accrues in an accrual period is the product of the comparable yield on the debt security (adjusted to 
reflect the length of the accrual period) and the adjusted issue price of the debt security at the beginning of the 
accrual period. The adjusted issue price of a debt security at the beginning of the first accrual period will equal its 
issue price (as described above). For any subsequent accrual period, the adjusted issue price will be (i) the sum of 
the issue price of the debt security and any interest previously accrued on the debt security by a holder (without 
regard to any positive or negative adjustments, described below) minus (ii) the amount of any projected payments on 
the debt security for previous accrual periods. 
A U.S. Holder of a debt security generally will be required to include in income OID in excess of actual cash 
payments received for certain taxable years. A U.S. Holder will be required to recognize interest income equal to the 
amount of any positive adjustment for a debt security for the taxable year in which a contingent payment is paid 
(including a payment of interest at maturity). A positive adjustment is the excess of actual payments in respect of 
contingent payments over the projected amount of contingent payments. A U.S. Holder also will be required to 
account for any “negative adjustment” for a taxable year in which a contingent payment is paid. A negative 
adjustment is the excess of the projected amounts of contingent payments over actual payments in respect of the 
contingent payments. A net negative adjustment is the amount by which total negative adjustments in a taxable year 
exceed total positive adjustments in such taxable year. A net negative adjustment (i) will first reduce the amount of 
interest for the debt security that a U.S. Holder would otherwise be required to include in income in the taxable year, 
and (ii) to the extent of any excess, will result in an ordinary loss equal to that portion of the excess as does not 
exceed the excess of (a) the amount of all previous interest inclusions under the debt security over (b) the total 
amount of the U.S. Holder’s net negative adjustments treated as ordinary loss on the note in prior taxable years. A 
net negative adjustment is not subject to the 2% floor limitation imposed on miscellaneous deductions under Section 
67 of the Code. Any net negative adjustment in excess of the amounts described above in (i) and (ii) will be carried 
forward to offset future interest income on the debt security or to reduce the amount realized on a sale, exchange, 
retirement or other disposition of the debt security and, in the case of a payment at maturity, should result in a 
capital loss. The deductibility of capital losses by a U.S. Holder is subject to limitations. 
If a contingent payment becomes fixed (within the meaning of applicable Treasury regulations) more than six 
months before its due date, a positive or negative adjustment, as appropriate, is made to reflect the difference 
between the present value of the amount that is fixed and the present value of the projected amount. The present 
value of each amount is determined by discounting the amount from the date the payment is due to the date the 
payment becomes fixed, using a discount rate equal to the comparable yield. If all contingent payments on the debt 
security become fixed, substantially contemporaneously, applicable Treasury regulations provide that, with regard to 
contingent payments that become fixed on a day that is more than six months before their due date, U.S. Holders 
should take into account positive or negative adjustments in respect of such contingent payments over the period to 
which they relate in a reasonable manner. U.S. Holders should consult their tax advisors as to what would be a 
“reasonable manner” in their particular situation. 
We expect that the applicable pricing supplement will include a table that sets forth the following information 
with respect to the principal amount of the debt securities for each of the applicable accrual periods through the 
maturity date of the debt securities: (i) the amount of interest deemed to have accrued during the accrual period, and 
(ii) the total amount of interest deemed to have accrued from the original issue date through the end of the accrual 
period. The 
71 
Table of Contents
table will be based upon a projected payment schedule and a comparable yield. The comparable yield will be 
determined based upon market conditions as of the date of the applicable pricing supplement. The comparable yield 
is likely to change between the date of any preliminary pricing supplement and the date of the related final pricing 
supplement. Therefore, the projected payment schedule included in any preliminary pricing supplement will be 
subject to change. We will determine the actual projected payment schedule and the actual comparable yield on the 
pricing date. Any tax accrual table included in a preliminary pricing supplement will be revised, and the revised 
table will be set forth in the final pricing supplement prepared in connection with the initial sale of the debt 
securities. 
Upon a sale, exchange, retirement, or other disposition of a debt security prior to maturity, a U.S. Holder 
generally will recognize taxable gain or loss equal to the difference between the amount realized on the sale, 
exchange, retirement, or other disposition and that holder’s tax basis in the debt security. A U.S. Holder’s tax basis 
in a debt security generally will equal the cost of that debt security, increased by the amount of OID previously 
accrued by the holder for that debt security (without regard to any positive or negative adjustments) and reduced by 
any projected payments for previous periods on the debt securities. A U.S. Holder generally will treat any gain as 
interest income, and will treat any loss as ordinary loss to the extent of the excess of previous interest inclusions 
over the total negative adjustments previously taken into account as ordinary losses, and the balance as long-term or 
short-term capital loss depending upon the U.S. Holder’s holding period for the debt security. The deductibility of 
capital losses by a U.S. Holder is subject to limitations. 
U.S. Holders considering the purchase of debt securities with these features should carefully examine the 
applicable pricing supplement and should consult their own tax advisors regarding the U.S. federal income tax 
consequences to a U.S. Holder of the purchase, ownership and disposition of such debt securities. 
Non-U.S. Dollar Denominated Debt Securities. Additional considerations apply to a U.S. Holder of a debt 
security payable in a currency other than U.S. dollars (“foreign currency”). 
We refer to these securities as Non-U.S. Dollar Denominated Debt Securities. In the case of payments of 
interest, U.S. Holders using the cash method of accounting for U.S. federal income tax purposes will be required to 
include in income the U.S. dollar value of the foreign currency payment on a Non-U.S. Dollar Denominated Debt 
Security (other than OID or market discount) when the payment of interest is received. The U.S. dollar value of the 
foreign currency payment is determined by translating the foreign currency received at the spot rate for such foreign 
currency on the date the payment is received, regardless of whether the payment is in fact converted to U.S. dollars 
at that time. The U.S. dollar value will be the U.S. Holder’s tax basis in the foreign currency received. A U.S. 
Holder will not recognize foreign currency exchange gain or loss with respect to the receipt of such payment. 
U.S. Holders using the accrual method of accounting for U.S. federal income tax purposes will be required to 
include in income the U.S. dollar value of the amount of interest income that has accrued and is otherwise required 
to be taken into account with respect to a Non-U.S. Dollar Denominated Debt Security during an accrual period. The 
U.S. dollar value of the accrued income will be determined by translating the income at the average rate of exchange 
for the accrual period or, with respect to an accrual period that spans two taxable years, at the average rate for the 
partial period within the taxable year. A U.S. Holder may elect, however, to translate the accrued interest income 
using the exchange rate on the last day of the accrual period or, with respect to an accrual period that spans two 
taxable years, using the exchange rate on the last day of the taxable year. If the last day of an accrual period is within 
five business days of the date of receipt of the 
72 
Table of Contents
accrued interest, a U.S. Holder may translate the interest using the exchange rate on the date of receipt. The above 
election will apply to all other debt obligations held by the U.S. Holder and may not be changed without the consent 
of the IRS. U.S. Holders should consult their own tax advisors before making the above election. Upon receipt of an 
interest payment (including, upon the sale of the debt security, the receipt of proceeds which include amounts 
attributable to accrued interest previously included in income), the holder will recognize foreign currency exchange 
gain or loss in an amount equal to the difference between the U.S. dollar value of such payment (determined by 
translating the foreign currency received at the spot rate for such foreign currency on the date such payment is 
received) and the U.S. dollar value of the interest income previously included in income with respect to such 
payment. This gain or loss will be treated as ordinary income or loss. 
OID on a debt security that is also a Non-U.S. Dollar Denominated Debt Security will be determined for any 
accrual period in the applicable foreign currency and then translated into U.S. dollars, in the same manner as interest 
income accrued by a holder on the accrual basis, as described above (regardless of such holder’s regular method of 
accounting). A U.S. Holder will recognize foreign currency exchange gain or loss when OID is paid (including, 
upon the sale of such debt security, the receipt of proceeds which include amounts attributable to OID previously 
included in income) to the extent of the difference between the U.S. dollar value of such payment (determined by 
translating the foreign currency received at the spot rate for such foreign currency on the date such payment is 
received) and the U.S. dollar value of the accrued OID (determined in the same manner as for accrued interest). For 
these purposes, all receipts on a debt security will be viewed: (i) first, as the receipt of any stated interest payment 
called for under the terms of the debt security, (ii) second, as receipts of previously accrued OID (to the extent 
thereof), with payments considered made for the earliest accrual periods first, and (iii) third, as the receipt of 
principal. 
The amount of market discount on Non-U.S. Dollar Denominated Debt Securities includible in income 
generally will be determined by translating the market discount determined in the foreign currency into U.S. dollars 
at the spot rate on the date the Non-U.S. Dollar Denominated Debt Security is retired or otherwise disposed of. If a 
U.S. Holder elected to accrue market discount currently, then the amount which accrues is determined in the foreign 
currency and then translated into U.S. dollars on the basis of the average exchange rate in effect during such accrual 
period. A U.S. Holder will recognize foreign currency exchange gain or loss with respect to market discount which 
is accrued currently using the approach applicable to the accrual of interest income as described above. 
Amortizable bond premium on a Non-U.S. Dollar Denominated Debt Security will be computed in the 
applicable foreign currency. If a U.S. Holder elected to amortize the premium, the amortizable bond premium will 
reduce interest income in the applicable foreign currency. At the time bond premium is amortized, foreign currency 
exchange gain or loss will be realized based on the difference between spot rates at such time and the time of 
acquisition of the Non-U.S. Dollar Denominated Debt Security. If a U.S. Holder does not elect to amortize bond 
premium, the bond premium computed in the foreign currency must be translated into U.S. dollars at the spot rate on 
the maturity date and such bond premium will constitute a capital loss which may be offset or eliminated by foreign 
currency exchange gain. 
If a U.S. Holder purchases a Non-U.S. Dollar Denominated Debt Security with previously owned foreign 
currency, foreign currency exchange gain or loss (which will be treated as ordinary income or loss) will be 
recognized in an amount equal to the difference, if any, between the tax basis in the foreign currency and the U.S. 
dollar fair market value of the foreign currency used to purchase the Non-U.S. Dollar Denominated Debt Security, 
determined on the date of purchase. 
Upon the sale, exchange, retirement, or other taxable disposition of a Non-U.S. Dollar Denominated Debt 
Security, a U.S. Holder will recognize gain or loss equal to the difference 
73 
Table of Contents
between the amount realized upon the sale, exchange, retirement, or other disposition (less an amount equal to any 
accrued and unpaid interest not previously included in income, which will be treated as a payment of interest for 
U.S. federal income tax purposes) and the adjusted tax basis in the Non-U.S. Dollar Denominated Debt Security. 
The adjusted tax basis in a Non-U.S. Dollar Denominated Debt Security will equal the amount paid for the Non-U.S. 
Dollar Denominated Debt Security, increased by the amounts of any market discount or OID previously included in 
income with respect to the Non-U.S. Dollar Denominated Debt Security and reduced by any amortized acquisition 
or other premium and any principal payments received in respect of the Non-U.S. Dollar Denominated Debt 
Security. The amount of any payment in or adjustments measured by foreign currency will be equal to the U.S. 
dollar value of the foreign currency on the date of the purchase or adjustment. The amount realized will be based on 
the U.S. dollar value of the foreign currency on the date the payment is received or the Non-U.S. Dollar 
Denominated Debt Security is disposed of (or deemed disposed of as a result of a material change in the terms of the 
debt security). If, however, a Non-U.S. Dollar Denominated Debt Security is traded on an established securities 
market and the U.S. Holder uses the cash basis method of tax accounting, the U.S. dollar value of the amount 
realized will be determined by translating the foreign currency payment at the spot rate of exchange on the 
settlement date of the purchase or sale. A U.S. Holder that uses the accrual basis method of tax accounting may elect 
the same treatment with respect to the purchase and sale of Non-U.S. Dollar Denominated Debt Securities traded on 
an established securities market, provided that the election is applied consistently. 
Except with respect to market discount as discussed above, and the foreign currency rules discussed below, 
gain or loss recognized upon the sale, exchange, retirement, or other taxable disposition of a Non-U.S. Dollar 
Denominated Debt Security will be capital gain or loss and will be long-term capital gain or loss if at the time of 
sale, exchange, retirement, or other disposition, the Non-U.S. Dollar Denominated Debt Security has been held for 
more than one year. Net long-term capital gain recognized by an individual U.S. Holder is generally taxed at 
preferential rates. The ability of U.S. Holders to deduct capital losses is subject to limitations under the Code. 
A portion of the gain or loss with respect to the principal amount of a Non-U.S. Dollar Denominated Debt 
Security may be treated as foreign currency exchange gain or loss. Foreign currency exchange gain or loss will be 
treated as ordinary income or loss. For these purposes, the principal amount of the Non-U.S. Dollar Denominated 
Debt Security is the purchase price for the Non-U.S. Dollar Denominated Debt Security calculated in the foreign 
currency on the date of purchase, and the amount of exchange gain or loss recognized is equal to the difference 
between (i) the U.S. dollar value of the principal amount determined on the date of the sale, exchange, retirement or 
other disposition of the Non-U.S. Dollar Denominated Debt Security and (ii) the U.S. dollar value of the principal 
amount determined on the date the Non-U.S. Dollar Denominated Debt Security was purchased. The amount of 
foreign currency exchange gain or loss will be limited to the amount of overall gain or loss realized on the 
disposition of the Non-U.S. Dollar Denominated Debt Security. 
The tax basis in foreign currency received as interest on a Non-U.S. Dollar Denominated Debt Security will be 
the U.S. dollar value of the foreign currency determined at the spot rate in effect on the date the foreign currency is 
received. The tax basis in foreign currency received on the sale, exchange, retirement, or other disposition of a NonU.S. Dollar Denominated Debt Security will be equal to the U.S. dollar value of the foreign currency, determined at 
the time of the sale, exchange, retirement or other disposition. As discussed above, if the Non-U.S. Dollar 
Denominated Debt Securities are traded on an established securities market, a cash basis U.S. Holder (or, upon 
election, an accrual basis U.S. Holder) will determine the U.S. dollar value of the foreign currency by translating the 
foreign currency received at the spot rate of exchange on the settlement date of the sale, exchange, retirement, or 
other disposition. Accordingly, in such case, no foreign currency 
74 
Table of Contents
exchange gain or loss will result from currency fluctuations between the trade date and settlement date of a sale, 
exchange, retirement, or other disposition. Any gain or loss recognized on a sale, exchange, retirement, or other 
disposition of foreign currency (including its exchange for U.S. dollars or its use to purchase debt securities) will be 
ordinary income or loss. 
Special rules may apply to Non-U.S. Dollar Denominated Debt Securities that are also treated as contingent 
payment debt instruments. For the special treatment, if any, of Non-U.S. Dollar Denominated Debt Securities that 
are also contingent payment debt securities, see the applicable supplement. 
Additional Medicare Tax on Unearned Income. With respect to taxable years beginning after December 31, 
2012, certain U.S. Holders, including individuals, estates and trusts, will be subject to an additional 3.8% Medicare 
tax on unearned income. For individual U.S. Holders, the additional Medicare tax applies to the lesser of (i) “net 
investment income” or (ii) the excess of “modified adjusted gross income” over $200,000 ($250,000 if married and 
filing jointly or $125,000 if married and filing separately). “Net investment income” generally equals the taxpayer’s 
gross investment income reduced by the deductions that are allocable to such income. Investment income generally 
includes passive income such as interest and capital gains. U.S. Holders are urged to consult their own tax advisors 
regarding the implications of the additional Medicare tax resulting from an investment in the debt securities. 
Consequences to Non-U.S. Holders 
The following is a summary of the material U.S. federal income tax consequences that will apply to Non-U.S. 
Holders of debt securities. Non-U.S. Holders should consult their own tax advisers regarding the U.S. and non-U.S. 
tax considerations of acquiring, holding, and disposing of debt securities. 
Payments of Interest. Under current U.S. federal income tax law and subject to the discussion below 
concerning backup withholding, principal (and premium, if any) and interest payments, including any OID, that are 
received from us or our agent and that are not effectively connected with the conduct by the Non-U.S. Holder of a 
trade or business within the United States, or a permanent establishment maintained in the United States if certain 
tax treaties apply, generally will not be subject to U.S. federal income or withholding tax except as provided below. 
Interest, including any OID, may be subject to a 30% withholding tax (or less under an applicable treaty, if any) if: 
• a Non-U.S. Holder actually or constructively owns 10% or more of the total combined voting power of all 
classes of our stock entitled to vote; 
• a Non-U.S. Holder is a “controlled foreign corporation” for U.S. federal income tax purposes that is related 
to us (directly or indirectly) through stock ownership; 
• a Non-U.S. Holder is a bank extending credit under a loan agreement in the ordinary course of its trade or 
business; 
• the interest payments on the debt security are determined by reference to the income, profits, changes in the 
value of property or other attributes of the debtor or a related party (other than payments that are based on 
the value of a security or index of securities that are, and will continue to be, actively traded within the 
meaning of Section 1092(d) of the Code, and that are not nor will be a “United States real property interest” 
as described in Section 897(c)(1) or 897(g) of the Code); or 
75 
Table of Contents
• the Non-U.S. Holder does not satisfy the certification requirements described below. 
A Non-U.S. Holder generally will satisfy the certification requirements if either: (A) the Non-U.S. Holder 
certifies to us or our agent, under penalties of perjury, that it is a non-United States person and provides its name and 
address (which certification may generally be made on an IRS Form W-8BEN, or a successor form), or (B) a 
securities clearing organization, bank, or other financial institution that holds customer securities in the ordinary 
course of its trade or business (a “financial institution”) and holds the debt security certifies to us or our agent under 
penalties of perjury that either it or another financial institution has received the required statement from the NonU.S. Holder certifying that it is a non-United States person and furnishes us with a copy of the statement. 
Payments not meeting the requirements set forth above and thus subject to withholding of U.S. federal income 
tax may nevertheless be exempt from withholding (or subject to withholding at a reduced rate) if the Non-U.S. 
Holder provides us with a properly executed IRS Form W-8BEN (or successor form) claiming an exemption from, 
or reduction in, withholding under the benefit of a tax treaty, or IRS Form W-8ECI (or other applicable form) stating 
that interest paid on the debt securities is not subject to withholding tax because it is effectively connected with the 
conduct of a trade or business within the United States as discussed below. To claim benefits under an income tax 
treaty, a Non-U.S. Holder must obtain a taxpayer identification number and certify as to its eligibility under the 
appropriate treaty’s limitations on benefits article. In addition, special rules may apply to claims for treaty benefits 
made by Non-U.S. Holders that are entities rather than individuals. A Non-U.S. Holder that is eligible for a reduced 
rate of U.S. federal withholding tax pursuant to an income tax treaty may obtain a refund of any excess amounts 
withheld by filing an appropriate claim for refund with the IRS. 
Additional Payments. If the amount or timing of any payments on a debt security is contingent, the interest 
payments on the debt security may be treated as “contingent interest” under Section 871(h)(4) of the Code, in which 
case such interest may not be eligible for the exemption from U.S. federal income and withholding tax, as described 
above (other than for a holder that otherwise claims an exemption from, or reduction in, withholding under the 
benefit of an income tax treaty). In certain circumstances, if specified in the applicable supplement, we will pay to a 
Non-U.S. Holder of any debt security additional amounts to ensure that every net payment on that debt security will 
not be less, due to the payment of U.S. federal withholding tax, than the amount then otherwise due and payable. See
“Description of Debt Securities— Payment of Additional Amounts” above. However, because the likelihood that 
such payments will be made is remote, we do not believe that, because of these potential additional payments, the 
interest on the debt securities should be treated as contingent interest. 
Sale, Exchange, or Retirement of Debt Securities. A Non-U.S. Holder generally will not be subject to U.S. 
federal income or withholding tax on any capital gain or market discount realized on the sale, exchange, retirement, 
or other disposition of debt securities, provided that: (a) the gain is not effectively connected with the conduct of a 
trade or business within the United States, or a permanent establishment maintained in the United States if certain 
tax treaties apply, (b) in the case of a Non-U.S. Holder that is an individual, the Non-U.S. Holder is not present in 
the United States for 183 days or more in the taxable year of the sale, exchange, or other disposition of the debt 
security, and (c) the Non-U.S. Holder is not subject to tax pursuant to certain provisions of U.S. federal income tax 
law applicable to certain expatriates. An individual Non-U.S. Holder who is present in the United States for 183 
days or more in the taxable year of sale, exchange, or other disposition of a debt security, and if certain other 
conditions are met, will be subject to U.S. federal income tax at a rate of 30% on the gain realized on the sale, 
exchange, or other disposition of such debt security. 
76 
Table of Contents
Income Effectively Connected with a Trade or Business within the United States. If a Non-U.S. Holder of a 
debt security is engaged in the conduct of a trade or business within the United States and if interest (including any 
OID) on the debt security, or gain realized on the sale, exchange, or other disposition of the debt security, is 
effectively connected with the conduct of such trade or business (and, if certain tax treaties apply, is attributable to a 
permanent establishment maintained by the Non-U.S. Holder in the United States), the Non-U.S. Holder, although 
exempt from U.S. federal withholding tax (provided that the certification requirements discussed above are 
satisfied), generally will be subject to U.S. federal income tax on such interest (including any OID) or gain on a net 
income basis in the same manner as if it were a U.S. Holder. Non-U.S. holders should read the material under the 
heading “—Consequences to U.S. Holders,” for a description of the U.S. federal income tax consequences of 
acquiring, owning, and disposing of debt securities. In addition, if such Non-U.S. Holder is a foreign corporation, it 
may also be subject to a branch profits tax equal to 30% (or such lower rate provided by an applicable U.S. income 
tax treaty) of a portion of its earnings and profits for the taxable year that are effectively connected with its conduct 
of a trade or business in the United States, subject to certain adjustments. 
Convertible, Renewable, Extendible, Indexed, and Other Debt Securities 
Special U.S. federal income tax rules are applicable to certain other debt securities, including contingent NonU.S. Dollar Denominated Debt Securities, debt securities that may be convertible into or exercisable or 
exchangeable for our common or preferred stock or other securities or debt or equity securities of one or more third 
parties, debt securities the payments on which are determined or partially determined by reference to any index and 
other debt securities that are subject to the rules governing contingent payment obligations which are not subject to 
the rules governing variable rate debt securities, any renewable and extendible debt securities and any debt securities 
providing for the periodic payment of principal over the life of the debt security. The material U.S. federal income 
tax considerations with respect to these debt securities will be discussed in the applicable supplement. 
Backup Withholding and Information Reporting 
In general, in the case of a U.S. Holder, other than certain exempt holders, we and other payors are required to 
report to the IRS all payments of principal, any premium, and interest on the debt security, and the accrual of OID 
on an OID debt security. In addition, we and other payors generally are required to report to the IRS any payment of 
proceeds of the sale of a debt security before maturity. Additionally, backup withholding generally will apply to any 
payments, including payments of OID, if a U.S. Holder fails to provide an accurate taxpayer identification number 
and certify that the taxpayer identification number is correct, the U.S. Holder is notified by the IRS that it has failed 
to report all interest and dividends required to be shown on its U.S. federal income tax returns or a U.S. Holder does 
not certify that it has not underreported its interest and dividend income. 
In the case of a Non-U.S. Holder, backup withholding and information reporting will not apply to payments 
made if the Non-U.S. Holder provides the required certification that it is not a United States person, or the Non-U.S. 
Holder otherwise establishes an exemption, provided that the payor or withholding agent does not have actual 
knowledge or reason to know that the holder is a United States person, or that the conditions of any exemption are 
not satisfied. However, we and other payors are required to report payments of interest on the debt securities on IRS 
Form 1042-S even if the payments are not otherwise subject to information reporting requirements. 
In addition, payments of the proceeds from the sale of a debt security to or through a foreign office of a broker 
or the foreign office of a custodian, nominee, or other dealer acting on behalf of a 
77 
Table of Contents
holder generally will not be subject to information reporting or backup withholding. However, if the broker, 
custodian, nominee, or other dealer is a United States person, the government of the United States or the government 
of any state or political subdivision of any state, or any agency or instrumentality of any of these governmental units, 
a controlled foreign corporation for U.S. federal income tax purposes, a foreign partnership that is either engaged in 
a trade or business within the United States or whose United States partners in the aggregate hold more than 50% of 
the income or capital interest in the partnership, a foreign person 50% or more of whose gross income for a certain 
period is effectively connected with a trade or business within the United States, or a United States branch of a 
foreign bank or insurance company, information reporting (but not backup withholding) generally will be required 
with respect to payments made to a holder unless the broker, custodian, nominee, or other dealer has documentation 
of the holder’s foreign status and the broker, custodian, nominee, or other dealer has no reason to know or actual 
knowledge to the contrary. 
Payment of the proceeds from a sale of a debt security to or through the United States office of a broker is 
subject to information reporting and backup withholding, unless the holder certifies as to its non-United States 
person status or otherwise establishes an exemption from information reporting and backup withholding. 
Any amounts withheld under the backup withholding rules will be allowed as a refund or a credit against a 
holder’s U.S. federal income tax liability provided the required information is furnished to the IRS. 
Taxation of Common Stock, Preferred Stock, and Depositary Shares 
This subsection describes the material U.S. federal income tax consequences of the acquisition, ownership and 
disposition of the common stock, preferred stock and depositary shares offered in this prospectus. 
Taxation of Holders of Depositary Shares 
For U.S. federal income tax purposes, holders of depositary shares generally will be treated as if they were the 
holders of the preferred stock represented by such depositary shares. Accordingly, such holders will be entitled to 
take into account, for U.S. federal income tax purposes, income, and deductions to which they would be entitled if 
they were holders of such preferred stock, as described more fully below. Exchanges of preferred stock for 
depositary shares and depositary shares for preferred stock generally will not be subject to U.S. federal income 
taxation. 
Consequences to U.S. Holders 
The following is a summary of the material U.S. federal income tax consequences that will apply to U.S. 
Holders of our common stock, preferred stock, and depositary shares. 
Distributions on Common Stock, Preferred Stock, and Depositary Shares. Distributions made to U.S. Holders 
out of our current or accumulated earnings and profits, as determined for U.S. federal income tax purposes, will be 
included in the income of a U.S. Holder as dividend income and will be subject to tax as ordinary income. Dividends 
received by an individual U.S. Holder in taxable years beginning before January 1, 2013 that constitute “qualified 
dividend income” are generally subject to tax at a maximum rate of 15% applicable to net long-term capital gains, 
provided that certain holding period and other requirements are met. Dividends received by a corporate U.S. Holder, 
except as described in the next subsection, generally will be eligible for the 70% dividends-received deduction. 
78 
Table of Contents
Distributions in excess of our current and accumulated earnings and profits will not be taxable to a U.S. 
Holder to the extent that the distributions do not exceed the U.S. Holder’s adjusted tax basis in the shares, but rather 
will reduce the adjusted tax basis of such shares. To the extent that distributions in excess of our current and 
accumulated earnings and profits exceed the U.S. Holder’s adjusted tax basis in the shares, such distributions will be 
included in income as capital gain. In addition, a corporate U.S. Holder will not be entitled to the dividends-received 
deduction on this portion of a distribution. 
We will notify holders of our shares after the close of our taxable year as to the portions of the distributions 
attributable to that year that constitute ordinary income, qualified dividend income and nondividend distributions, if 
any. 
Limitations on Dividends-Received Deduction. A corporate U.S. Holder may not be entitled to take the 70% 
dividends-received deduction in all circumstances. Prospective corporate investors in our common stock, preferred 
stock, or depositary shares should consider the effect of: 
• Section 246A of the Code, which reduces the dividends-received deduction allowed to a corporate U.S. 
Holder that has incurred indebtedness that is “directly attributable” to an investment in portfolio stock, 
which may include our common stock, preferred stock, and depositary shares; 
• Section 246(c) of the Code, which, among other things, disallows the dividends-received deduction in 
respect of any dividend on a share of stock that is held for less than the minimum holding period (generally, 
for common stock, at least 46 days during the 90 day period beginning on the date which is 45 days before 
the date on which such share becomes ex-dividend with respect to such dividend); and 
• Section 1059 of the Code, which, under certain circumstances, reduces the basis of stock for purposes of 
calculating gain or loss in a subsequent disposition by the portion of any “extraordinary dividend” (as 
defined below) that is eligible for the dividends-received deduction. 
Extraordinary Dividends. A corporate U.S. Holder will be required to reduce its tax basis (but not below zero) 
in our common stock, preferred stock, or depositary shares by the nontaxed portion of any “extraordinary dividend” 
if the stock was not held for more than two years before the earliest of the date such dividend is declared, 
announced, or agreed. Generally, the nontaxed portion of an extraordinary dividend is the amount excluded from 
income by operation of the dividends-received deduction. An extraordinary dividend generally would be a dividend 
that: 
• in the case of common stock, equals or exceeds 10% of the corporate U.S. Holder’s adjusted tax basis in the 
common stock, treating all dividends having ex-dividend dates within an 85 day period as one dividend; or 
• in the case of preferred stock, equals or exceeds 5% of the corporate U.S. Holder’s adjusted tax basis in the 
preferred stock, treating all dividends having ex-dividend dates within an 85 day period as one dividend; or 
• exceeds 20% of the corporate U.S. Holder’s adjusted tax basis in the stock, treating all dividends having exdividend dates within a 365 day period as one dividend. 
In determining whether a dividend paid on stock is an extraordinary dividend, a corporate U.S. Holder may 
elect to substitute the fair market value of the stock for its tax basis for purposes of applying these tests if the fair 
market value as of the day before the ex-dividend date is established to the satisfaction of the Secretary of the 
Treasury. An extraordinary dividend also includes any 
79 
Table of Contents
amount treated as a dividend in the case of a redemption that is either non-pro rata as to all stockholders or in partial 
liquidation of the corporation, regardless of the stockholder’s holding period and regardless of the size of the 
dividend. Any part of the nontaxed portion of an extraordinary dividend that is not applied to reduce the corporate 
U.S. Holder’s tax basis as a result of the limitation on reducing its basis below zero would be treated as capital gain 
and would be recognized in the taxable year in which the extraordinary dividend is received. 
Corporate U.S. Holders should consult with their own tax advisors with respect to the possible application of 
the extraordinary dividend provisions of the Code to the ownership or disposition of common stock, preferred stock, 
or depositary shares in their particular circumstances. 
Sale, Exchange, or other Taxable Disposition. Upon the sale, exchange, or other taxable disposition of our 
common stock, preferred stock, or depositary shares (other than by redemption or repurchase by us), a U.S. Holder 
generally will recognize gain or loss equal to the difference between the amount realized upon the sale, exchange, or 
other taxable disposition and the U.S. Holder’s adjusted tax basis in the shares. The amount realized by the U.S. 
Holder will include the amount of any cash and the fair market value of any other property received upon the sale, 
exchange, or other taxable disposition of the shares. A U.S. Holder’s tax basis in a share generally will be equal to 
the cost of the share to such U.S. Holder, which may be adjusted for certain subsequent events (for example, if the 
U.S. Holder receives a nondividend distribution, as described above). Gain or loss realized on the sale, exchange, or 
other taxable disposition of our common stock, preferred stock, or depositary shares generally will be capital gain or 
loss and will be long-term capital gain or loss if the shares have been held for more than one year. Net long-term 
capital gain recognized by an individual U.S. Holder is generally taxed at preferential rates. The ability of U.S. 
Holders to deduct capital losses is subject to limitations under the Code. 
Redemption or Repurchase of Common Stock, Preferred Stock, or Depositary Shares. If we are permitted to 
and redeem or repurchase a U.S. Holder’s common stock, preferred stock, or depositary shares, the redemption or 
repurchase generally would be a taxable event for U.S. federal income tax purposes. A U.S. Holder would be treated 
as if it had sold its shares if the redemption or repurchase: 
• results in a complete termination of the U.S. holder’s stock interest in us; 
• is substantially disproportionate with respect to the U.S. Holder; or 
• is not essentially equivalent to a dividend with respect to the U.S. Holder, in each case as determined under 
the Code. 
In determining whether any of these tests has been met, shares of stock considered to be owned by a U.S. 
Holder by reason of certain constructive ownership rules set forth in Section 318 of the Code, as well as shares 
actually owned, must be taken into account. 
If we redeem or repurchase a U.S. Holder’s shares in a redemption or repurchase that meets one of the tests 
described above, the U.S. Holder generally would recognize taxable gain or loss equal to the sum of the amount of 
cash and fair market value of property (other than our stock or the stock of a successor to us) received less the U.S. 
Holder’s tax basis in the shares redeemed or repurchased. This gain or loss generally would be long-term capital 
gain or capital loss if the shares have been held for more than one year. 
If a redemption or repurchase does not meet any of the tests described above, a U.S. Holder generally will be 
taxed on the cash and fair market value of the property received as a dividend to the extent paid out of our current 
and accumulated earnings and profits. Any amount in excess of 
80 
Table of Contents
our current or accumulated earnings and profits would first reduce the U.S. holder’s tax basis in the shares and 
thereafter would be treated as capital gain. If a redemption or repurchase is treated as a distribution that is taxable as 
a dividend, the U.S. Holder’s tax basis in the redeemed or repurchased shares would be transferred to the remaining 
shares of our stock that the U.S. Holder owns, if any. 
Special rules apply if we redeem our common stock, preferred stock, or depositary shares for our debt 
securities. We will discuss any special U.S. federal income tax considerations in the applicable supplement if we 
have the option to redeem our common stock, preferred stock, or depositary shares for our debt securities. 
Additional Medicare Tax on Unearned Income. With respect to taxable years beginning after December 31, 
2012, certain U.S. Holders, including individuals, estates and trusts, will be subject to an additional 3.8% Medicare 
tax on unearned income. For individual U.S. Holders, the additional Medicare tax applies to the lesser of (i) “net 
investment income” or (ii) the excess of “modified adjusted gross income” over $200,000 ($250,000 if married and 
filing jointly or $125,000 if married and filing separately). “Net investment income” generally equals the taxpayer’s 
gross investment income reduced by the deductions that are allocable to such income. Investment income generally 
includes passive income such as dividends and capital gains. U.S. Holders are urged to consult their own tax 
advisors regarding the implications of the additional Medicare tax resulting from an investment in the preferred 
stock, common stock, or depositary shares. 
Consequences to Non-U.S. Holders 
The following is a summary of the material U.S. federal income tax consequences that will apply to Non-U.S. 
Holders of our common stock, preferred stock, and depositary shares. 
Distributions on Common Stock, Preferred Stock, and Depositary Shares. Distributions made to Non-U.S. 
Holders out of our current or accumulated earnings and profits, as determined for U.S. federal income tax purposes, 
and that is not effectively connected with the conduct by the Non-U.S. Holder of a trade or business within the 
United States, or a permanent establishment maintained in the United States if certain tax treaties apply, generally 
will be subject to U.S. federal income and withholding tax at a rate of 30% (or lower rate under an applicable treaty, 
if any). Payments subject to withholding of U.S. federal income tax may nevertheless be exempt from withholding 
(or subject to withholding at a reduced rate) if the Non-U.S. Holder provides us with a properly executed IRS Form 
W-8BEN (or successor form) claiming an exemption from, or reduction in, withholding under the benefit of a tax 
treaty, or IRS Form W-8ECI (or other applicable form) stating that a dividend paid on our shares is not subject to 
withholding tax because it is effectively connected with the conduct of a trade or business within the United States, 
as discussed below. 
To claim benefits under an income tax treaty, a Non-U.S. Holder must certify to us or our agent, under 
penalties of perjury, that it is a non-United States person and provide its name and address (which certification may 
generally be made on an IRS Form W-8BEN, or a successor form), obtain and provide a taxpayer identification 
number, and certify as to its eligibility under the appropriate treaty’s limitations on benefits article. In addition, 
special rules may apply to claims for treaty benefits made by Non-U.S. Holders that are entities rather than 
individuals. A Non-U.S. Holder that is eligible for a reduced rate of U.S. federal withholding tax under an income 
tax treaty may obtain a refund of any excess amounts withheld by filing an appropriate claim for refund with the 
IRS. 
Sale, Exchange, or other Taxable Disposition. A Non-U.S. Holder generally will not be subject to U.S. federal 
income or withholding tax on any capital gain realized on the sale, exchange, or 
81 
Table of Contents
other taxable disposition of our common stock, preferred stock, or depositary shares, provided that: (a) the gain is 
not effectively connected with the conduct of a trade or business within the United States, or a permanent 
establishment maintained in the United States if certain tax treaties apply, (b) in the case of a Non-U.S. Holder that 
is an individual, the Non-U.S. Holder is not present in the United States for 183 days or more in the taxable year of 
the sale, exchange, or other disposition of the shares, (c) the Non-U.S. Holder is not subject to tax pursuant to certain 
provisions of U.S. federal income tax law applicable to certain expatriates, and (d) we are not nor have we been a 
“United States real property holding corporation” for U.S. federal income tax purposes. An individual Non-U.S. 
Holder who is present in the United States for 183 days or more in the taxable year of sale, exchange, or other 
disposition of our common stock, preferred stock, or depositary shares and if certain other conditions are met, will 
be subject to U.S. federal income tax at a rate of 30% on the gains realized on the sale, exchange, or other 
disposition of such shares. 
We would not be treated as a “United States real property holding corporation” if less than 50% of our assets 
throughout a prescribed testing period consist of interests in real property located within the United States, 
excluding, for this purpose, interests in real property solely in a capacity as a creditor. Even if we are treated as a 
“United States real property holding corporation,” a Non-U.S. Holder’s sale of our common stock, preferred stock, 
or depositary shares nonetheless generally will not be subject to U.S. federal income or withholding tax, provided 
that (a) our stock owned is of a class that is “regularly traded,” as defined by applicable Treasury regulations, on an 
established securities market, and (b) the selling Non-U.S. Holder held, actually or constructively, 5% or less of our 
outstanding stock of that class at all times during the five-year period ending on the date of disposition. 
To the extent we are or have been a “United States real property holding corporation” for U.S. federal income 
tax purposes and a Non-U.S. Holder held, directly or indirectly, at any time during the five-year period ending on 
the date of disposition, more than 5% of the class of stock and the non-U.S. Holder was not eligible for any treaty 
exemption, any gain on the sale of our common stock, preferred stock, or depositary shares would be treated as 
effectively connected with a trade or business within the United States, the treatment of which is described below, 
and the purchaser of the stock could be required to withhold 10% of the purchase price and remit such amount to the 
IRS. 
We believe that we are not currently, and do not anticipate becoming, a “United States real property holding 
corporation” for U.S. federal income tax purposes. 
Income Effectively Connected with a Trade or Business within the United States. If a Non-U.S. Holder of our 
common stock, preferred stock, or depositary shares is engaged in the conduct of a trade or business within the 
United States and if dividends on the shares, or gain realized on the sale, exchange, or other disposition of the 
shares, are effectively connected with the conduct of such trade or business (and, if certain tax treaties apply, is 
attributable to a permanent establishment maintained by the Non-U.S. Holder in the United States), the Non-U.S. 
Holder, although exempt from U.S. federal withholding tax (provided that the certification requirements discussed 
above are satisfied), generally will be subject to U.S. federal income tax on such dividends or gain on a net income 
basis in the same manner as if it were a U.S. Holder. Non-U.S. Holders should read the material under the heading 
“—Consequences to U.S. Holders” above for a description of the U.S. federal income tax consequences of 
acquiring, owning, and disposing of our common stock, preferred stock, or depositary shares. In addition, if such 
Non-U.S. Holder is a foreign corporation, it may also be subject to a branch profits tax equal to 30% (or such lower 
rate provided by an applicable U.S. income tax treaty) of a portion of its earnings and profits for the taxable year that 
are effectively connected with its conduct of a trade or business in the United States, subject to certain adjustments. 
82 
Table of Contents
Backup Withholding and Information Reporting 
In general, in the case of a U.S. Holder, other than certain exempt holders, we and other payors are required to 
report to the IRS all payments of dividends on our common stock, preferred stock, or depositary shares. In addition, 
we and other payors generally are required to report to the IRS any payment of proceeds of the sale of common 
stock, preferred stock, or depositary shares. Additionally, backup withholding generally will apply to any dividend 
payment and to proceeds received on a sale or exchange if a U.S. Holder fails to provide an accurate taxpayer 
identification number and certify that the taxpayer identification number is correct, the U.S. Holder is notified by the 
IRS that it has failed to report all dividends required to be shown on its U.S. federal income tax returns, or the U.S. 
Holder does not certify that it has not underreported its interest and dividend income. 
In the case of a Non-U.S. Holder, backup withholding and information reporting will not apply to payments 
made if the Non-U.S. Holder provides the required certification that it is not a United States person, as described 
above, or the Non-U.S. Holder otherwise establishes an exemption, provided that the payor or withholding agent 
does not have actual knowledge or reason to know that the holder is a United States person, or that the conditions of 
any exemption are not satisfied. 
In addition, payments of the proceeds from the sale of our common stock, preferred stock, or depositary shares 
to or through a foreign office of a broker or the foreign office of a custodian, nominee, or other dealer acting on 
behalf of a holder generally will not be subject to information reporting or backup withholding. However, if the 
broker, custodian, nominee, or other dealer is a United States person, the government of the United States or the 
government of any state or political subdivision of any state, or any agency or instrumentality of any of these 
governmental units, a controlled foreign corporation for U.S. federal income tax purposes, a foreign partnership that 
is either engaged in a trade or business within the United States or whose United States partners in the aggregate 
hold more than 50% of the income or capital interest in the partnership, a foreign person 50% or more of whose 
gross income for a certain period is effectively connected with a trade or business within the United States, or a 
United States branch of a foreign bank or insurance company, information reporting (but not backup withholding) 
generally will be required with respect to payments made to a holder unless the broker, custodian, nominee, or other 
dealer has documentation of the holder’s foreign status and the broker, custodian, nominee, or other dealer has no 
reason to know or actual knowledge to the contrary. 
Payment of the proceeds from a sale of our common stock, preferred stock, or depositary shares to or through 
the United States office of a broker is subject to information reporting and backup withholding, unless the holder 
certifies as to its non-United States person status or otherwise establishes an exemption from information reporting 
and backup withholding. 
Any amounts withheld under the backup withholding rules will be allowed as a refund or a credit against a 
holder’s U.S. federal income tax liability provided the required information is furnished to the IRS. 
Convertible Preferred Stock and Other Equity Securities 
Special U.S. federal income tax rules are applicable to certain other of our equity securities, including 
preferred stock convertible into or exercisable or exchangeable for our common stock or other securities. The 
material U.S. federal income tax considerations with respect to these securities will be discussed in the applicable 
pricing supplement. Investors should consult with their own tax advisors regarding the specific U.S. federal income 
tax considerations with respect to these securities. 
83 
Table of Contents
Taxation of Warrants 
The applicable supplement will contain a discussion of any special U.S. federal income tax considerations 
with respect to the acquisition, ownership and disposition of warrants offered in this prospectus, including any tax 
considerations relating to the specific terms of the warrants. Investors considering the purchase of warrants we are 
offering should carefully examine the applicable supplement regarding the special U.S. federal income tax 
considerations, if any, of the acquisition, ownership and disposition of the warrants. 
Investors should consult with their own tax advisors regarding the U.S. federal income tax consequences and 
the tax consequences of any other taxing jurisdiction relating to the ownership and disposition of warrants we are 
offering in light of their investment or tax circumstances. 
Taxation of Purchase Contracts 
The applicable supplement will contain a discussion of any special U.S. federal income tax considerations 
with respect to the acquisition, ownership and disposition of purchase contracts offered in this prospectus, including 
any tax considerations relating to the specific terms of the purchase contracts. Investors considering the purchase of 
purchase contracts we are offering should carefully examine the applicable supplement regarding the special U.S. 
federal income tax considerations, if any, of the acquisition, ownership and disposition of the purchase contracts. 
Investors should consult with their own tax advisors regarding the U.S. federal income tax consequences and 
the tax consequences of any other taxing jurisdiction relating to the ownership and disposition of the purchase 
contracts in light of their investment or tax circumstances. 
Taxation of Units 
The applicable supplement will contain a discussion of any special U.S. federal income tax considerations 
with respect to the acquisition, ownership and disposition of units that we are offering, including any tax 
considerations relating to the specific terms of the units. Investors considering the purchase of units that we are 
offering should carefully examine the applicable supplement regarding the special U.S. federal income tax 
consequences, if any, of the acquisition, ownership and disposition of the units. 
Investors should consult with their own tax advisors regarding the U.S. federal income tax consequences and 
the tax consequences of any other taxing jurisdiction relating to the ownership and disposition of units comprised of 
two or more of the securities we are offering in light of their investment or tax circumstances. 
Reportable Transactions
Applicable Treasury regulations require taxpayers that participate in “reportable transactions” to disclose their 
participation to the IRS by attaching Form 8886 to their U.S. federal tax returns and to retain a copy of all 
documents and records related to the transaction. In addition, “material advisors” with respect to such a transaction 
may be required to file returns and maintain records, including lists identifying investors in the transactions, and to 
furnish those records to the IRS upon demand. A transaction may be a “reportable transaction” based on any of 
several criteria, one or more of which may be present with respect to an investment in the securities that we are 
offering. Whether an investment in these securities constitutes a “reportable transaction” for any investor depends on 
the investor’s particular circumstances. The Treasury regulations provide that, 
84 
Table of Contents
in addition to certain other transactions, a “loss transaction” constitutes a “reportable transaction.” A “loss 
transaction” is any transaction resulting in the taxpayer claiming a loss under Section 165 of the Code, in an amount 
equal to or in excess of certain threshold amounts, subject to certain exceptions. The Treasury regulations 
specifically provide that a loss resulting from a “Section 988 transaction” will constitute a Section 165 loss, and 
certain exceptions will not be available if the loss from sale or exchange is treated as ordinary under Section 988. In 
general, certain securities issued in a foreign currency will be subject to the rules governing foreign currency 
exchange gain or loss. Therefore, losses realized with respect to such a security may constitute a Section 988 
transaction, and a holder of such a security that recognizes exchange loss in an amount that exceeds the loss 
threshold amount applicable to that holder may be required to file Form 8886. Investors should consult their own tax 
advisors concerning any possible disclosure obligation they may have with respect to their investment in the 
securities that we are offering and should be aware that, should any “material advisor” determine that the return 
filing or investor list maintenance requirements apply to such a transaction, they would be required to comply with 
these requirements. 
Foreign Account Tax Compliance Act 
The Foreign Account Tax Compliance Act, enacted on March 18, 2010, will impose a 30% U.S. withholding 
tax on certain U.S. source payments, including interest (and OID), dividends, other fixed or determinable annual or 
periodical gain, profits, and income, and on the gross proceeds from a disposition of property of a type which can 
produce U.S. source interest or dividends (“Withholdable Payments”), if paid to a foreign financial institution, 
unless such institution enters into an agreement with Treasury to collect and provide to Treasury substantial 
information regarding U.S. account holders, including certain account holders that are foreign entities with U.S. 
owners, with such institution. The legislation also generally imposes a withholding tax of 30% on Withholdable 
Payments made to a non-financial foreign entity unless such entity provides the withholding agent with a 
certification that it does not have any substantial U.S. owners or a certification identifying the direct and indirect 
substantial U.S. owners of the entity. 
In addition, under the Foreign Account Tax Compliance Act, “passthru payments” made by a foreign financial 
institution to “recalcitrant holders” or non-compliant foreign financial institutions are subject to a 30% U.S. 
withholding tax. A “recalcitrant holder” generally is a holder of an account with a foreign financial institution that 
fails to comply with reasonable requests for information that will help enable the relevant foreign financial 
institution to comply with its reporting requirements. A “passthru payment” is any Withholdable Payment or other 
payment (including non-U.S. source payments) to the extent attributable to any Withholdable Payment. The IRS has 
issued a notice indicating that a payment will be attributable to a Withholdable Payment to the extent of a percentage 
determined by dividing the sum of the foreign financial institution’s U.S. assets by the sum of the institution’s total 
assets, each as determined on certain testing dates. However, if the proposed Treasury regulations are finalized in 
their current form, a passthru payment will be any Withholdable Payment and any “foreign passthru payment,” 
which has yet to be defined. 
These withholding and reporting requirements will generally apply to payments made after December 31, 
2012; however, the withholding tax will not be imposed on payments pursuant to obligations outstanding as of 
March 18, 2012. Pursuant to proposed U.S. Treasury regulations, if finalized in their current form, the withholding 
tax will not be imposed on payments made under obligations outstanding on January 1, 2013. This exception from 
withholding does not apply to any preferred stock, depositary shares, and common stock. In addition, the IRS has 
issued a notice indicating that withholding under the Foreign Account Tax Compliance Act will begin no earlier 
than January 1, 2014. Holders are urged to consult with their own tax advisors regarding the 
85 
Table of Contents
possible implications of this recently enacted legislation on their investment in the debt securities, preferred stock, 
depositary shares, or common stock. 
EU DIRECTIVE ON THE TAXATION OF SAVINGS INCOME 
On July 1, 2005, a directive adopted by the European Union Council of Economic and Finance Ministers 
regarding the taxation of savings income payments came into effect. The directive obliges a member state of the 
European Union, (“EU”), to provide to the tax authorities of another EU member state details of payments of 
interest or other similar income payments made by a person (such as an issuer or paying agent) within its jurisdiction 
for the immediate benefit of an individual in that other EU member state (including certain payments secured for 
their benefit). However, Austria and Luxembourg have opted out of the above reporting requirements and are 
instead applying a special withholding tax for a transitional period in relation to such payments of interest. The 
withholding tax is currently imposed at the rate of 35%. Withholding tax is not applied if the individual presents a 
certificate in the required form from the tax authority of his or her EU member state of residence that confirms that 
the applicable tax authority is aware of the investment made abroad. This transitional period will terminate at the 
end of the first fiscal year following agreement by certain non-EU countries to the exchange of information relating 
to such payments. 
Also with effect from July 1, 2005, a number of non-EU countries and certain dependent or associated 
territories of EU member states have adopted similar measures (either provision of information or transitional 
withholding) in relation to payments of interest or other similar income payments made by a person in that 
jurisdiction for the immediate benefit of an individual or to certain non-corporate entities in any EU member state. 
The EU member states have entered into reciprocal provision of information or transactional special withholding tax 
arrangements with certain of those dependent or associated territories. These apply in the same way as payments by 
persons in any EU member state to individuals of another EU member state. 
On November 13, 2008, the European Commission proposed changes to the EU savings directive which 
extended its scope so that it applies to interest payments to certain intermediate persons or structures interposed 
between the person making the payment and the individual who is the beneficial owner of the interest. It is proposed 
that an EU member state intermediary that receives an interest payment be treated as a person making payment, so 
as to subject it to the exchange of information or withholding obligation in the EU savings directive. Further, it is 
proposed that an interest payment made to an intermediary established outside the EU be treated as a payment made 
directly to the individual beneficiary if the person making the payment knows that the individual beneficiary is EU 
resident. 
86 
Table of Contents
PLAN OF DISTRIBUTION (CONFLICTS OF INTEREST) 
We may sell the securities offered under this prospectus: 
• through underwriters; 
• through dealers; 
• through agents; or 
• directly to purchasers. 
In addition, we may issue the securities as a dividend or distribution or in a subscription rights offering to our 
existing security holders. 
The underwriters, dealers, or agents may include Merrill Lynch, Pierce, Fenner & Smith Incorporated 
(“MLPF&S”), or any of our other affiliates. 
Each supplement relating to an offering of securities will state the terms of the offering, including: 
• the names of any underwriters, dealers, or agents; 
• the public offering or purchase price of the offered securities and the net proceeds that we will receive from 
the sale; 
• any underwriting discounts and commissions or other items constituting underwriters’ compensation; 
• any discounts, commissions, or fees allowed or paid to dealers or agents; and 
• any securities exchange on which the offered securities may be listed. 
Distribution Through Underwriters 
We may offer and sell securities from time to time to one or more underwriters who would purchase the 
securities as principal for resale to the public, either on a firm commitment or best efforts basis. If we sell securities 
to underwriters, we will execute an underwriting agreement with them at the time of the sale and will name them in 
the applicable supplement. In connection with these sales, the underwriters may be deemed to have received 
compensation from us in the form of underwriting discounts and commissions. The underwriters also may receive 
commissions from purchasers of securities for whom they may act as agent. Unless we specify otherwise in the 
applicable supplement, the underwriters will not be obligated to purchase the securities unless the conditions set 
forth in the underwriting agreement are satisfied, and if the underwriters purchase any of the securities, they will be 
required to purchase all of the offered securities. The underwriters may acquire the securities for their own account 
and may resell the securities from time to time in one or more transactions, including negotiated transactions, at a 
fixed public offering price or varying prices determined at the time of sale. The underwriters may sell the offered 
securities to or through dealers, and those dealers may receive discounts, concessions, or commissions from the 
underwriters as well as from the purchasers for whom they may act as agent. Any initial public offering price and 
any discounts or concessions allowed or reallowed or paid to dealers may be changed from time to time. 
87 
Table of Contents
Distribution Through Dealers 
We may offer and sell securities from time to time to one or more dealers who would purchase the securities 
as principal. The dealers then may resell the offered securities to the public at fixed or varying prices to be 
determined by those dealers at the time of resale. We will set forth the names of the dealers and the terms of the 
transaction in the applicable supplement. 
Distribution Through Agents 
We may offer and sell securities on a continuous basis through agents that become parties to an underwriting 
or distribution agreement. We will name any agent involved in the offer and sale, and describe any commissions 
payable by us in the applicable supplement. Unless we specify otherwise in the applicable supplement, the agent will 
be acting on a best efforts basis during the appointment period. 
Direct Sales 
We may sell directly to, and solicit offers from, institutional investors or others who may be deemed to be 
underwriters, as defined in the Securities Act of 1933, for any resale of the securities. We will describe the terms of 
any sales of this kind in the applicable supplement. 
General Information 
Underwriters, dealers, or agents participating in an offering of securities may be deemed to be underwriters, 
and any discounts and commissions received by them and any profit realized by them on resale of the offered 
securities for whom they act as agent, may be deemed to be underwriting discounts and commissions under the 
Securities Act of 1933. 
We may offer to sell securities either at a fixed price or at prices that may vary, at market prices prevailing at 
the time of sale, at prices related to prevailing market prices, or at negotiated prices. Securities may be sold in 
connection with a remarketing after their purchase by one or more firms including our affiliates, acting as principal 
for their own accounts or as our agent. 
In connection with an underwritten offering of the securities, the underwriters may engage in over-allotment, 
stabilizing transactions and syndicate covering transactions in accordance with Regulation M under the Securities 
Exchange Act of 1934. Over-allotment involves sales in excess of the offering size, which creates a short position 
for the underwriters. The underwriters may enter bids for, and purchase, securities in the open market in order to 
stabilize the price of the securities. Syndicate covering transactions involve purchases of the securities in the open 
market after the distribution has been completed in order to cover short positions. In addition, the underwriting 
syndicate may reclaim selling concessions allowed to an underwriter or a dealer for distributing the securities in the 
offering if the syndicate repurchases previously distributed securities in transactions to cover syndicate short 
positions, in stabilization transactions, or otherwise. These activities may cause the price of the securities to be 
higher than it would otherwise be. Those activities, if commenced, may be discontinued at any time. 
Ordinarily, each issue of securities will be a new issue, and there will be no established trading market for any 
security other than our common stock prior to its original issue date. We may not list any particular series of 
securities on a securities exchange or quotation system. Any underwriters to whom or agents through whom the 
offered securities are sold for offering and sale may make a market in the offered securities. However, any 
underwriters or agents that make a 
88 
Table of Contents
market will not be obligated to do so and may stop doing so at any time without notice. We cannot assure you that 
there will be a liquid trading market for the offered securities. 
If we offer securities in a subscription rights offering to our existing security holders, we may enter into a 
standby underwriting agreement with dealers, acting as standby underwriters. We may pay the standby underwriters 
a commitment fee for the securities they commit to purchase on a standby basis. If we do not enter into a standby 
underwriting arrangement, we may retain a dealer-manager to manage a subscription rights offering for us. 
Under agreements entered into with us, underwriters and agents may be entitled to indemnification by us 
against certain civil liabilities, including liabilities under the Securities Act of 1933, or to contribution for payments 
the underwriters or agents may be required to make. 
Although we expect that delivery of securities generally will be made against payment on or about the third 
business day following the date of any contract for sale, we may specify a longer settlement cycle in the applicable 
supplement. Under Rule 15c6-1 of the Securities Exchange Act of 1934, trades in the secondary market generally 
are required to settle in three business days, unless the parties to a trade expressly agree otherwise. Accordingly, if 
we have specified a longer settlement cycle in the applicable supplement for an offering of securities, purchasers 
who wish to trade those securities on the date of the contract for sale, or on one or more of the next succeeding 
business days as we will specify in the applicable supplement, will be required, by virtue of the fact that those 
securities will settle in more than T+3, to specify an alternative settlement cycle at the time of the trade to prevent a 
failed settlement and should consult their own advisors in connection with that election. 
Market-Making Transactions by Affiliates 
Following the initial distribution of securities, our affiliates, including MLPF&S, may buy and sell the 
securities in secondary market transactions as part of their business as broker-dealers. Resales of this kind may occur 
in the open market or may be privately negotiated, at prevailing market prices at the time of resale or at related or 
negotiated prices. This prospectus and any related supplements may be used by one or more of our affiliates in 
connection with these market-making transactions to the extent permitted by applicable law. Our affiliates may act 
as principal or agent in these transactions. 
The aggregate initial offering price specified on the cover of the applicable supplement will relate to the initial 
offering of securities not yet issued as of the date of this prospectus. This amount does not include any securities to 
be sold in market-making transactions. The securities to be sold in market-making transactions include securities 
issued after the date of this prospectus. 
Information about the trade and settlement dates, as well as the purchase price, for a market-making 
transaction will be provided to the purchaser in a separate confirmation of sale. 
Unless we or our agent inform you in your confirmation of sale that the security is being purchased in its 
original offering and sale, you may assume that you are purchasing the security in a market-making transaction. 
Conflicts of Interest 
MLPF&S is our wholly-owned subsidiary, and unless otherwise set forth in the applicable supplement, we 
will receive the net proceeds of any offering in which MLPF&S participates as an underwriter, dealer or agent. The 
offer and sale of any securities by MLPF&S, or any of our other 
89 
Table of Contents
affiliates that is a member of the Financial Industry Regulatory Authority, Inc., or “FINRA,” will comply with the 
requirements of FINRA Rule 5121 regarding a FINRA member firm’s offer and sale of securities of an affiliate. As 
required by FINRA Rule 5121, any such offer and sale will not be made to any discretionary account without the 
prior approval of the customer. 
The maximum commission or discount to be received by any FINRA member or independent broker-dealer 
will not be greater than 8% of the initial gross proceeds from the sale of any security being sold. 
The underwriters, agents and their affiliates may engage in financial or other business transactions with us and 
our subsidiaries in the ordinary course of business. 
In addition, in the ordinary course of their business activities, one or more of the underwriters, dealers or 
agents and/or their respective affiliates, may make or hold a broad array of investments and actively trade debt and 
equity securities (or related derivative securities) and financial instruments (including bank loans) for their own 
account and for the accounts of their customers. These investments and securities activities may involve securities 
and/or instruments of ours or our affiliates. These underwriters, dealers, agents, or their affiliates, that have a lending 
relationship with us routinely hedge their credit exposure to us consistent with their customary risk management 
policies. Typically, these parties would hedge such exposure to us by entering into transactions which consist of 
either the purchase of credit default swaps or the creation of short positions in our securities, including potentially 
the securities offered hereby. Any such short positions could adversely affect future trading prices of the securities 
offered hereby. These broker-dealers or their affiliates may also make investment recommendations and/or publish 
or express independent research views in respect of such securities or financial instruments and may hold, or 
recommend to clients that they acquire, long and/or short positions in such securities and instruments. 
90 
Table of Contents
ERISA CONSIDERATIONS 
A fiduciary of a pension, profit-sharing or other employee benefit plan governed by the Employee Retirement 
Income Security Act of 1974, as amended (“ERISA”), should consider the fiduciary standards of ERISA in the 
context of the ERISA plan’s particular circumstances before authorizing an investment in the offered securities of 
Bank of America. Among other factors, the fiduciary should consider whether such an investment is in accordance 
with the documents governing the ERISA plan and whether the investment is appropriate for the ERISA plan in 
view of its overall investment policy and diversification of its portfolio. 
Certain provisions of ERISA and the Code, prohibit employee benefit plans (as defined in Section 3(3) of 
ERISA) that are subject to Title I of ERISA, plans described in Section 4975(e)(1) of the Code (including, without 
limitation, individual retirement accounts and Keogh Plans), and entities whose underlying assets include plan assets 
by reason of a plan’s investment in such entities (including, without limitation, as applicable, insurance company 
general accounts) (collectively, “plans”), from engaging in certain transactions involving “plan assets” with parties 
that are “parties in interest” under ERISA or “disqualified persons” under the Code with respect to the plan or entity 
(referred to as “prohibited transactions”). Governmental and other plans that are not subject to ERISA or to the Code 
may be subject to similar restrictions under state, federal or local law. Any employee benefit plan or other entity, to 
which such provisions of ERISA, the Code or similar law apply, proposing to acquire the offered securities should 
consult with its legal counsel. 
Each of Bank of America Corporation and certain of its affiliates may be considered a “party in interest” or a 
“disqualified person” with respect to many plans. As a result, a prohibited transaction may arise if the securities are 
acquired by or on behalf of a plan unless those securities are acquired and held pursuant to an available exemption. 
In addition, certain regulatory requirements applicable under ERISA could cause investments in certain 
offered securities by a plan (whether directly or indirectly) to be deemed to include not only the purchased securities 
but also an undivided interest in certain of the underlying assets of the relevant issuer. In the absence of an 
applicable exception to this general rule, the relevant issuer could be considered to hold a portion of the assets of the 
investing plan such that persons providing services in connection with such assets might be considered “parties in 
interest” or “disqualified persons” with respect to the investing plan and could be governed by the fiduciary 
responsibility provisions of Title I of ERISA and the prohibited transaction provisions referenced above. If this were 
the case, any discretionary actions undertaken by that person regarding those assets could be deemed to be a 
prohibited transaction under ERISA or the Code (e.g., the use of fiduciary authority or responsibility in 
circumstances under which that person has interests that may conflict with the interests of the investing plan and 
affect the exercise of that person’s best judgment as a fiduciary). Whether the underlying assets of an issuer of any 
offered securities would be considered to be the assets of any employee benefit plan investor will depend on the 
specific terms of such security, and a plan investor should look to the prospectus supplement for that particular 
security in order to make that determination. 
The U.S. Department of Labor has issued five prohibited transaction class exemptions (“PTCEs”) that may 
provide exemptive relief for direct or indirect prohibited transactions resulting from or occurring in connection with 
the purchase or holding of these securities. Those class exemptions are PTCE 96-23 (for certain transactions 
determined by in-house asset managers), PTCE 95-60 (for certain transactions involving insurance company general 
accounts), PTCE 91-38 (for certain transactions involving bank collective investment funds), PTCE 90-1 (for certain 
transactions involving insurance company separate accounts) and PTCE 84-14 (for certain 
91 
Table of Contents
transactions determined by independent qualified professional asset managers). In addition, ERISA Section 
408(b)(17) and Section 4975(d)(20) of the Code provide an exemption for the purchase and sale of securities and the 
related lending transactions, provided that neither the issuer of the securities nor any of its affiliates has or exercises 
any discretionary authority or control or renders any investment advice with respect to the assets of any plan 
involved in the transaction and provided further that the plan pays no more than adequate consideration in 
connection with the transaction (the so-called “Service Provider Exemption”). There can be no assurance that any of 
these class or statutory exemptions will be available with respect to transactions involving these securities. 
Accordingly, unless otherwise provided in connection with a particular offering of securities, offered 
securities may not be purchased, held or disposed of by any plan or any other person investing “plan assets” of any 
plan that is subject to the prohibited transaction rules of ERISA or Section 4975 of the Code or other similar law, 
unless one of the following exemptions (or a similar exemption or exception acceptable to us) applies to such 
purchase, holding, and disposition: the Service Provider Exemption, PTCE 96-23, PTCE 95-60, PTCE 91-38, PTCE 
90-1, or PTCE 84-14. 
Unless otherwise provided in connection with a particular offering of securities, any purchaser of the offered 
securities or any interest therein will be deemed to have represented and warranted to us on each day including the 
date of its purchase of the offered securities through and including the date of disposition of such offered securities 
that either: 
(a) it is not a plan subject to Title I of ERISA or Section 4975 of the Code and is not purchasing such 
securities or interest therein on behalf of, or with “plan assets” of, any such plan; 
(b) its purchase, holding, and disposition of such securities are not and will not be prohibited because they are 
exempted by the Service Provider Exemption or one or more of the following prohibited transaction 
exemptions: PTCE 96-23, 95-60, 91-38, 90-1 or 84-14; or 
(c) it is a governmental plan (as defined in section 3 of ERISA) or other plan that is not subject to the 
provisions of Title I of ERISA or Section 4975 of the Code and its purchase, holding, and disposition of 
such securities are not otherwise prohibited. 
In addition, any purchaser that is a plan or is acquiring the offered securities on behalf of a plan, including any 
fiduciary purchasing on behalf of a plan, will be deemed to have represented, in its corporate and its fiduciary 
capacity, by its purchase and holding of the offered securities that (a) neither we, the underwriter nor any of our 
respective affiliates (collectively the “Seller”) is a “fiduciary” (under Section 3(21) of ERISA, or under any final or 
proposed regulations thereunder, or with respect to a governmental, church, or foreign plan under any similar laws) 
with respect to the acquisition, holding or disposition of the offered securities, or as a result of any exercise by the 
Seller of any rights in connection with the offered securities, (b) no advice provided by the Seller has formed a 
primary basis for any investment decision by or on behalf of such purchaser in connection with the offered securities 
and the transactions contemplated with respect to the securities, and (c) such purchaser recognizes and agrees that 
any communication from the Seller to the purchaser with respect to the offered securities is not intended by the 
Seller to be impartial investment advice and is rendered in its capacity as a seller of such offered securities and not a 
fiduciary to such purchaser. 
Due to the complexity of these rules and the penalties imposed upon persons involved in prohibited 
transactions, it is important that any person considering the purchase of the offered securities with plan assets 
consult with its counsel regarding the consequences under ERISA and the 
92 
Table of Contents
Code, or other similar law, of the acquisition and ownership of offered securities and the availability of exemptive 
relief under the class exemptions listed above. The sale of the securities of Bank of America to a plan is in no respect 
a representation by Bank of America or the underwriters that such an investment meets all relevant legal 
requirements with respect to investments by plans generally or any particular plan, or that such an investment is 
appropriate for plans generally or any particular plan. 
WHERE YOU CAN FIND MORE INFORMATION 
We have filed a registration statement on Form S-3 with the SEC covering the securities to be offered and sold 
using this prospectus. You should refer to this registration statement and its exhibits for additional information about 
us. This prospectus summarizes material provisions of contracts and other documents that we refer you to. Because 
the prospectus may not contain all of the information that you may find important, you should review the full text of 
these documents, which we have included as exhibits to the registration statement. 
We file annual, quarterly, and special reports, proxy statements and other information with the SEC. You may 
read and copy any document that we file with the SEC at the Public Reference Room of the SEC at 100 F Street, 
N.E., Room 1580, Washington, D.C. 20549. You may obtain information on the operation of the Public Reference 
Room by calling the SEC at 1-800-SEC-0330. You also may inspect our filings over the Internet at the SEC’s 
website, www.sec.gov. The reports and other information we file with the SEC also are available at our website, 
www.bankofamerica.com. We have included the SEC’s web address and our web address as inactive textual 
references only. Except as specifically incorporated by reference into this prospectus, information on those websites 
is not part of this prospectus. 
You also can inspect reports and other information we file at the offices of The New York Stock Exchange 
LLC, 20 Broad Street, 17th Floor, New York, New York 10005. 
The SEC allows us to incorporate by reference the information we file with it. This means that: 
• incorporated documents are considered part of this prospectus; 
• we can disclose important information to you by referring you to those documents; and 
• information that we file with the SEC automatically will update and supersede this incorporated information 
and information in this prospectus. 
We incorporate by reference the documents listed below which were filed with the SEC under the Securities 
Exchange Act of 1934:
• our annual report on Form 10-K for the year ended December 31, 2011; 
• our current reports on Form 8-K or Form 8-K/A filed January 13, 2012, January 19, 2012, February 10, 
2012, and March 16, 2012 (in each case, other than information that is furnished but deemed not to have 
been filed); and 
• the description of our common stock which is contained in our registration statement filed under Section 12 
of the Securities Exchange Act of 1934, as updated by our current reports on Form 8-K filed April 20, 2009, 
February 24, 2010, May 3, 2010, and September 1, 2011 and any other amendment or report filed for the 
purpose of updating such description. 
93 
Table of Contents
We also incorporate by reference reports that we will file under Sections 13(a), 13(c), 14, and 15(d) of the 
Securities Exchange Act of 1934 on or after the date of this prospectus, but not any information that we may furnish 
but that is not deemed to be filed. 
You should assume that the information appearing in this prospectus is accurate only as of the date of this 
prospectus. Our business, financial position, and results of operations may have changed since that date. 
You may request a copy of any filings referred to above (excluding exhibits), at no cost, by contacting us at 
the following address: 
Bank of America Corporation 
Fixed Income Investor Relations 
100 North Tryon Street 
Charlotte, North Carolina 28255-0065 
1-866-607-1234 
FORWARD-LOOKING STATEMENTS 
We have included or incorporated by reference in this prospectus and the applicable supplements statements 
that may constitute “forward-looking statements” within the meaning of Section 27A of the Securities Act of 1933 
and Section 21E of the Securities Exchange Act of 1934. You may find these statements by looking for words such 
as “plan,” “believe,” “expect,” “intend,” “anticipate,” “estimate,” “project,” “potential,” “possible,” or other similar 
expressions, or future or conditional verbs such as “will,” “should,” “would,” and “could.” 
All forward-looking statements, by their nature, are subject to risks and uncertainties. Our actual results may 
differ materially from those set forth in our forward-looking statements. As a large, international financial services 
company, we face risks that are inherent in the businesses and market places in which we operate. Information 
regarding important factors that could cause our future financial performance to vary from that described in our 
forward-looking statements is contained in our annual report on Form 10-K for the year ended December 31, 2011, 
which is incorporated by reference in this prospectus, under the captions “Item 1A. Risk Factors,” and “Item 7. 
Management’s Discussion and Analysis of Financial Condition and Results of Operations,” as well as those 
discussed in our subsequent filings that are incorporated in this prospectus by reference. See “Where You Can Find 
More Information” above for information about how to obtain a copy of our annual report. 
You should not place undue reliance on any forward-looking statements, which speak only as of the dates they 
are made. 
All subsequent written and oral forward-looking statements attributable to us or any person on our behalf are 
expressly qualified in their entirety by the cautionary statements contained or referred to in this section. Except to 
the extent required by applicable law or regulation, we undertake no obligation to update these forward-looking 
statements to reflect events or circumstances after the date of this prospectus or to reflect the occurrence of 
unanticipated events. 
LEGAL MATTERS 
The legality of the securities being registered will be passed upon for us by McGuireWoods LLP, Charlotte, 
North Carolina, and for the underwriters or agents by Morrison & Foerster LLP, New York, New York. Certain U.S. 
federal income tax matters will be passed upon for Bank of America by Morrison & Foerster LLP, New York, New 
York, special tax counsel to Bank of America. McGuireWoods LLP regularly performs legal services for us. Some 
members of McGuireWoods LLP performing those legal services own shares of our common stock. 
94 
Table of Contents
EXPERTS 
The consolidated financial statements and management’s assessment of the effectiveness of internal control 
over financial reporting (which is included in the Report of Management on Internal Control Over Financial 
Reporting) incorporated in this prospectus by reference to our annual report on Form 10-K for the year ended 
December 31, 2011 have been so incorporated in reliance on the report of PricewaterhouseCoopers LLP, an 
independent registered public accounting firm, given on the authority of said firm as experts in auditing and 
accounting. 
95 
Table of Contents
You should rely only on the information incorporated by reference or provided in this prospectus 
supplement, the accompanying prospectus, any related product supplement, index supplement, and/or 
pricing supplement. We have not authorized anyone to provide you with different information. We are not 
offering the securities in any jurisdiction where the offer is not permitted. You should not assume that the 
information in this prospectus supplement and the accompanying prospectus is accurate as of any date other 
than the date on the front of this document. 
Our affiliates, including Merrill Lynch, Pierce, Fenner & Smith Incorporated, will deliver this prospectus 
supplement, the accompanying prospectus, and any related product supplement, index supplement, and/or 
pricing supplement for offers and sales in the secondary market. 
Medium-Term Notes, 
Series L 
PROSPECTUS SUPPLEMENT 
BofA Merrill Lynch"""

In [38]:
len(text)

417549

In [41]:
n = len(text)
chunk = n//100
documents = [Document(page_content=text[i*chunk: (i+1)*chunk]) for i in range(100)]
graph_documents = llm_transformer.convert_to_graph_documents(documents)
print(f"Nodes:{graph_documents[0].nodes}")
print(f"Relationships:{graph_documents[0].relationships}")

Nodes:[Node(id='Bank Of America Corporation', type='Organization'), Node(id='Medium-Term Notes, Series L', type='Financial_instrument'), Node(id='Merrill Lynch, Pierce, Fenner & Smith Incorporated', type='Organization'), Node(id='Securities And Exchange Commission', type='Organization'), Node(id='Federal Deposit Insurance Corporation', type='Organization'), Node(id='March 30, 2012', type='Date'), Node(id='Risk Factors', type='Concept'), Node(id='Rule 424(B)(5)', type='Regulation'), Node(id='Senior', type='Concept'), Node(id='Subordinated', type='Concept'), Node(id='Fixed Rates', type='Concept'), Node(id='Floating Rates', type='Concept'), Node(id='No Interest', type='Concept'), Node(id='Federal Funds Rate', type='Concept'), Node(id='Libor', type='Concept'), Node(id='Euribor', type='Concept'), Node(id='Prime Rate', type='Concept'), Node(id='Treasury Rate', type='Concept'), Node(id='Indexed Notes', type='Concept'), Node(id='Market Measures', type='Concept'), Node(id='U.S. Dollars', type='

In [43]:
graph.add_graph_documents(
  graph_documents, 
  baseEntityLabel=True, 
  include_source=True
)

Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('592d73fc.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))


In [44]:
vector_index = Neo4jVector.from_existing_graph(
    embeddings,
    search_type="hybrid",
    node_label="Document",
    text_node_properties=["text"],
    embedding_node_property="embedding"
)

Received notification from DBMS server: {severity: WARNING} {code: Neo.ClientNotification.Statement.FeatureDeprecationWarning} {category: DEPRECATION} {title: This feature is deprecated and will be removed in future versions.} {description: The query used a deprecated procedure. ('db.create.setVectorProperty' has been replaced by 'db.create.setNodeVectorProperty')} {position: line: 1, column: 70, offset: 69} for query: "UNWIND $data AS row MATCH (n:`Document`) WHERE elementId(n) = row.id CALL db.create.setVectorProperty(n, 'embedding', row.embedding) YIELD node RETURN count(*)"


In [45]:
qa_chain = RetrievalQA.from_chain_type(
    llm, retriever=vector_index.as_retriever()
)

result = qa_chain({"query": query})
print(result["result"])

Failed to write data to connection ResolvedIPv4Address(('34.126.171.25', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))
Failed to write data to connection IPv4Address(('592d73fc.databases.neo4j.io', 7687)) (ResolvedIPv4Address(('34.126.171.25', 7687)))


Based on the provided document, the required information is not explicitly available. Therefore, all fields will return null. Here is the markdown table format with the columns "Field" and "Value":

```markdown
| Field                    | Value |
|--------------------------|-------|
| ISIN                     | null  |
| Entity                   | null  |
| Deal Type                | null  |
| Seniority                | null  |
| COCO                     | null  |
| Pledge Status            | null  |
| Issue Price (%)          | null  |
| Face Value               | null  |
| Min Denomination/Subscription | null  |
| Issue Date               | null  |
| Principal Amount         | null  |
| Issue Currency           | null  |
| Scheduled Maturity Date  | null  |
| First Payment Date       | null  |
| Type                     | null  |
| Rate                     | null  |
| Base Index               | null  |
| Spread                   | null  |
| Day Count                | null  |
| First